In [3]:
## run-1
## download data sku dari tatanama
## common problems: slow server pyany & make sure path to file

import time
import pandas as pd
import numpy as np
import math

import smtplib

from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os


print('Reading Data SKU')
data_SKU = pd.read_excel('SKU_File\data_SKU.xlsx')
print('Scrapping Tatanama')
s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open('SKU_File\Master tatanama.xlsx', 'wb') as output:
    output.write(r.content)

print('ok')

if os.path.isfile('SKU_File\Master tatanama.xlsx') :    
    SKU_append = pd.read_excel('SKU_File\Master tatanama.xlsx')
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

data_SKU.loc[data_SKU['Price List NFI'].isin(['-']), 'Price List NFI'] = 0
to_excel = data_SKU.to_excel('SKU_File\data_SKU.xlsx', index = False)
print('data SKU done')

data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')

download = True

Reading Data SKU
Scrapping Tatanama
ok


AttributeError: 'DataFrame' object has no attribute 'append'

In [2]:
## run-2
## belum scrap dari forstok
download = False

In [2]:
## run-3
## scrap data forstok & blibli lmen

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

diff_date = 10
data_now = str(datetime.today().day) + ' ' + str(datetime.today().strftime("%B"))

print('Input Forstok Date')
start_date = datetime.today() - timedelta(days=diff_date)
end_date = datetime.today()
date = start_date.strftime('%Y-%m-%d') + '-' + end_date.strftime('%Y-%m-%d')
print(date)

options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')

if not download:
    print('Opening Chrome')
    driver = webdriver.Chrome()
    driver.maximize_window()
    actions = ActionChains(driver)
    driver.get("https://www.forstok.com/dashboard/users/login")
    print('Login Forstok')
    
    time.sleep(10)
    print('input email')
    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("timotius.giovandi@nutrifood.co.id")

    print('input password')
    password = driver.find_element(By.NAME, "password")
    password.send_keys("timo123")
    
    print('click')
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/form/input[3]").click()
    
    #time.sleep(30)
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #print('scroll down')
    time.sleep(10)
    datefield = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button')))
    time.sleep(20)
    #datef = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'///html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[5]/div/section/h2')))
    #driver.execute_script("return arguments[0].scrollIntoView(true);", datef)
    time.sleep(20)
    driver.execute_script("return arguments[0].scrollIntoView(true);", datefield)
    time.sleep(10)
    driver.find_element(By.XPATH, "/html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button").click()
    time.sleep(10)
    driver.find_element(By.XPATH, '//*[@id="exportSalesOrderForm"]/div/section/section/div[1]/section/article/article[1]/div/section/div/div[2]').click()
    #datefield.click()
    pilih=0
    while pilih < 3:    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==start_date.strftime('%B'):
                    # print(n_elem)
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==end_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if end_date.strftime('%B')==start_date.strftime('%B'):
                    text = '('+text+')'#[2]
                else :
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
            pilih=pilih+1
    driver.find_element(By.XPATH, '//*[@type="button" and contains(text(),"Apply")]').click()
    time.sleep(30)
    driver.find_element(By.XPATH, '/html/body/div[1]/section/section[2]/div/form/div/section/section/div[2]/aside[2]/button[2]').click()
    #WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Save')])"))).click()
    #driver.find_element(By.XPATH, '//span[text()="Sales Order v2"]').click()
    #driver.find_element(By.XPATH, '//span[text()="Sales Order v2"]').click()
    #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Export')])[2]"))).click()
    time.sleep(2)
    driver.quit()
   
    print('Forstok Downloaded')
    download = True

print('Download Data SKU')
#Import library

import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os

print('Waiting to Download Forstok')
s = requests.Session()
r = s.get("https://www.forstok.com/dashboard/users/login")
data = {'dashboard_user_email' : 'andra.miftah@nutrifood.co.id', "dashboard_user_password" : 'nutrimart1234'}
r = s.post("https://www.forstok.com/dashboard/users/login", data = data)
startdate = start_date-timedelta(days=1)
startdate1 = startdate.strftime("%Y-%m-%d")
enddate = end_date.strftime("%Y-%m-%d")

text = 'https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/Nutrifood-forstok-sales_orders-version_2-' + startdate1 + 'T17:00:00Z-' + enddate + 'T16:59:59Z.xlsx'
print(text)
while True:
   r = s.get(text)
   print('Waiting to Download Forstok')
  
   if r.status_code == requests.codes.ok:
       print(r.status_code)
       with open('Input Data/Forstok_new_input.xls', 'wb') as output:
           output.write(r.content)
       break
   else :
       time.sleep(60)
          
with open('Input Data/nutrifood--forstok-sales_orders-' + date + '.xls', 'wb') as output:
   output.write(r.content)
    
# Import data
start_time = time.time()
print("Import Data ====== 1/10")

data_forstok = pd.read_excel(r'Input Data/Forstok_new_input.xls')

#EMERGENCY DOWNLOAD IF MAINTAINANCE
# data_forstok = pd.read_excel(r"C:\Users\steven.nathanael\Downloads\Nutrifood-forstok-sales_orders-version_2-2023-06-27T17 00 00Z-2023-07-07T16 59 59Z.xlsx")
# data_forstok = data_forstok[['Order Date', 'Channel', 'Store', 'Sales Order ID',
#                             'Order Reference No.', 'Payment Status', 'Fulfillment Status', 'Payment Type',
#                             'Payment Date', 'Shipping Label Printed Date', 'Shipped Date', 'Delivered Date',
#                             'Cancelled Date', 'Return ID', 'Shipping Courier', 'Service Type', 'AWB', 'Customer Name',
#                             'Customer Email', 'Currency Code', 'Item Name', 'SKU Code', 'Bundle SKU Code', 'Barcode ID',
#                             'Warehouse Name', 'Warehouse Code', 'Quantity', 'Regular Price', 'Selling Price',
#                             'Sub Total', 'VAT', 'Shipping Fee (Non-cashless)', 'Voucher Seller', 'Platform Rebate',
#                             'Shipping Customer Name', 'Shipping Address1', 'Shipping Address2', 'Shipping City',
#                             'Shipping Zip', 'Shipping Province', 'Shipping Country', 'Shipping Phone', 'Order Note',
#                             'Invoice ID']]
data_forstok.rename(columns = {'Shipping Label Printed Date' : 'Printed Date',
                              'Shipping Date' : 'Fulfilled Date',
                              'SKU Code' : 'SKU',
                              'Shipping Fee (Non-cashless)' : 'Shipping',
                              'Voucher Seller' : 'Seller Voucher',
                              'Platform Rebate' : 'Channel Rebate',
                              'Order Note' : 'Note'}, inplace = True)

data_forstok_pure = data_forstok.copy()

data_forstok = data_forstok.dropna(how = 'all')

pd.options.mode.chained_assignment = None
# Forstok formatting
print("--- %s seconds ---" % (time.time() - start_time))
print("Formatting Data ====== 2/10")

if 'Unnamed: 35' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 35'], axis = 'columns')
if 'Unnamed: 36' in data_forstok:
    data_forstok = data_forstok.rename(columns={'Unnamed: 36' : 'Comment'})
if 'Unnamed: 37' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 37'], axis = 'columns')
if 'Unnamed: 38' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 38'], axis = 'columns')
if 'Unnamed: 39' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 39'], axis = 'columns')
if 'Unnamed: 40' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 40'], axis = 'columns')
if 'Unnamed: 41' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 41'], axis = 'columns')
    
data_forstok["Order Date"] = data_forstok["Order Date"].str.replace('WIB', '')
data_forstok["Payment Date"] = data_forstok["Payment Date"].str.replace('WIB', '')
#data_forstok["Cancelled Date"] = data_forstok["Cancelled Date"].str.replace('WIB', '')
data_forstok["Order Date"] = pd.to_datetime(data_forstok["Order Date"], errors = 'coerce')
data_forstok["Payment Date"] = pd.to_datetime(data_forstok["Payment Date"], errors = 'coerce')
data_forstok["Cancelled Date"] = pd.to_datetime(data_forstok["Cancelled Date"], errors = 'coerce')
data_forstok['Customer Name'] = data_forstok['Customer Name'].fillna(data_forstok['Shipping Customer Name'])
data_forstok = data_forstok.rename(columns = {'Seller Voucher' : 'Seller Discount'})
data_forstok = data_forstok[~((data_forstok['Store']=='Shopee')&(data_forstok['Order Date']<'21-12-2021'))] ###TMO###
print('Date formatted')

# Phone formatting

data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('=','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('"','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('(','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace(')','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('-','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('+','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^620','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^62','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^620','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^8','08', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^21','021', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^008','08', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('(021)','021', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('+62','0', regex = False)
print('Phone Formatted')
# Forstok SKU
print("--- %s seconds ---" % (time.time() - start_time))
print("Fulfilling SKU ====== 3/10")
indeks = data_forstok[data_forstok['Item Name'].astype(str) == 'Buy 1 Get 1 FREE Tropicana Slim Goldenmil Vanilla Manuka Honey (6 Sch)'][data_forstok[data_forstok['Item Name'].astype(str) == 'Buy 1 Get 1 FREE Tropicana Slim Goldenmil Vanilla Manuka Honey (6 Sch)']['SKU'] == 'PE8B27'].index.to_list()
data_forstok['SKU'][indeks] = '2101384106P2'
data_forstok.loc[data_forstok['SKU']=="(E)2101492P04",'SKU']="(E)2101492P4"
data_forstok.loc[data_forstok['SKU']=="P2101151180P2",'SKU']="PT7(2)B65"
data_forstok.loc[data_forstok['SKU']==' PT28(2)B65','SKU']='PT28(2)B65'
data_forstok.loc[data_forstok['SKU']==' PT6(2)B65','SKU']='PT6(2)B65'
data_forstok.loc[data_forstok['SKU']==' PT1(2)B65','SKU']='PT1(2)B65'
data_forstok.loc[(data_forstok['SKU']=='2101865450P3')&(data_forstok['Item Name'].astype(str)=='Tropicana Slim White Coffee 4 Sachet x 3 pcs - Kopi Susu Nikmat Tanpa Gula Pasir'),'SKU']='2104170164P3'
data_forstok.loc[(data_forstok['SKU']=='2104170104P3')&(data_forstok['Item Name'].astype(str)=='Triple Pack: Tropicana Slim White Coffee - Kopi Bebas Gula'),'SKU']='2104170164P3'
data_forstok.loc[(data_forstok['SKU']=='COLLETTEP24')&(data_forstok['Item Name'].astype(str)=='HiLo School Cotton Candy 200ml - Ready to Drink (24 Pcs)'),'SKU']='2HF1403250P24'
data_forstok.loc[(data_forstok['SKU']=='COLLETTEP4')&(data_forstok['Item Name'].astype(str)=='HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium'),'SKU']='2HF1403250P4'

skushopee = data_forstok[data_forstok['SKU'].astype(str).str.contains('(S)',regex = False)]
data_forstok = data_forstok[~data_forstok['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]

skushopee = skushopee.reset_index(drop = True)
data_forstok = data_forstok.reset_index(drop = True)

forstok_all_sku = pd.read_excel(r'SKU_File\forstok_all_sku.xlsx')
indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()

for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains(' - ')].index.to_list()

# Formatting double name
for i in indeks:
    if data_forstok['Item Name'][i].count(' - ') == 1 :
        if (data_forstok['Item Name'][i].split(' - ')[0] == data_forstok['Item Name'][i].split(' - ')[1]):
            data_forstok['Item Name'][i] = data_forstok['Item Name'][i].split(' - ')[0]
    elif data_forstok['Item Name'][i].count(' - ') > 1:
        temp = math.ceil(data_forstok['Item Name'][i].count(' - ')/2)
        itemname = ''
        duplicate = ''
        for j in range(temp):
            if j == 0:
                itemname = itemname + data_forstok['Item Name'][i].split(' - ')[j]
                duplicate = duplicate + data_forstok['Item Name'][i].split(' - ')[temp+j]
            else :
                itemname = itemname + ' ' +data_forstok['Item Name'][i].split(' - ')[j]
                duplicate = duplicate + ' ' +data_forstok['Item Name'][i].split(' - ')[temp+j]
        if itemname == duplicate:
            data_forstok['Item Name'][i] = itemname
        
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace(r' - $', '')
data_forstok["Item Name"] = data_forstok["Item Name"].str.replace('Special Promo by Nutrimart Serba 12 RIBU Varian:', '', regex=False)
data_forstok["Item Name"] = data_forstok["Item Name"].str.replace('Khusus Jabodetabek', '- Khusus Jabodetabek', regex=False)
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 1 Get F', 'Buy 1 Get 1 F')
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 1 Get H', 'Buy 1 Get 1 H')
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 12 FREE', 'Buy 12 FREE 12')

# Formatting SKU based on name
indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower().strip() in data_SKU['Nama Produk'].astype(str).str.lower().str.strip().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower().strip() == data_SKU['Nama Produk'].astype(str).str.lower().str.strip()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.tolist()

data_adasku = data_forstok[['Item Name', 'SKU']]
data_adasku = data_adasku[data_adasku['SKU'].notnull()]

data_nosku = data_forstok[['Item Name', 'SKU']]
data_nosku = data_nosku[data_nosku['SKU'].isnull()]

for i in indeks:
    if data_adasku['SKU'].loc[data_nosku['Item Name'][i] == data_adasku['Item Name']].size != 0:
        data_forstok['SKU'][i] = data_adasku['SKU'].loc[data_nosku['Item Name'][i] == data_adasku['Item Name']].values[0]

for i in indeks:
    if data_forstok['Item Name'][i] == 'Lokalate Kopi Durian 10s':
        data_forstok['SKU'][i] = '1101675318'
    elif data_forstok['Item Name'][i] == 'Nutrisari Madu Kurma Isi 16 Renceng X 10 Sachet Karton' or data_forstok['Item Name'][i] =='Nutrisari Madu Kurma Isi 16 Renceng X 10 SachetKarton': 
        data_forstok['SKU'][i] = 'PN30(16)'
    elif data_forstok['Item Name'][i] == 'L-Men Protein Bar Crunchy Chocolate Isi X12 (Exp Date:10-Apr-2019)':
        data_forstok['SKU'][i] = '2306592173'
    elif data_forstok['Item Name'][i] == 'FS Hilo Active Chocolate Minuman Kesehatan [750 gr]' or data_forstok['Item Name'][i] == 'FSHilo Active Chocolate Minuman Kesehatan [750 gr]':
        data_forstok['SKU'][i] = '2101452190'
    elif data_forstok['Item Name'][i] == 'FS L-Men Platinum Suplemen Kesehatan + Free Spider Bottle [800 g] Hitam' or data_forstok['Item Name'][i] == 'FSL-Men Platinum Suplemen Kesehatan + Free Spider Bottle [800 g] Hitam':
        data_forstok['SKU'][i] = '2305551288P1G26'
    elif data_forstok['Item Name'][i] == 'NutriSari Premium ala Jus Mangga':
        data_forstok['SKU'][i] = '1100534104'
    elif data_forstok['Item Name'][i] == 'Buy 1 Get 1 FREE Tropicana Slim Sweetener Honey (50 Sch) - FS':
        data_forstok['SKU'][i] = '2102501125P1G53'

indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

list_alias = []
list_alias_name = []
for colname in data_SKU.columns:
    if 'Alias SKU' in colname:
        list_alias.append(colname)
    if 'Alias Nama' in colname:
        list_alias_name.append(colname)


for i in indeks:
    for j in list_alias:
        if str(data_forstok['SKU'][i]) in data_SKU[j].astype(str).values:
            idx = data_SKU[str(data_forstok['SKU'][i]) == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(data_forstok['Item Name'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    data_forstok['SKU'][i] = data_SKU['SKU'][k]
                        
indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()

for i in indeks:
    for j in list_alias_name:
        if str(data_forstok['Item Name'][i]).lower() in data_SKU[j].astype(str).str.lower().values:
            data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU[j].astype(str).str.lower()].values[0]

indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()

for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]

indeks = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    if str(skushopee['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        skushopee['SKU'][i] = data_SKU['SKU'].loc[str(skushopee['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]
        skushopee['SKU'][i] = '(S)' + str(skushopee['SKU'][i])

indeks = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    for j in list_alias:
        if str(skushopee['SKU'][i]).replace('(S)','') in data_SKU[j].astype(str).values:
            idx = data_SKU[str(skushopee['SKU'][i]).replace('(S)','') == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(skushopee['Item Name'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    skushopee['SKU'][i] = data_SKU['SKU'][k]

indeks = data_forstok[data_forstok['SKU'].astype(str) == 'Gift Sosro'].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains('Fricella')].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains('Zaskia Mecca')].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

print("--- %s seconds ---" % (time.time() - start_time))
print("Listing SKU Missing ====== 4/10")

data_forstok = data_forstok.append(skushopee, ignore_index = True, sort = False)
data_forstok = data_forstok.reset_index(drop = True)

indeks = data_forstok[data_forstok['SKU'].astype(str) == '71210111'].index.to_list()
data_forstok['SKU'][indeks] = 'PN28N29N34(2)N53N54N55'

indeks = data_forstok[data_forstok['SKU'].astype(str) == '71210112'].index.to_list()
data_forstok['SKU'][indeks] = 'PN19N22N30N34(2)N53N55'

indeks = data_forstok[data_forstok['SKU'].astype(str) == '(B) 2101656'].index.to_list()
data_forstok['SKU'][indeks] = '(B)2101656'

indeks = data_forstok[data_forstok['Item Name'].astype(str) == 'PRE ORDER NutriSari NUTRI-C1000 40sch Suplemen Kesehatan Vit C 1000mg '].index.to_list()
data_forstok['SKU'][indeks] = '1102110453'

data_forstok.loc[data_forstok['Item Name']=="HiLo Active Caramel Latte 500gr - Susu Tinggi Kalsium",'SKU']="2101478180"
data_forstok.loc[data_forstok['Item Name']=="NutriSari Es Rujak 40 Sachet - Minuman Buah Vitamin C - Expired 4 Bulan",'SKU']="(E)1101984453"
data_forstok.loc[data_forstok['SKU']=='PH39(12)U8(12) ','SKU']='PH39(12)U8(12)'


shopee_con = pd.read_excel(r'data_supp\Shopee Converter.xlsx')
indeks = data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))][data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))]['Item Name'].astype(str).str.lower().isin(shopee_con['Product Name'].astype(str).str.lower())].index.to_list()
product = data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))][data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))]['Item Name'].astype(str).str.lower().isin(shopee_con['Product Name'].astype(str).str.lower())]['Item Name'].unique()

for i in product:
    indeks_shopee = data_forstok.iloc[indeks][data_forstok.iloc[indeks]['Item Name'].astype(str).str.lower() == str(i).lower()].index.to_list()
    data_forstok['SKU'][indeks_shopee] = shopee_con[shopee_con['Product Name'].astype(str).str.lower() == str(i).lower()]['Real SKU'].values[0]


to_excel = data_forstok.to_excel(r'forstok_new.xlsx', index = False)

skushopee = data_forstok[data_forstok['SKU'].astype(str).str.contains('(S)',regex = False)]
data_forstok = data_forstok[~data_forstok['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]

skushopee = skushopee.reset_index(drop = True)
data_forstok = data_forstok.reset_index(drop = True)

sku_exclude = ['5337754001-1623228299159-0',"TESSS"]
data_forstok = data_forstok[~data_forstok['SKU'].isin(sku_exclude)]

data_forstok.loc[data_forstok['Item Name']=="HiLo Teen Strawberry Milkshake 500 gram – Susu Tinggi Kalsium - Milkshake",'SKU']="2101683180"
data_forstok.loc[data_forstok['Item Name']=="Paket Minyak Hemat – Tropicana Slim Canola Oil & Tropicana Slim Sunflower Oil - Package",'SKU']="PT3T46"
data_forstok.loc[data_forstok['Item Name']=="Buy 2 Get 1 - L-Men Protein Crunch BBQ Beef (20gr) - Triplepack",'SKU']="2309005300P3"
data_forstok.loc[data_forstok['Item Name']=="Paket Kecap Sehat - Topicana Slim Kecap Manis & Tropicana Slim Kecap Asin - Package","SKU"]="PT26T44"
data_forstok.loc[(data_forstok['Item Name']=="(Free Gift) 3pcs L-Men Protein Crunch BBQ Beef 20g")&
                 (data_forstok['Store']=="Lazada"),"SKU"]="(B)2309005305P3"
data_forstok.loc[data_forstok['Item Name']=="WHS - Paket NutriSari Jeruk Peras, HiLo Es Ketan Hitam, dan HiLo Klepon Latte Powder Drink Refill 500 gram",'SKU']="PH81H82N60G187"
data_forstok.loc[data_forstok['Item Name']=="WHS - HiLo Milky Brown Sugar Ready to Drink 200ml (24 tetrapack) - Minuman Sumber Kalsium Rendah Lemak",'SKU']="2101941250P24"
data_forstok.loc[data_forstok['Item Name']=="WHS - Twin Pack - Tropicana Slim Shirataki Noodle: Mie Goreng [71 g]",'SKU']="2106023014"
data_forstok.loc[data_forstok['Item Name']=="WHS - HiLo School Chocolate Ready to Drink 200ml (4 tetrapack) - Susu Tinggi Kalsium Rendah Lemak",'SKU']="2101492P4"
data_forstok.loc[data_forstok['Item Name']=="WHS - HiLo Kacang Hijau Ready to Drink 200ml (24 tetrapack) - Minuman Sumber Kalsium Rendah Lemak",'SKU']="2101907250P24"
data_forstok.loc[data_forstok['Item Name']=="WHS NutriSari Es Rujak Jeruk Bali 40sch Minuman Buah Vitamin C Vitamin D",'SKU']="1101996453"
data_forstok.loc[data_forstok['Item Name']=="DISKON BOMBASTIS - Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) - Minuman Almond Ready to Drink Rasa Cokelat ",'SKU']="2T01402249"
data_forstok.loc[data_forstok['Item Name']=="DISKON BOMBASTIS Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) Minuman Almond Ready to Drink Rasa Cokelat ",'SKU']="2T01402249"
data_forstok.loc[data_forstok['Item Name']=="WHS NutriSari RTD Squeezed Orange Minuman Jeruk Peras Vitamin C 200 mL ",'SKU']="1N01402250P4"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Active Vanilla 500g x 2 pcs Susu Tinggi Kalsium Rendah Lemak",'SKU']="2101485180P2"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Teen Taro 500 gr Susu Tinggi Kalsium",'SKU']="2101609180"
data_forstok.loc[data_forstok['Item Name']=="Tropicana Slim Hampers Lebaran Idul Fitri Fancy Tote Bag",'SKU']="PT24T51T52T61T79G292G294"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Chocolate Avocado Ready to Drink 200ml (4 Tetrapack) Susu Siap Minum Tinggi Kalsium",'SKU']="(E)2HH1407250P4"
data_forstok.loc[data_forstok['Item Name']=="TG HiLo School Chocolate RTD 200ml",'SKU']="2101492"
data_forstok.loc[data_forstok['Item Name']=="TG HiLo School Chocolate RTD 200ml ",'SKU']="2101492"
data_forstok.loc[data_forstok['Item Name']=="TG Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) Minuman Almond Ready to Drink Rasa Cokelat ",'SKU']="2T01402249"
data_forstok.loc[data_forstok['Item Name']=="Tropicana Slim Oat Drink Japanese Cantaloupe Melon 190 ml (1 Pcs) Minuman Oat Vegan Plant Based Bebas Gula Ready to Drink",'SKU']="2T01403249"
data_forstok.loc[data_forstok['SKU']=="1101588454",'SKU']="1101588453"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Gold Biscuit Cereal 500g Susu Tinggi Kalsium Lebih Rendah Lemak",'SKU']="(E)2101524180"
data_forstok.loc[data_forstok['Item Name']=="Nutrifood BBF Mystery Box",'SKU']="PH100H108(2)T71"
data_forstok.loc[data_forstok['SKU']=="(E)2104406P1",'SKU']="(E)2104406"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Teen Chocolate Ready to Drink (4 pcs) Susu Tinggi Kalsium Rendah Lemak",'SKU']="(E)2101461P4"
data_forstok.loc[data_forstok['Item Name']=="WHS Paket HiLo School HiLo School Vanilla Vegiberi 500g & Original 400g",'SKU']="PH4H102"
data_forstok.loc[data_forstok['Item Name']=="WHS Twin Pack HiLo Gold Biscuit Cereal 500g Susu Tinggi Kalsium Lebih Rendah Lemak",'SKU']="(E)2101524180P2"
data_forstok.loc[data_forstok['Item Name']=="WHS Paket School 3+ HiLo School 3+ Strawberry Pop 400g & School 3+ Soya Vanilla Malt 400g",'SKU']="PE229E230"
data_forstok.loc[data_forstok['Item Name']=="WHS HiLo Gold Susu Tinggi Kalsium Rendah Lemak Chocolate [500 g ]",'SKU']="(E)2101551180"
data_forstok.loc[data_forstok['Item Name']=="TG HiLo Protein Chocofit UHT 190 mL (1 pcs) – Susu Tinggi Protein dengan Collagen Rasa Cokelat",'SKU']="2HH1409249"

idx = []
idx = idx + data_forstok[data_forstok['SKU'].isnull()].drop_duplicates().index.to_list()
idx = idx + data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx = list(dict.fromkeys(idx))
idx_s = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_s = list(dict.fromkeys(idx_s))


for i in product:
    indeks_shopee = data_forstok.iloc[indeks][data_forstok.iloc[indeks]['Item Name'].astype(str).str.lower() == str(i).lower()].index.to_list()
    data_forstok['SKU'][indeks_shopee] = shopee_con[shopee_con['Product Name'].astype(str).str.lower() == str(i).lower()]['Real SKU'].values[0]

print("--- %s seconds ---" % (time.time() - start_time))
if len(idx) != 0 or len(idx_s) != 0:
    alert = data_forstok.iloc[idx, ][['SKU', 'Item Name', 'Channel']].drop_duplicates()
    alert = alert.append(skushopee.iloc[idx_s][['SKU', 'Item Name', 'Channel', 'Selling Price']].drop_duplicates(), ignore_index = True, sort = False)
    alert['SKU'] = alert['SKU'].astype(str)
    alert['SKU Valid'] = np.nan
    to_excel = alert.to_excel('ALERT_FORSTOK_SKU_MISSING.xlsx')
    print("Some SKU Missing Please Complete It ====== 5/10")
    print("--- %s seconds ---" % (time.time() - start_time))
    
    # creates SMTP session 
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login("automationnfi@gmail.com", "nutrifood2020") 

    msg = MIMEMultipart()
    msg['Subject'] = "ALERT SKU FORSTOK MISSING"


    html = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(alert.to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    # sending the mail 
    s.sendmail("automationnfi@gmail.com", "steven.nathanael@nutrifood.co.id", msg.as_string()) 

    # terminating the session 
    s.quit() 
else :
    print("Filling Brand ====== 5/10")
    data_forstok['SKU'] = data_forstok['SKU'].astype(str)
    data_forstok['Item Name'] = data_forstok['Item Name'].astype(str)
    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str).str.replace('(S)', '', regex = False)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
    
    data_forstok = data_forstok.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

    temp = data_forstok[data_forstok['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_forstok[data_forstok['Real SKU'].isnull()].index.to_list()
    data_forstok['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_forstok['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    temp = data_forstok[data_forstok['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
    temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_forstok[data_forstok['Real SKU'].isnull()].index.to_list()
    data_forstok['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_forstok['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    data_forstok['Real SKU'] = data_forstok['Real SKU'].astype(str)
    data_forstok = data_forstok.merge(data_SKU[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_forstok = data_forstok.drop(['SKU_y'], axis = 1)
    data_forstok = data_forstok.rename(columns = {'SKU_x':'SKU'})

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Unbundling ====== 6/10")        
    # Forstok Unbundling    
    list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
    data_forstok = data_forstok.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    list_pcs = [x for x in data_forstok.columns if 'PCS' in x]
    for i in list_pcs:
        data_forstok[i] = data_forstok[i] * data_forstok['Quantity']
    data_forstok = data_forstok.drop(['SKU_y'], axis = 1)
    data_forstok = data_forstok.rename(columns = {'SKU_x':'SKU'})

    indeks = data_forstok[data_forstok['Brand'] == 'Bundle'].index.to_list()
    data_forstok['Bundle Flag'] = np.nan
    data_forstok['Bundle Flag'][indeks] = 'Bundle'

    indeks = data_forstok[data_forstok['Brand'] == 'Bundle'][data_forstok[data_forstok['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
    data_forstok['SKU Produk 1'][indeks] = '(S)' + data_forstok['SKU Produk 1'][indeks].astype(str)
    data_forstok['SKU Produk 2'][indeks] = '(S)' + data_forstok['SKU Produk 2'][indeks].astype(str)
    data_forstok['SKU Produk 3'][indeks] = '(S)' + data_forstok['SKU Produk 3'][indeks].astype(str)
    data_forstok['SKU Produk 4'][indeks] = '(S)' + data_forstok['SKU Produk 4'][indeks].astype(str)
    data_forstok['SKU Produk 5'][indeks] = '(S)' + data_forstok['SKU Produk 5'][indeks].astype(str)
    data_forstok['SKU Produk 6'][indeks] = '(S)' + data_forstok['SKU Produk 6'][indeks].astype(str)
    data_forstok['SKU Produk 7'][indeks] = '(S)' + data_forstok['SKU Produk 7'][indeks].astype(str)

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Filling Date ====== 7/10")
    data_forstok['Date'] = np.nan
    data_forstok['Month'] = np.nan
    data_forstok['Year'] = np.nan

    for i in range(data_forstok.shape[0]):
        if int(data_forstok['Order Date'][i].strftime('%d')) <= 12:
            data_forstok['Date'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
            data_forstok['Month'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            data_forstok['Year'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            data_forstok['Date'][i] = pd.to_datetime(data_forstok['Order Date'][i]).day
            data_forstok['Month'][i] = pd.to_datetime(data_forstok['Order Date'][i]).month_name()
            data_forstok['Year'][i] = pd.to_datetime(data_forstok['Order Date'][i]).year

    quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
            ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
    data_forstok = data_forstok.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
    data_forstok = data_forstok.drop(['Bulan'], axis = 1)
    data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
            {'Bulan' : 'January' , 'Number': 1},
            {'Bulan' : 'February' , 'Number': 2},
            {'Bulan' : 'March' , 'Number': 3},
            {'Bulan' : 'April' , 'Number': 4},
            {'Bulan' : 'May' , 'Number': 5},
            {'Bulan' : 'June', 'Number': 6},
            {'Bulan' : 'July' , 'Number': 7},
            {'Bulan' : 'August', 'Number' : 8},
            {'Bulan' : 'September', 'Number' : 9},
            {'Bulan' : 'October' , 'Number': 10},
            {'Bulan' : 'November' , 'Number': 11}])
    temp = data_forstok.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    data_forstok['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).strftime("%U")
    temp['Hour'] = pd.to_datetime(data_forstok['Order Date']).dt.hour
    temp['Minute'] = pd.to_datetime(data_forstok['Order Date']).dt.minute
    temp['Second'] = pd.to_datetime(data_forstok['Order Date']).dt.second
    data_forstok['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])
    
    
    forstok_all = data_forstok
    forstok_all['Total'] = forstok_all['Sub Total']
    forstok_all['Price List NFI'] = np.nan
    forstok_all['Total Net'] = np.nan

    forstok_all = forstok_all.rename(columns={'Order Reference No.' : 'Order #',
                                            'Fulfillment Status' : 'Order Status',
                                            'Order Date' : 'Order date',
                                            'Item Name' :'Product Name',
                                            'Bundle Name' : 'Bundle',
                                            'Shipping Country' : 'Country',
                                            'Shipping Province' : 'Region',
                                            'Shipping City' : 'City',
                                            'Shipping Zip' : 'Zip Code',
                                            'Shipping Address1' : 'Address',
                                            'Shipping Phone' : 'Phone',
                                            'Quantity' : 'Qty. Invoiced',
                                            'Sub Total' : 'Subtotal'})
    forstok_all['Kecamatan'] = np.nan
    forstok_all['Kelurahan'] = np.nan
    forstok_all['Store'] = forstok_all['Channel']

    print("Filling Location ===== 7/10")
    indeks = forstok_all[forstok_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kecamatan'][indeks] = forstok_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
        forstok_all['City'][indeks] = forstok_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kelurahan'][indeks] = forstok_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
        forstok_all['Kecamatan'][indeks] = forstok_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kecamatan'][indeks] = forstok_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
        forstok_all['City'][indeks] = forstok_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kelurahan'][indeks] = forstok_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
        forstok_all['Kecamatan'][indeks] = forstok_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

    forstok_all['City'] = forstok_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

    master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
    master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
    master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
    master_map['Kode Prov'][515] = 14
    master_map['Province'][515] = 'Riau'
    master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
    master_map['Province'] = master_map['Province'].str.title()
    master_map['City'] = master_map['City'].str.title()

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = forstok_all.copy()
    temp['City'] = temp['City'].astype(str).str.lower()
    temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    forstok_all['City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = forstok_all.copy()
    temp['Region'] = temp['Region'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    forstok_all['Region'][indeks] = temp['Real Province'][indeks]

    temp = forstok_all.copy()
    temp = temp[temp['Region'].isnull()]
    temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
    forstok_all['Region'][temp.index] = temp['Region']  
    
    district = pd.read_excel(r'All Data/list_district.xlsx')
    temp = forstok_all.copy()
    temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
    district['All District'] = district['All District'].astype(str).str.lower()
    temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
    indeks = temp[temp['Real District'].notnull()].index.to_list()
    forstok_all['Kecamatan'][indeks] = temp['Real District'][indeks]

    temp = forstok_all.copy()
    temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
    indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
    forstok_all['City'][indeks] = np.nan

    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

    print("Unbundling ===== 9/10")
    data_bundle1 = forstok_all[~forstok_all['Produk 1'].isnull()]
    data_bundle1['Bundle Name'] = data_bundle1['Product Name']
    data_bundle1['Bundle SKU'] = data_bundle1['SKU']
    data_bundle1['Product Name'] = data_bundle1['Produk 1']
    data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
    data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
    data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
    data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
    data_bundle1['Bundle Flag'] = np.nan

    data_bundle2 = forstok_all[~forstok_all['Produk 2'].isnull()]
    data_bundle2['Bundle Name'] = data_bundle2['Product Name']
    data_bundle2['Bundle SKU'] = data_bundle2['SKU']
    data_bundle2['Product Name'] = data_bundle2['Produk 2']
    data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
    data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
    data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
    data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
    data_bundle2['Bundle Flag'] = np.nan

    data_bundle3 = forstok_all[~forstok_all['Produk 3'].isnull()]
    data_bundle3['Bundle Name'] = data_bundle3['Product Name']
    data_bundle3['Bundle SKU'] = data_bundle3['SKU']
    data_bundle3['Product Name'] = data_bundle3['Produk 3']
    data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
    data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
    data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
    data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
    data_bundle3['Bundle Flag'] = np.nan

    data_bundle4 = forstok_all[~forstok_all['Produk 4'].isnull()]
    data_bundle4['Bundle Name'] = data_bundle4['Product Name']
    data_bundle4['Bundle SKU'] = data_bundle4['SKU']
    data_bundle4['Product Name'] = data_bundle4['Produk 4']
    data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
    data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
    data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
    data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
    data_bundle4['Bundle Flag'] = np.nan

    data_bundle5 = forstok_all[~forstok_all['Produk 5'].isnull()]
    data_bundle5['Bundle Name'] = data_bundle5['Product Name']
    data_bundle5['Bundle SKU'] = data_bundle5['SKU']
    data_bundle5['Product Name'] = data_bundle5['Produk 5']
    data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
    data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
    data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
    data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
    data_bundle5['Bundle Flag'] = np.nan

    data_bundle6 = forstok_all[~forstok_all['Produk 6'].isnull()]
    data_bundle6['Bundle Name'] = data_bundle6['Product Name']
    data_bundle6['Product Name'] = data_bundle6['Produk 6']
    data_bundle6['Bundle SKU'] = data_bundle6['SKU']
    data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
    data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
    data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
    data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
    data_bundle6['Bundle Flag'] = np.nan

    data_bundle7 = forstok_all[~forstok_all['Produk 7'].isnull()]
    data_bundle7['Bundle Name'] = data_bundle7['Product Name']
    data_bundle7['Bundle SKU'] = data_bundle7['SKU']
    data_bundle7['Product Name'] = data_bundle7['Produk 7']
    data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
    data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
    data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
    data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
    data_bundle7['Bundle Flag'] = np.nan

    data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
    data_bundle['SKU'] = data_bundle['SKU'].astype(str)
    data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
    data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

    temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

    indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
    data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
    data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
    data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
    data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
    data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
    data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

    print("Pricing ===== 10/10")
    forstok_all = forstok_all.append(data_bundle, ignore_index = True, sort = False)
    
    forstok_all = forstok_all.merge(data_SKU[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(forstok_all.index)
    forstok_all['Price List NFI_x'] = forstok_all['Price List NFI_x'].fillna(forstok_all['Price List NFI_y'])
    forstok_all =  forstok_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
    forstok_all = forstok_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

    forstok_all['Price List NFI'] = pd.to_numeric(forstok_all['Price List NFI']).astype(int)
    forstok_all['Harga Cost'] = pd.to_numeric(forstok_all['Harga Cost']).astype(int)
    forstok_all['Qty. Invoiced'] = pd.to_numeric(forstok_all['Qty. Invoiced']).astype(int)
    
    forstok_all['Total Net'] = forstok_all['Price List NFI'] * forstok_all['Qty. Invoiced']
    forstok_all['Total Harga Cost'] = forstok_all['Harga Cost'] * forstok_all['Qty. Invoiced']
    
    forstok_all = forstok_all.reset_index(drop = True)
    forstok_all['Order #'] = forstok_all['Order #'].astype(str).str.replace('.0', '', regex = False)
    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get("https://seller.blibli.com/sign-in")
    time.sleep(40)

    username = driver.find_element(By.ID, "email")
    username.clear()
    username.send_keys("customer1@nutrimart.co.id")

    password = driver.find_element(By.ID, "password")
    password.send_keys("NutrimartNutrif00d")

    driver.find_element(By.ID, "sign-in").click()
    time.sleep(40)

    driver.get("https://seller.blibli.com/MTA/order/summary")
    time.sleep(30)
    WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div/div/div/div/div[4]/button'))).click()
    time.sleep(30)
    WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[1]/div/div/div[2]/div[2]/div[3]/button'))).click()
    time.sleep(10)
    driver.find_element(By.ID, "semuaFilter").click()

    driver.find_element(By.CLASS_NAME, "order-filter-input").click()
    start_date = datetime.today() - timedelta(days=14)
    end_date = datetime.today()
    date = start_date.strftime('%Y-%m-%d') + '-' + end_date.strftime('%Y-%m-%d')

    if start_date.month == end_date.month:
        text = "//div[@class='calendar left']//td[@class = 'available' or @class='weekend available'][text() = " + str(start_date.day) + "]"
        driver.find_element(By.XPATH, text).click()
        text = "//div[@class='calendar left']//td[contains(@class, 'today')][text() = " + str(end_date.day) + "]"
        driver.find_element(By.XPATH, text).click()
    else :
        driver.find_element(By.CSS_SELECTOR, "th.prev.available").click()
        text = "//div[@class='calendar left']//td[@class = 'available' or @class='weekend available'][text() = " + str(start_date.day) + "]"
        driver.find_element(By.XPATH, text).click()
        text = "//div[@class='calendar right']//td[contains(@class, 'today')][text() = " + str(end_date.day) + "]"
        driver.find_element(By.XPATH, text).click()
    driver.find_element(By.CSS_SELECTOR, 'button.btn.col-sm-3.btn-default.order-option-buttons.dropdown-toggle').click()
    driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/div/div/div[1]/div[3]/div[2]/ul").click()
    try:
        time.sleep(10)
        driver.find_element(By.CLASS_NAME, "btn-primary-mta").click()
        time.sleep(10)
    except NoSuchElementException:
        pass

    cond = False
    while not cond:
        time.sleep(30)
        driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
        print('click 1')
        time.sleep(20)
        driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
        print('click 2')
        time.sleep(20)
        driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
        print('click 3')

        time_now = str(datetime.now().hour) + ':' + str(datetime.now().minute)
        time.sleep(10)
        time_notif = driver.find_element(By.XPATH, '//*[@id="viewNotification"]/div[2]/div[2]/ul/li[1]/span[2]').text
        print(time_notif)
        time_notif1 = driver.find_element(By.CSS_SELECTOR, 'li.ng-scope.notification-unread').text[-5:]
        end = datetime.strptime(time_notif, '%H:%M').time()
        print(end)
        duration = int((datetime.today() - timedelta(hours=end.hour, minutes=end.minute)).strftime('%M'))
        print(duration)
        if duration < 40:
            time.sleep(1)
            driver.find_element(By.XPATH, '//ul[@class = "notification-item-container"]//li[@class = "ng-scope notification-unread"]').click()
            cond = True
        else :
            driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
            time.sleep(10)
            driver.find_element(By.CLASS_NAME, 'fa-bell-o').click()
            time.sleep(10)
            
    print('Blibli LMS Downloaded')     

   
    time.sleep(15)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    else:
        print("More than one file or no file downloaded")
    print(file_name)
    new_name = 'Blibli ' + str(date) + '.csv'
    if os.path.isfile('Input Data/' + str(new_name)) :
        os.remove('Input Data/' + str(new_name))
    os.rename('Input Data/' + str(file_name), 'Input Data/' + str(new_name))
    driver.quit()
    print("Import Data ====== 1/10")
    
    lmen = pd.read_csv(r'Input Data/' + str(new_name) , dayfirst = True)
    lmen = lmen.dropna(how = 'all')
    lmen_pure = lmen.copy()

    pd.options.mode.chained_assignment = None
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Formatting Data ====== 2/10")
    lmen['No. Order'] = lmen['No. Order'].astype(str).str.replace('^="','')
    lmen['No. Order'] = lmen['No. Order'].astype(str).str.replace('"$','')
    lmen['No. Order Item'] = lmen['No. Order Item'].astype(str).str.replace('^="','')
    lmen['No. Order Item'] = lmen['No. Order Item'].astype(str).str.replace('"$','')
    lmen['No. Awb'] = lmen['No. Awb'].astype(str).str.replace('^="','')
    lmen['No. Awb'] = lmen['No. Awb'].astype(str).str.replace('"$','')
    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('^="','')
    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('"$','')
    
    #INPUT SKU BLIBLI LMS ERROR
    lmen.loc[lmen['Nama Produk']=='Daily Protein Package - L-Men Daily Popcorn Caramel 250g & L-Men Protein Bar Crunchy Chocolate 12 Sachet','Merchant SKU']='PL19L23'
    lmen.loc[lmen['Nama Produk']=='[Tebus Murah] L-Men Lose Weight Mango Sticky Rice 300g - Suplemen Penurun Berat Badan Tinggi Whey Protein ','Merchant SKU']='2304576112'
    lmen.loc[lmen['Nama Produk']=='WHS - FREE PROTEIN CRUNCH - L-Men PlantProtein Ogura 216g - Suplemen Tinggi Protein Nabati Rendah Lemak Vegan Lactose Free','Merchant SKU']='PL24B102'
    lmen.loc[lmen['Nama Produk']=='WHS - L-Men Platinum Ketan Hitam 800g (25g protein) - Suplemen Tinggi Whey Protein Rendah Lemak','Merchant SKU']='2305545288'
    lmen.loc[lmen['Nama Produk']=='WHS- Paket High Protein Snack - L-Men Protein Bar & 2 pcs L-Men Protein Crunch','Merchant SKU']='PL18(2)L19'
    lmen.loc[lmen['Nama Produk']=='WHS - L-Men Protein Crunch BBQ Beef (20gr)','Merchant SKU']='2309005305'
    lmen.loc[lmen['Nama Produk'].isin(['FREE SPIDER BOTTLE ']),'Merchant SKU']='(B)71210026'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00307-00001']),'Merchant SKU']='(G)71310033'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00306-00001']),'Merchant SKU']='(G)71310032'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00311-00001']),'Merchant SKU']='2307061250P4'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00310-00001']),'Merchant SKU']='(E)2304576112P2'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00309-00001']),'Merchant SKU']='(E)2304576112'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00314-00001']),'Merchant SKU']='PL7(2)B102(2)'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00313-00001']),'Merchant SKU']='2304515112P2'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00319-00001']),'Merchant SKU']='(E)2305551288'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00318-00001']),'Merchant SKU']='(E)2305551288P2'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00322-00001']),'Merchant SKU']='(E)2LL1402249P24'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00323-00001']),'Merchant SKU']='PL3G128'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00324-00001']),'Merchant SKU']='PE66E80'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00325-00001']),'Merchant SKU']='(E)2305559288'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00320-00001']),'Merchant SKU']='(E)2LL1402249P4'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00330-00001']),'Merchant SKU']='(E)2305584288'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00332-00001']),'Merchant SKU']='(E)2305551288P3'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00333-00001']),'Merchant SKU']='(E)2301050106'
    lmen.loc[lmen['Blibli SKU'].isin(['LMS-60022-00335-00001']),'Merchant SKU']='(E)2304036151'
     
    for i in lmen[lmen['Merchant SKU']=='nan']['Blibli SKU'].unique():
        print(i)
        skunfi=lmen[(lmen['Blibli SKU'] == i)&(lmen['Merchant SKU']!='nan')].reset_index()['Merchant SKU'][0]
        lmen.loc[(lmen['Blibli SKU']==i)&(lmen['Merchant SKU']=='nan'),'Merchant SKU']=skunfi

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Listing SKU ====== 3/10")
    lmen_sku = pd.read_excel(r'SKU_File/Converter Paket Blibli.xlsx')

    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('\.0$','', regex = True)

    null_sku = lmen[lmen['Merchant SKU'] == 'nan'].copy()
    lmen = lmen[lmen['Merchant SKU'] != 'nan']

    null_sku['Nama Produk'] = null_sku['Nama Produk'].astype(str).str.strip()
    lmen_sku['Nama Produk'] = lmen_sku['Nama Produk'].astype(str).str.strip()
    null_sku['Merchant SKU'] = null_sku.merge(lmen_sku[['SKU Merchant', 'Nama Produk']].drop_duplicates(['Nama Produk']), how = 'left', on = 'Nama Produk').set_index(null_sku.index)['SKU Merchant']

    lmen = lmen.append(null_sku, ignore_index = True, sort = False)

    gaada_sku = lmen[~lmen['Merchant SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].copy()
    lmen = lmen[lmen['Merchant SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]

    gaada_sku['Nama Produk'] = gaada_sku['Nama Produk'].astype(str).str.strip()
    lmen_sku['Nama Produk'] = lmen_sku['Nama Produk'].astype(str).str.strip()
    gaada_sku['Merchant SKU'] = gaada_sku.merge(lmen_sku[['SKU Merchant', 'Nama Produk']].drop_duplicates(['Nama Produk']), how = 'left', on = 'Nama Produk').set_index(gaada_sku.index)['SKU Merchant']
    lmen = lmen.append(gaada_sku, ignore_index = True, sort = False)

    indeks = []
    indeks = indeks + lmen[lmen['Merchant SKU'].isnull()].index.to_list()
    indeks = indeks + lmen[~lmen['Merchant SKU'].astype(str).str.replace('\.0$','', regex = True).isin(data_SKU['SKU'].astype(str))].index.to_list()

    if len(indeks) != 0 :
        alert = lmen.iloc[indeks][['Blibli SKU','Merchant SKU', 'Nama Produk']].drop_duplicates()
        alert['SKU Valid'] = np.nan
        to_excel = alert.to_excel('ALERT_LMEN-STORE_SKU_MISSING.xlsx')
        print("Some SKU Missing Please Complete It ====== 4/10")
        print("--- %s seconds ---" % (time.time() - start_time))
    else :
        data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
        data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
        lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str)
        data_SKU.loc[data_SKU['Price List NFI'].isin(['-']),'Price List NFI'] = 0
        data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')
        lmen = lmen.merge(data_SKU[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'Merchant SKU', right_on = 'Real SKU')
        lmen['Total Net'] = lmen['Price List NFI'] * lmen['Total Barang']

        indeks = lmen[lmen['Brand'] == 'Bundle'].index.to_list()
        lmen['Bundle Flag'] = np.nan
        lmen['Bundle Flag'][indeks] = 'Bundle'

        lmen['Date'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.day
        lmen['Month'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.month_name()
        lmen['Year'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                    ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        lmen = lmen.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        lmen = lmen.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = lmen.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        lmen['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        lmen['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day']])

        lmen = lmen.rename(columns = {'No. Order' : 'Order #', 'No. Order Item' : 'No. Order Item L-Men Blibli',
                                    'Tanggal Order' : 'Order date', 'Nama Pemesan' : 'Customer Name',
                                    'Merchant SKU' : 'SKU', 'Nama Produk' : 'Product Name', 'Total Barang':'Qty. Invoiced',
                                    'Servis Logistik' : 'Shipping Courier', 'Nama Store' : 'Channel',
                                    'Toko/Gudang' : 'Warehouse Name', 'No. Awb' : 'AWB'
                                    })

        lmen['Selling Price'] = (lmen['Qty. Invoiced']-lmen['Diskon'])/lmen['Qty. Invoiced']
        lmen['Channel'] = 'L-Men Store Blibli'
        lmen['Store'] = 'Blibli'

        data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
        data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

        lmen['SKU'] = lmen['SKU'].astype(str)
        list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Cost 7')+1].to_list()
        lmen = lmen.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        lmen = lmen.drop(['SKU_y'], axis = 1)
        lmen = lmen.rename(columns = {'SKU_x':'SKU'})
        
        data_bundle1 = lmen[~lmen['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1'] * data_bundle1['Qty. Invoiced']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1'] * data_bundle1['Qty. Invoiced']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = lmen[~lmen['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2'] * data_bundle2['Qty. Invoiced']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] * data_bundle2['Qty. Invoiced']
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = lmen[~lmen['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']* data_bundle3['Qty. Invoiced']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] * data_bundle3['Qty. Invoiced']
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = lmen[~lmen['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']* data_bundle4['Qty. Invoiced']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] * data_bundle4['Qty. Invoiced']
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = lmen[~lmen['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']* data_bundle5['Qty. Invoiced']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5'] * data_bundle5['Qty. Invoiced']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = lmen[~lmen['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']* data_bundle6['Qty. Invoiced']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6'] * data_bundle6['Qty. Invoiced']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = lmen[~lmen['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']* data_bundle7['Qty. Invoiced']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7'] * data_bundle7['Qty. Invoiced']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        lmen = lmen.append(data_bundle, ignore_index = True, sort = False)
        lmen = lmen.merge(data_SKU[['Real SKU', 'Harga Cost']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')
        lmen['Total Harga Cost'] = pd.to_numeric(lmen['Harga Cost'], errors = 'coerce') * lmen['Qty. Invoiced']
        print(lmen[lmen['Harga Cost'] == 0][['SKU','Product Name']].drop_duplicates())
        lmen['Total'] = lmen['Selling Price'] * lmen['Qty. Invoiced']
        lmen['Subtotal'] = lmen['Selling Price'] * lmen['Qty. Invoiced']
        lmen = lmen.rename(columns = {'Real SKU_x' : 'Real SKU'})

        forstok_all = forstok_all.append(lmen, ignore_index = True, sort = False)

        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        forstok_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        forstok_all['Total'][temp.index] = temp['Total']
        forstok_all['Subtotal'][temp.index] = temp['Subtotal']
        forstok_all['Selling Price'][temp.index] = temp['Selling Price']
        

        forstok_all['Order #'] = forstok_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        forstok_all['Seller Discount'][temp.index] = temp['Seller Discount']

        temp = forstok_all[forstok_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = forstok_all.merge(temp_group, how = 'left', on = 'Order #').set_index(forstok_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        forstok_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        forstok_all['Shipping'][temp.index] = temp['Shipping']
        forstok_all['True datetime'] = pd.to_datetime(forstok_all['True datetime'])
        forstok_all['Promo'] = np.nan
        forstok_all['Discount MC'] = np.nan

print('DONE')

Input Forstok Date
2024-01-28-2024-02-07


NameError: name 'download' is not defined

NameError: name 'lmen' is not defined

In [20]:
lmen[lmen['Merchant SKU']=='nan']['Blibli SKU']

32    LMS-60022-00330-00001
Name: Blibli SKU, dtype: object

In [5]:
skushopee[['Item Name', 'SKU']].drop_duplicates().values
### ini harus null juga, jgn ada item masuk pake sku shopee (S)

array([], shape=(0, 2), dtype=object)

In [6]:
### opsional , if errror
### cek sku missing (belum ada di tatanama)
### to do: regist single/bundle/alias 

data_forstok[data_forstok['SKU'].isnull()]['Item Name'] .unique()
data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))][['Store','Item Name','SKU']].drop_duplicates().values#.unique()

array([], shape=(0, 3), dtype=object)

In [5]:
#Pre Run-4
#Belum scraping magento dan blibli
magento_scrape = False
magentoUser_scrape = False

In [ ]:
## Run-4
## scrap magento detail order blibli update order status
## kadang auto logout di blibli -> solusi: run partial / add code

from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

if not magento_scrape: 
    print("Starting Magento")

    # Download the file using Selenium here
    #service = Service(executable_path=r"C:\Users\steven.nathanael\Documents\Python\Data\2. Master Data Git\chromedriver.exe")
    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath(r"C:\Users\steven.nathanael\Documents\Python\Data\2. Master Data Git\Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    print('Opening Chrome')
    driver = webdriver.Chrome(options = options)
    driver.get("https://nutrimart.co.id/backoffice")
    print('Login Magento')
    username = driver.find_element(By.ID, "username")
    username.clear()
    username.send_keys("timotius")
    password = driver.find_element(By.ID, "login")
    password.send_keys("timo123")

    print('Go To Sales Form')
    time.sleep(5)
    before = os.listdir(os.getcwd() + '/Input Data')
    driver.find_element(By.XPATH, "//button[@class = 'action-login action-primary']").click()
    time.sleep(5)
    WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-magento-sales-sales"]'))).click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="menu-magento-sales-sales"]/div/ul/li[2]/ul/li[2]/div/ul/li[1]/a').click()
    driver.find_element(By.XPATH, '//*[@id="profile_id"]').click()
    driver.find_element(By.XPATH, '//*[@id="profile_id"]/optgroup[2]/option[2]').click()
    

    start_date = datetime.today() - timedelta(days=12)
    print(start_date)
    end_date = datetime.today()
    print(end_date)
    date = start_date.strftime('%d %b %Y') + '-' + end_date.strftime('%d %b %Y')

    print('Input Date')
    date_from = driver.find_element(By.NAME, 'daterange_from')
    date_from.clear()
    date_from.send_keys(start_date.strftime('%m/%d/%Y'))

    date_to = driver.find_element(By.NAME, 'daterange_to')
    date_to.clear()
    date_to.send_keys(end_date.strftime('%m/%d/%Y'))
    print('Download Magento')
    driver.find_element(By.XPATH, '//*[@id="export_button"]').click()

    time.sleep(120)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        magento_name = change.pop()
    else:
        print("More than one file or no file downloaded")
    #data_magento = pd.read_csv('Input Data\export_sales_detailed_report_320667.csv', sep = ',', index_col = False)
    data_magento = pd.read_csv('Input Data/' + magento_name, sep = ',', index_col = False)
    magento_scrape = True
else :
    #data_magento = pd.read_csv('Input Data\export_sales_detailed_report_321223.csv', sep = ',', index_col = False)
    data_magento = pd.read_csv('Input Data/' + magento_name, sep = ',', index_col = False)
    
if not magentoUser_scrape:
    before = os.listdir(os.getcwd() + '/Input Data')

    driver.find_element(By.XPATH, '//*[@id="profile_id"]').click()
    driver.find_element(By.XPATH, '//*[@id="profile_id"]/optgroup[2]/option[1]').click()

    start_date = datetime.today() - timedelta(days=14)
    end_date = datetime.today()
    date = start_date.strftime('%d %b %Y') + '-' + end_date.strftime('%d %b %Y')

    date_from = driver.find_element(By.NAME, 'daterange_from')
    date_from.clear()
    date_from.send_keys(start_date.strftime('%m/%d/%Y'))
    date_to = driver.find_element(By.NAME, 'daterange_to')
    date_to.clear()
    date_to.send_keys(end_date.strftime('%m/%d/%Y'))

    driver.find_element(By.XPATH, '//*[@id="export_button"]').click()
    time.sleep(60)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        magentouser_name = change.pop()
        print(file_name)
    else:
        print("More than one file or no file downloaded")
        print(change)
    driver.quit()

    print(file_name)
    
    #data_magentoUser2 = pd.read_csv(r"C:\Users\steven.nathanael\Documents\Python\Data\2. Master Data Git\Input Data\export_320691.csv", sep = ',', index_col = False)
    data_magentoUser2 = pd.read_csv('Input Data/' + magentouser_name, sep = ',', index_col = False)
    for i in range(data_magentoUser2.shape[1]):
        data_magentoUser2 = data_magentoUser2.rename(columns={ data_magentoUser2.columns[i] : data_magentoUser2.columns[i].replace("    "," ")})
    magentoUser_scrape = True
else :
    data_magentoUser2 = pd.read_csv('Input Data/' + magentouser_name, sep = ',', index_col = False)
    #data_magentoUser2 = pd.read_csv(r"C:\Users\steven.nathanael\Documents\Python\Data\2. Master Data Git\Input Data\export_321224.csv", sep = ',', index_col = False)
    for i in range(data_magentoUser2.shape[1]):
        data_magentoUser2 = data_magentoUser2.rename(columns={ data_magentoUser2.columns[i] : data_magentoUser2.columns[i].replace("    "," ")})

data_SKU = pd.read_excel(r"SKU_File\data_SKU.xlsx")

s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

to_excel = data_SKU.to_excel(r"SKU_File\data_SKU.xlsx", index = False)


# Import library
import time
import pandas as pd
import numpy as np
import datetime
import math
import os

# Import data
start_time = time.time()
print("Import Data ====== 1/10")

data_magentoUser = pd.read_excel('All Data\data_magentoUser_2020.xlsx', index_col = False)
data_magentoUser = data_magentoUser[~data_magentoUser['Sales Order Id'].astype(str).isin(data_magentoUser2['Sales Order Id'].astype(str))]
data_magentoUser = data_magentoUser.append(data_magentoUser2, ignore_index = True, sort = False)
magento_pure = data_magento.copy()

list_skumiss = []

# Formatting Magento
print("--- %s seconds ---" % (time.time() - start_time))
print("Formatting Data ====== 2/10")

pd.options.mode.chained_assignment = None
if set(['Qty. Ordered"', 'Qty. Shipped"']).issubset(data_magento.columns):
    data_magento = data_magento.rename(columns={'Qty. Ordered"' : 'Qty. Ordered', 
                                            'Qty. Shipped"' : 'Qty. Shipped'})
if "Manufacturer" in data_magento.columns:
    data_magento= data_magento.drop(["Manufacturer"], axis=1)
if "Item Cost" in data_magento.columns:
    data_magento= data_magento.drop(["Item Cost"], axis=1)
if "Package Name" in data_magento.columns:
    data_magento= data_magento.drop(["Package Name"], axis=1)
if "Tax Refunded" in data_magento.columns:
    data_magento= data_magento.drop(["Tax Refunded"], axis=1)
if "Catalog Name Rule" in data_magento.columns:
    data_magento= data_magento.drop(["Catalog Name Rule"], axis=1)
if "Order date" in data_magento.columns:
    data_magento["Order date"] = pd.to_datetime(data_magento["Order date"], errors = 'coerce')
if "Customer Email" in data_magento.columns:
    data_magento["Customer Email"] = data_magento["Customer Email"].replace(regex = r'"', value="")
if "Qty. Ordered" in data_magento.columns:
    data_magento['Qty. Ordered'] = data_magento['Qty. Ordered'].fillna(0)
    if "Qty. Invoiced" in data_magento.columns:
        data_magento['Qty. Invoiced'] = data_magento['Qty. Invoiced'].fillna(0)
    if "Qty. Shipped" in data_magento.columns:
        data_magento['Qty. Shipped'] = data_magento['Qty. Shipped'].fillna(0)
    if "Qty. Refunded" in data_magento.columns:
        data_magento['Qty. Refunded'] = data_magento['Qty. Refunded'].fillna(0)
    if "Item Price" in data_magento.columns:
        data_magento['Item Price'] = data_magento['Item Price'].fillna(0)
    if "Subtotal" in data_magento.columns:
        data_magento['Subtotal'] = data_magento['Subtotal'].fillna(0)
    if "Discounts" in data_magento.columns:
        data_magento['Discounts'] = data_magento['Discounts'].fillna(0)
    if "Invoiced" in data_magento.columns:
        data_magento['Invoiced'] = data_magento['Invoiced'].fillna(0)
    if "Tax Invoiced" in data_magento.columns:
        data_magento['Tax Invoiced'] = data_magento['Tax Invoiced'].fillna(0)
    if "Voucher Amount" in data_magento.columns:
        data_magento[data_magento['Voucher Amount'].astype(str).str.isdigit()]['Voucher Amount'] = data_magento[data_magento['Voucher Amount'].fillna(0).astype(str).str.isdigit()]['Voucher Amount'].astype(float).div(10000)
    if "Discount Poin Reward" in data_magento.columns:
        data_magento['Discount Poin Reward'] = data_magento['Discount Poin Reward'].fillna(0)
    if "Discount Product" in data_magento.columns:
        data_magento['Discount Product'] = data_magento['Discount Product'].fillna(0)
if "Tax" in data_magento.columns:
    data_magento['Tax'] = data_magento['Tax'].astype(np.float32)
if "Total" in data_magento.columns:
    data_magento['Total'] = data_magento['Total'].astype(np.float32)
if "Total incl. Tax" in data_magento.columns:
    data_magento['Total incl. Tax'] = pd.to_numeric(data_magento['Total incl. Tax'], errors = 'coerce').astype(np.float32)
if "Invoiced incl. Tax" in data_magento.columns:
    data_magento['Invoiced incl. Tax'] = data_magento['Invoiced incl. Tax'].astype(np.float32)
if "Refunded" in data_magento.columns:
    data_magento['Refunded'] = data_magento['Refunded'].astype(np.float32)
if "Refunded incl. Tax" in data_magento.columns:
    data_magento['Refunded incl. Tax'] = data_magento['Refunded incl. Tax'].astype(np.float32)
if "Ship Date" in data_magento.columns:
    data_magento['Ship Date'] = data_magento['Ship Date'].astype(str).str.replace('false', '')

# Customer Information Filling  
for i in range(data_magentoUser.shape[1]):
    data_magentoUser = data_magentoUser.rename(columns={ data_magentoUser.columns[i] : data_magentoUser.columns[i].replace("    "," ")})

for i in range(data_magento.shape[0]):
    if str(data_magento["Customer Group"][i]) == 'nan':
        if data_magento["Order #"][i] in data_magentoUser["Sales Order Id"].values:
            data_magento["Customer Name"][i] = data_magentoUser["Shipping Name"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Customer Group"][i] = "Not Logged In"
            data_magento["Country"][i] = data_magentoUser["Shipping Country"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Region"][i] = data_magentoUser["Shipping Province"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["City"][i] = data_magentoUser["Shipping City"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Zip Code"][i] = data_magentoUser["Shipping Zip"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Address"][i] = data_magentoUser["Shipping Address1"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Phone"][i] = data_magentoUser["Shipping Phone"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]

indeks = data_magento[data_magento['Phone'].isnull()].index.to_list()
data_magento['Phone Condition'] = np.nan
data_magento['Phone Condition'][indeks] = 'X'

data_magentoUser['Sales Order Id'] = data_magentoUser['Sales Order Id'].astype(str)
data_magento['Order #'] = data_magento['Order #'].astype(str)

temp2 = data_magento.merge(data_magentoUser[['Sales Order Id', 'AWB', 'Shipping Cost', 'Shipping Address1', 'Shipping City', 'Shipping Province', 'Shipping Phone', 'Shipping Courier']].drop_duplicates('Sales Order Id'), how = 'left', left_on = 'Order #', right_on = 'Sales Order Id').set_index(data_magento.index)

data_magento['Shipping'] = np.nan
data_magento['AWB'] = np.nan
data_magento['Shipping Courier'] = np.nan

data_magento['Shipping'][temp2.index] = temp2['Shipping Cost']
data_magento['AWB'][temp2.index] = temp2['AWB']
data_magento['Shipping Courier'][temp2.index] = temp2['Shipping Courier']


# Phone formatting
data_magento["Phone"] = data_magento["Phone"].fillna(0.0)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace(" ","")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("-","")
temp = data_magento["Phone"].astype(str).str.split(",", expand = True)
data_magento["Phone"] = temp[0]
temp = data_magento["Phone"].astype(str).str.split("/", expand = True)
data_magento["Phone"] = temp[0]
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^\+620", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^\+62", "0")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^620", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^8", "08", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("(021)", "021")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^21", "021", regex = True)


data_magento['Shipping Address1'] = np.nan
data_magento['Shipping City'] = np.nan
data_magento['Shipping Province'] = np.nan
data_magento['Shipping Phone'] = np.nan


data_magento["Shipping Phone"] = data_magento["Shipping Phone"].fillna(0.0)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace(" ","")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("-","")
temp = data_magento["Shipping Phone"].astype(str).str.split(",", expand = True)
data_magento["Shipping Phone"] = temp[0]
temp = data_magento["Shipping Phone"].astype(str).str.split("/", expand = True)
data_magento["Shipping Phone"] = temp[0]
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^\+620", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^\+62", "0")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^620", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^8", "08", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("(021)", "021")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^21", "021", regex = True)

# Master tatanama
print("--- %s seconds ---" % (time.time() - start_time))
print("Fulfilling SKU ====== 3/10")

data_magento['SKU'] = data_magento['SKU'].astype(str).str.split('-').str[0]
indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()

skuhd = data_magento[data_magento['SKU'].astype(str).str.contains('hd', case = False)]
skushopee = data_magento[data_magento['SKU'].astype(str).str.contains('(S)',regex = False)]
data_magento = data_magento[~data_magento['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]
data_magento = data_magento[~data_magento['SKU'].astype(str).isin(skuhd['SKU'].astype(str))]

skuhd = skuhd.reset_index(drop = True)
skushopee = skushopee.reset_index(drop = True)
data_magento = data_magento.reset_index(drop = True)

indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()
for i in indeks:
    if data_magento['Product Name'][i].lower() in data_SKU['Nama Produk'].str.lower().values:
        data_magento['SKU'][i] = data_SKU['SKU'].loc[data_SKU['Nama Produk'].str.lower() == str(data_magento['Product Name'][i]).lower()].values[0]

list_alias = []
list_alias_name = []
for colname in data_SKU.columns:
    if 'Alias SKU' in colname:
        list_alias.append(colname)
    if 'Alias Nama' in colname:
        list_alias_name.append(colname)

for i in indeks:
    for j in list_alias:
        if str(data_magento['SKU'][i]) in data_SKU[j].astype(str).values:
            idx = data_SKU[str(data_magento['SKU'][i]) == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(data_magento['SKU'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    data_magento['SKU'][i] = data_SKU['SKU'][k]

indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()

for i in indeks:
    for j in list_alias_name:
        if str(data_magento['Product Name'][i]).lower() in data_SKU[j].astype(str).str.lower().values:
            data_magento['SKU'][i] = data_SKU['SKU'].loc[str(data_magento['Product Name'][i]).lower() == data_SKU[j].astype(str).str.lower()].values[0]

data_magento = data_magento[data_magento['Product Name'].astype(str) != 'WRP Everyday Low Fat Milk Coklat 4 Pillow Bag']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Kitchen Vietnamese Coffee Latte (12 Sch)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Raspberry Pumpkin (24 pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Raspberry Pumpkin (1 pc)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Kitchen Thai Tea Latte (12 Sch)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Greek Yogurt Blueberry (24 Pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Greek Yogurt Blueberry']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Lychee Spinach (24 Pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Lychee Spinach (1 pc)']



data_magento['SKU'] = data_magento['SKU'].astype(str).str.replace('7300281P24', '7300281')

data_magento = data_magento[~data_magento['Product Name'].astype(str).str.contains('JANGAN DIORDER INI TESTING')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('7300851')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002006')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0302004002')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002005P24')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002005')]


print("--- %s seconds ---" % (time.time() - start_time))
print("Listing SKU Missing ====== 4/10")   
idx = data_magento[['SKU']][data_magento['SKU'].isnull()].drop_duplicates().index.to_list()
idx = idx + data_magento[['SKU', 'Product Name']][~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx = list(dict.fromkeys(idx))
idx_s = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_s = list(dict.fromkeys(idx_s))
idx_hd = skuhd[~skuhd['SKU'].astype(str).str.replace('hd','',case = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_hd = list(dict.fromkeys(idx_hd))

to_excel = data_magento.to_excel(r'magento_new.xls', index = False)
print("--- %s seconds ---" % (time.time() - start_time))
if len(idx) != 0 or len(idx_s) != 0 or len(idx_hd) != 0:
    print('SKU is missing')
    alert = data_magento.iloc[idx, ][['SKU', 'Product Name']].drop_duplicates()
    alert = alert.append(skushopee.iloc[idx_s][['SKU', 'Product Name']].drop_duplicates(), ignore_index = True, sort = False)
    alert['SKU Valid'] = np.nan
    to_excel = alert.to_excel('ALERT_MAGENTO_SKU_MISSING.xlsx')
    print("Some SKU Missing Please Complete It ====== 5/10")
    print("--- %s seconds ---" % (time.time() - start_time))
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login("automationnfi@gmail.com", "nutrifood2020") 

    msg = MIMEMultipart()
    msg['Subject'] = "ALERT SKU MAGENTO MISSING"


    html = """\
    <html>
    <head></head>
    <body>
        {0}
    </body>
    </html>
    """.format(alert.to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    # sending the mail 
    s.sendmail("automationnfi@gmail.com", "andra.miftah@nutrifood.co.id", msg.as_string()) 

    # terminating the session 
    s.quit()
else :
    print("Preparing Appending Magento to Masterdata")
    print("Filling Brand ====== 5/10")  
    data_magento = data_magento.append(skushopee, ignore_index = True, sort = False)
    data_magento = data_magento.reset_index(drop = True)

    data_magento['SKU'] = data_magento['SKU'].astype(str)
    data_magento['Product Name'] = data_magento['Product Name'].astype(str)
    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

    data_magento = data_magento.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

    temp = data_magento[data_magento['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_magento[data_magento['Real SKU'].isnull()].index.to_list()
    data_magento['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_magento['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    temp = data_magento[data_magento['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_magento[data_magento['Real SKU'].isnull()].index.to_list()
    data_magento['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_magento['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    data_magento['Real SKU'] = data_magento['Real SKU'].astype(str)
    data_magento = data_magento.merge(data_SKU[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_magento = data_magento.drop(['SKU_y'], axis = 1)
    data_magento = data_magento.rename(columns = {'SKU_x':'SKU'})

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Unbundling ====== 6/10")
    list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
    data_magento = data_magento.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_magento = data_magento.drop(['SKU_y'], axis = 1)
    data_magento = data_magento.rename(columns = {'SKU_x':'SKU'})

    indeks = data_magento[data_magento['Brand'] == 'Bundle'].index.to_list()
    data_magento['Bundle Flag'] = np.nan
    data_magento['Bundle Flag'][indeks] = 'Bundle'

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Filling Date ====== 7/10")
    data_magento['Date'] = np.nan
    data_magento['Month'] = np.nan
    data_magento['Year'] = np.nan

    for i in range(data_magento.shape[0]):
        if int(data_magento['Order date'][i].strftime('%d')) <= 12:
            data_magento['Date'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).day
            data_magento['Month'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            data_magento['Year'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            data_magento['Date'][i] = pd.to_datetime(data_magento['Order date'][i]).day
            data_magento['Month'][i] = pd.to_datetime(data_magento['Order date'][i]).month_name()
            data_magento['Year'][i] = pd.to_datetime(data_magento['Order date'][i]).year


    quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
            ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
    data_magento = data_magento.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
    data_magento = data_magento.drop(['Bulan'], axis = 1)
    data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
            {'Bulan' : 'January' , 'Number': 1},
            {'Bulan' : 'February' , 'Number': 2},
            {'Bulan' : 'March' , 'Number': 3},
            {'Bulan' : 'April' , 'Number': 4},
            {'Bulan' : 'May' , 'Number': 5},
            {'Bulan' : 'June', 'Number': 6},
            {'Bulan' : 'July' , 'Number': 7},
            {'Bulan' : 'August', 'Number' : 8},
            {'Bulan' : 'September', 'Number' : 9},
            {'Bulan' : 'October' , 'Number': 10},
            {'Bulan' : 'November' , 'Number': 11}])
    temp = data_magento.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    data_magento['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
    data_magento['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day']])
    data_magento['Channel'] = 'Nutrimart'
    data_magento['Store'] = 'Nutrimart'
    data_magento['Selling Price'] = data_magento['Item Price']
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Pricing")
    data_magento = data_magento.merge(data_SKU[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(data_magento.index)
    data_magento = data_magento.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})
    data_magento['Price List NFI'] = pd.to_numeric(data_magento['Price List NFI']).astype(int)
    data_magento['Harga Cost'] = pd.to_numeric(data_magento['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
    data_magento['Qty. Invoiced'] = pd.to_numeric(data_magento['Qty. Invoiced']).astype(int)
    data_magento['Total Net'] = data_magento['Price List NFI'] * data_magento['Qty. Invoiced']
    data_magento['Total Harga Cost'] = data_magento['Harga Cost'] * data_magento['Qty. Invoiced']

    data_magento = data_magento.drop('SKU_y', axis = 1)
    data_magento = data_magento.reset_index(drop = True)
    data_magento['Order #'] = data_magento['Order #'].astype(str).str.replace('.0', '', regex = False)

    list_bundle = data_magento[data_magento['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
    list_nobundle = data_magento[data_magento['Bundle Name'].notnull()]
    list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
    list_nobundle

    data_magento['Total'][list_nobundle.index] = list_nobundle['Total_y']
    data_magento['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

    temp = data_magento[data_magento['Bundle Name'].notnull()]
    temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
    temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
    temp['Selling Price'] = temp['Subtotal'] / temp['Qty. Invoiced']

    data_magento['Total'][temp.index] = temp['Total'].fillna(0).astype(int)
    data_magento['Subtotal'][temp.index] = temp['Subtotal'].fillna(0).astype(int)
    data_magento['Selling Price'][temp.index] = temp['Selling Price'].fillna(0).astype(int)

    print("Filling Location")
    data_magento['Kecamatan'] = np.nan
    data_magento['Kelurahan'] = np.nan

    indeks = data_magento[data_magento['City'].astype(str).str.contains('/')]['City'].index.to_list()
    if len(indeks)>0:
        data_magento['Kecamatan'][indeks] = data_magento['City'][indeks].str.split('/', n = 1,expand = True)[1]
        data_magento['City'][indeks] = data_magento['City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        data_magento['Kelurahan'][indeks] = data_magento['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
        data_magento['Kecamatan'][indeks] = data_magento['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['City'].astype(str).str.contains(',')]['City'].index.to_list()
    if len(indeks)>0:
        data_magento['Kecamatan'][indeks] = data_magento['City'][indeks].str.split(',', n = 1,expand = True)[1]
        data_magento['City'][indeks] = data_magento['City'][indeks].str.split(',', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        data_magento['Kelurahan'][indeks] = data_magento['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
        data_magento['Kecamatan'][indeks] = data_magento['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = data_magento.copy()
    temp['City'] = temp['City'].astype(str).str.lower()
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    data_magento['City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = data_magento.copy()
    temp['Region'] = temp['Region'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    data_magento['Region'][indeks] = temp['Real Province'][indeks]

    temp = data_magento.copy()
    temp = temp[temp['Region'].isnull()]
    temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
    data_magento['Region'][temp.index] = temp['Region']

    district = pd.read_excel(r'All Data/list_district.xlsx')
    temp = data_magento.copy()
    temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
    district['All District'] = district['All District'].astype(str).str.lower()
    temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
    indeks = temp[temp['Real District'].notnull()].index.to_list()
    data_magento['Kecamatan'][indeks] = temp['Real District'][indeks]

    temp = data_magento.copy()
    temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
    indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
    data_magento['City'][indeks] = np.nan

    indeks = data_magento[data_magento['Shipping City'].astype(str).str.contains('/')]['Shipping City'].index.to_list()
    if len(indeks)>0:
        data_magento['Shipping City'][indeks] = data_magento['Shipping City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Shipping City'].astype(str).str.contains(',')]['Shipping City'].index.to_list()
    if len(indeks)>0:
        data_magento['Shipping City'][indeks] = data_magento['Shipping City'][indeks].str.split(',', n = 1,expand = True)[0]

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = data_magento.copy()
    temp['Shipping City'] = temp['Shipping City'].astype(str).str.lower()
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'Shipping City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    data_magento['Shipping City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = data_magento.copy()
    temp['Shipping Province'] = temp['Shipping Province'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Shipping Province', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    data_magento['Shipping Province'][indeks] = temp['Real Province'][indeks]

    temp = data_magento.copy()
    temp = temp[temp['Shipping Province'].isnull()]
    temp['Shipping Province'] = temp.merge(master_map, how = 'left', left_on = 'Shipping City', right_on = 'City').set_index(temp.index)['Shipping Province']
    data_magento['Shipping Province'][temp.index] = temp['Shipping Province']

    temp = data_magento.copy()
    temp2 = temp[['Shipping Province', 'Shipping City']].merge(master_map, how = 'left', left_on = 'Shipping City', right_on = 'City')
    indeks = temp2[temp2['Shipping Province'] != temp2['Province']][temp2[temp2['Shipping Province'] != temp2['Province']]['Shipping City'].notnull()].index.to_list()
    data_magento['Shipping City'][indeks] = np.nan
    data_magento['Warehouse Name'] = 'Primary Warehouse'

    import os
    import glob

    list_of_files = glob.glob('D:\Masterdata\Clean Data\*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)

    data_all = pd.read_csv(latest_file, sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})
    #data_all = pd.read_csv(r"D:\Masterdata\Clean Data\data_all_3 November With Order Online.csv", sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})
    
    data_all = data_all[data_all['Store Type'] != 'Retail Online']

    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('="', '', regex = False)
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('"', '', regex = False)

    lmen_store = forstok_all[forstok_all['Channel'] == 'L-Men Store Blibli']

    lmen_store = lmen_store[~lmen_store['Order #'].astype(str).isin(data_all['Order #'].astype(str))]
    lmen_store['Sales Order ID'] = lmen_store['Sales Order ID'].fillna(lmen_store['No. Order Item L-Men Blibli'])
    ornum = lmen_store[['Order #', 'Sales Order ID']].drop_duplicates('Order #')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": str(os.getcwd())  + '/Input Data',
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get("https://seller.blibli.com/sign-in")
    time.sleep(30)


    username = driver.find_element(By.ID, "email")
    username.clear()
    username.send_keys("customer1@nutrimart.co.id")

    password = driver.find_element(By.ID, "password")
    password.send_keys("NutrimartNutrif00d")

    driver.find_element(By.ID, "sign-in").click()
    time.sleep(25)

    driver.get("https://seller.blibli.com/MTA/order/summary")
    WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn-primary-mta'))).click()
    driver.find_element(By.XPATH, '//*[@id="semuaFilter"]/span[1]').click()

    ornum['Phone'] = np.nan
    ornum['Email'] = np.nan
    ornum['Address'] = np.nan

    for i in ornum.index:
        orderNo = ornum['Order #'][i]
        orderItemNo = int(pd.to_numeric(ornum['Sales Order ID'][i]))
        link = "https://seller.blibli.com/MTA/neo/order/order-detail/" + "?orderNo=" + str(orderNo) + "&orderItemNo=" + str(orderItemNo)
        driver.get(link)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="customer-phone"]/div/span')))
        hp = driver.find_elements(By.XPATH, '//*[@id="customer-phone"]/div/span')[0].text
        if len(driver.find_elements(By.XPATH, '//*[@id="customer-email"]/span')) != 0:
            email = driver.find_elements(By.XPATH, '//*[@id="customer-email"]/span')[0].text
            ornum['Email'][i] = email
        address = driver.find_elements(By.XPATH, '//*[@id="alamat-pengiriman"]//div//p')[0].text
        ornum['Phone'][i] = hp
        ornum['Address'][i] = address

    driver.quit()
    if len(ornum) != 0:
        tes = ornum.copy()
        tes['Real Address'] = tes['Address'].str.split('\n', expand = True)[1].str.strip()
        tes['Kelurahan'] = tes['Address'].str.split('\n', expand = True)[3].str.split('-', expand = True)[0].str.strip()
        tes['Kecamatan'] = tes['Address'].str.split('\n', expand = True)[3].str.split('-', expand = True)[1].str.strip()
        tes['City'] = tes['Address'].str.split('\n', expand = True)[4].str.split(',', expand = True)[0].str.strip()
        tes['Region'] = tes['Address'].str.split('\n', expand = True)[4].str.split(',', expand = True)[1].str.strip()
        tes['Zip Code'] = tes['Address'].str.split('\n', expand = True)[5].str.strip()

        lmen_store['Order #'] = lmen_store['Order #'].astype(str)
        tes['Order #'] = tes['Order #'].astype(str)
        lmen_store = lmen_store.merge(tes[['Order #', 'Phone', 'Email', 'Real Address', 'Kelurahan', 'Kecamatan', 'City', 'Region', 'Zip Code']].drop_duplicates('Order #'), how = 'left', on = 'Order #')

        indeks = lmen_store[lmen_store['Phone_y'].notnull()].index.to_list()
        lmen_store['Phone_x'][indeks] = lmen_store['Phone_y'][indeks]
        lmen_store['Customer Email'][indeks] = lmen_store['Email'][indeks]
        lmen_store['Phone_x'][indeks] = lmen_store['Phone_y'][indeks]
        lmen_store['Region_x'][indeks] = lmen_store['Region_y'][indeks]
        lmen_store['City_x'][indeks] = lmen_store['City_y'][indeks]
        lmen_store['Kecamatan_x'][indeks] = lmen_store['Kecamatan_y'][indeks]
        lmen_store['Kelurahan_x'][indeks] = lmen_store['Kelurahan_y'][indeks]
        lmen_store['Zip Code_x'][indeks] = lmen_store['Zip Code_y'][indeks]
        lmen_store['Address'][indeks] = lmen_store['Real Address'][indeks]

        lmen_store = lmen_store.drop(['Phone_y', 'Email', 'Region_y', 'City_y', 'Kecamatan_y', 'Kelurahan_y', 'Real Address', 'Zip Code_y'], axis = 1)
        lmen_store = lmen_store.rename(columns = {'Phone_x' : 'Phone', 'Region_x' : 'Region', 'City_x' : 'City', 'Kecamatan_x' : 'Kecamatan', 'Kelurahan_x' : 'Kelurahan', 'Zip Code_x' : 'Zip Code'})

    forstok_all = forstok_all[forstok_all['Channel'] != 'L-Men Store Blibli']
    forstok_all = forstok_all.append(lmen_store, ignore_index = True, sort = False)

    indeks = data_all[data_all['Channel'] == 'L-Men Store Blibli'].index.to_list()
    data_all['Store'][indeks] = 'Blibli'

    print('Blibli Done')
    
    #UNBUNDLING (J)
    print('Unbundling Paket J')
    
    skuj = pd.read_excel(r"C:\Users\steven.nathanael\Documents\Python\Data\2. Master Data Git\data_supp\Rumus Realisasi.xlsx")
    skuj = skuj[['SKU','Nama Paket','SKU.1','Konversi ke UOM']]
    skuj.rename(columns = {'SKU' : 'Bundle SKU',
                           'Nama Paket' : 'Bundle Name',
                           'SKU.1' : 'Real SKU',
                           'Konversi ke UOM' : 'Qty Bundle'}, inplace = True)
    skuj['Real SKU'] = skuj['Real SKU'].astype(str)

    # data_SKU['Real SKU'] = data_SKU['SKU'].astype(str).str.replace('(S)', '', regex = False)
    # data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

    skuj2 = pd.merge(skuj,data_SKU[['Real SKU','Brand','Real Nama Produk','Price List NFI','Sub Brand', 'Parent Item', 'Parent SKU']], on = 'Real SKU', how = 'left')
    skuj2.rename(columns = {'Real SKU' : 'Real SKU.1',
                            'Brand' : 'Brand.1',
                            'Real Nama Produk' : 'Real Nama Produk.1',
                            'Price List NFI' : 'Price List NFI.1',
                            'Sub Brand' : 'Sub Brand.1',
                            'Parent Item' : 'Parent Item.1',
                            'Parent SKU' : 'Parent SKU.1'}, inplace = True)

    skuj1 = skuj['Bundle SKU'].unique()
    skuj3 = skuj['Bundle Name'].unique()

    tes = forstok_all[forstok_all['Real SKU'].isin(skuj1)]
    tes['Bundle SKU'] = tes['Real SKU']
    tes1 = pd.merge(tes, skuj2, on = 'Bundle SKU', how = 'left')

    tes1['Real SKU'] = tes1['Real SKU.1'].astype(str)
    tes1['Real Nama Produk'] = tes1['Real Nama Produk.1']
    tes1['Qty. Invoiced.1'] = tes1['Qty. Invoiced'] * tes1['Qty Bundle']
    tes1['Qty. Invoiced'] = pd.to_numeric(tes1['Qty. Invoiced.1'])
    tes1['Brand'] = tes1['Brand.1']
    tes1['Sub Brand'] = tes1['Sub Brand.1']
    tes1['Real Nama Produk'] = tes1['Real Nama Produk.1']
    tes1['Parent Item'] = tes1['Parent Item.1']
    tes1['Parent SKU'] = tes1['Parent SKU.1'].astype(str)
    tes1['Price List NFI'] = pd.to_numeric(tes1['Price List NFI.1'].astype(int))

    tes1.rename(columns = {'Bundle Name_x' : 'Bundle Name'}, inplace = True)
    tes1['Bundle Name'] = tes1['Bundle Name_y']
    tes1.drop(columns = ['Bundle SKU','Bundle Name_y','Real SKU.1','Real Nama Produk.1','Qty. Invoiced.1','Qty Bundle','Brand.1','Sub Brand.1','Real Nama Produk.1','Parent Item.1','Parent SKU.1','Price List NFI.1'], inplace = True)

    tes1['Total Net'] = tes1['Price List NFI'] * tes1['Qty. Invoiced']
    tes1['Harga Cost'] = tes1['Price List NFI']
    tes1['Total Harga Cost'] = tes1['Total Net']

    forstok_all.loc[forstok_all['Real SKU'].str.contains('J', na = False, case = False),'Brand']= 'Bundle'
    forstok_all.loc[forstok_all['Real SKU'].str.contains('J', na = False, case = False),'Sub Brand']= np.nan
    forstok_all.loc[forstok_all['Real SKU'].str.contains('J', na = False, case = False),'Bundle Flag']= 'Bundle J'
    forstok_all = forstok_all.append(tes1, ignore_index = True)
    
    print('Done')
    
    #GABUNG MASTERDATA
    data_all = data_all[~data_all['Order #'].astype(str).isin(forstok_all['Order #'].astype(str))]
    data_all = data_all[~data_all['Order #'].astype(str).isin(data_magento['Order #'].astype(str))]
    data_all = data_all.append(forstok_all, ignore_index = True, sort = False)
    data_all = data_all.append(data_magento, ignore_index = True, sort = False)
    data_all['Brand'] = data_all['Brand'].astype(str).str.replace('Tropicana Slim', 'TS')

    indeks = data_all[data_all['Qty. Invoiced'] == 0].index.to_list()
    data_all['Qty. Invoiced'][indeks] = data_all['Qty. Shipped'][indeks]
    data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]
    indeks = data_all[data_all['Qty. Invoiced'] == 0].index.to_list()
    data_all['Qty. Invoiced'][indeks] = data_all['Qty. Ordered'][indeks]
    data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

# #   Cleaning Phone Number
    import re 
    from re import sub            
    data_all['Phone'] = data_all['Phone'].astype(str).str.split(';', expand = True)[0]
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('.0','', regex = False)
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('+','', regex = False)
    non_phone = data_all[(data_all['Phone'].astype(str).str.startswith('62'))]['Phone'].unique()
    non_phone1 = data_all[(data_all['Phone'].astype(str).str.startswith('62'))][['Store','Phone']].drop_duplicates()
    for i in non_phone:
        index = data_all[data_all['Phone'].astype(str) == str(i)].index.to_list()
        phone = str(i).replace('62','0', 1)
        print(i, phone, sep='-')
        data_all['Phone'][index] = phone
      
    
    print('done')
    print("--- %s seconds ---" % (time.time() - start_time))
    data_all['Phone'] = data_all['Phone'].apply('="{}"'.format)
    data_all['Shipping Phone'] = data_all['Shipping Phone'].apply('="{}"'.format)
    indeks = data_all[data_all['City'] == 'nan'].index.to_list()
    data_all['City'][indeks] = np.nan

    indeks = data_all[data_all['Region'] == 'nan'].index.to_list()
    data_all['Region'][indeks] = np.nan

    indeks = data_all[data_all['Channel'] == 'FBL'].index.to_list()
    data_all['Warehouse Name'][indeks] = 'Lazada Warehouse'

    indeks = data_all[data_all['Warehouse Name'] == 'Lazada Warehouse'].index.to_list()
    data_all['Channel'][indeks] = 'FBL'

    print("Read Data Settlement")
    
    #EDIT KHUSUS PERIOD FORSTOK
    from datetime import datetime, timedelta
    start_date = datetime.today() - timedelta(days=12)
    end_date = datetime.today()
    
    print('Opening Chrome')
    driver = webdriver.Chrome()
    driver.maximize_window()
    actions = ActionChains(driver)
    driver.get("https://www.forstok.com/dashboard/users/login")
    print('Login Forstok')
    
    time.sleep(10)
    print('input email')
    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("timotius.giovandi@nutrifood.co.id")

    print('input password')
    password = driver.find_element(By.NAME, "password")
    password.send_keys("timo123")
    
    print('click')
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/form/input[3]").click()
    
    #time.sleep(30)
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #print('scroll down')
    time.sleep(10)
    datefield = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button')))
    time.sleep(20)
    #datef = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'///html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[5]/div/section/h2')))
    #driver.execute_script("return arguments[0].scrollIntoView(true);", datef)
    time.sleep(20)
    driver.execute_script("return arguments[0].scrollIntoView(true);", datefield)
    time.sleep(10)
    driver.find_element(By.XPATH, "/html/body/div[1]/section/section[2]/article/div/div/aside/div/div[1]/section[4]/div/button").click()
    time.sleep(10)
    driver.find_element(By.XPATH, '//*[@id="exportSalesOrderForm"]/div/section/section/div[1]/section/article/article[1]/div/section/div/div[2]').click()
    #datefield.click()
    pilih=0
    while pilih < 3:    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==start_date.strftime('%B'):
                    # print(n_elem)
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
    
            n_elem=0
            for i in driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements(By.XPATH, '(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==end_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if len(driver.find_elements(By.XPATH, text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if end_date.strftime('%B')==start_date.strftime('%B'):
                    text = '('+text+')'#[2]
                else :
                    text = '('+text+')[1]'
            driver.find_element(By.XPATH, text).click()
            time.sleep(10)
            pilih=pilih+1
    driver.find_element(By.XPATH, '//*[@type="button" and contains(text(),"Apply")]').click()
    time.sleep(30)
    driver.find_element(By.XPATH, '/html/body/div[1]/section/section[2]/div/form/div/section/section/div[2]/aside[2]/button[2]').click()
    #WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Save')])"))).click()
    #driver.find_element(By.XPATH, '//span[text()="Sales Order v2"]').click()
    #driver.find_element(By.XPATH, '//span[text()="Sales Order v2"]').click()
    #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Export')])[2]"))).click()
    time.sleep(2)
    driver.quit()
    
    print('Forstok Downloaded')

    print('Download Data SKU')
    # Import library

    import time
    import pandas as pd
    import numpy as np
    import math

    import smtplib 
    from email.mime.text import MIMEText
    from email.mime.application import MIMEApplication
    from email.mime.multipart import MIMEMultipart
    from smtplib import SMTP
    import smtplib
    import sys
    import requests
    import os

    startdate = start_date-timedelta(days=1)
    startdate1 = startdate.strftime("%Y-%m-%d")
    enddate = end_date.strftime("%Y-%m-%d")

    text = 'https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/Nutrifood-forstok-sales_orders-version_2-' + startdate1 + 'T17:00:00Z-' + enddate + 'T16:59:59Z.xlsx'
    print(text)
    while True:
        r = s.get(text)
        print('Waiting to Download Forstok 2')

        if r.status_code == requests.codes.ok:
            print(r.status_code)
            with open('Input Data/Forstok_new_input.xls', 'wb') as output:
                output.write(r.content)
            print('output out')
            break
        else :
            time.sleep(30)

    with open('Input Data/nutrifood--forstok-sales_orders-' + str(date) + '.xls', 'wb') as output:
        output.write(r.content)
        
    orstat_forstok = pd.read_excel(r'Input Data/Forstok_new_input.xls')
    orstat_forstok['Date'] = np.nan
    orstat_forstok['Month'] = np.nan
    orstat_forstok['Year'] = np.nan

#    orstat_forstok = pd.read_excel(r"C:\Users\steven.nathanael\Downloads\Nutrifood-forstok-sales_orders-version_2-2023-05-14T17 00 00Z-2023-05-29T16 59 59Z.xlsx")
#    orstat_forstok = orstat_forstok[['Order Date', 'Channel', 'Store', 'Sales Order ID',
#                             'Order Reference No.', 'Payment Status', 'Fulfillment Status', 'Payment Type',
#                             'Payment Date', 'Shipping Label Printed Date', 'Shipped Date', 'Delivered Date',
#                             'Cancelled Date', 'Return ID', 'Shipping Courier', 'Service Type', 'AWB', 'Customer Name',
#                             'Customer Email', 'Currency Code', 'Item Name', 'SKU Code', 'Bundle SKU Code', 'Barcode ID',
#                             'Warehouse Name', 'Warehouse Code', 'Quantity', 'Regular Price', 'Selling Price',
#                             'Sub Total', 'VAT', 'Shipping Fee (Non-cashless)', 'Voucher Seller', 'Platform Rebate',
#                             'Shipping Customer Name', 'Shipping Address1', 'Shipping Address2', 'Shipping City',
#                             'Shipping Zip', 'Shipping Province', 'Shipping Country', 'Shipping Phone', 'Order Note',
#                             'Invoice ID']]
    orstat_forstok.rename(columns = {'Shipping Label Printed Date' : 'Printed Date',
                              'Shipping Date' : 'Fulfilled Date',
                              'SKU Code' : 'SKU',
                              'Shipping Fee (Non-cashless)' : 'Shipping',
                              'Voucher Seller' : 'Seller Voucher',
                              'Platform Rebate' : 'Channel Rebate',
                              'Order Note' : 'Note'}, inplace = True)
    orstat_forstok["Order Date"] = orstat_forstok["Order Date"].str.replace('WIB', '')
    orstat_forstok["Payment Date"] = orstat_forstok["Payment Date"].str.replace('WIB', '')
    #orstat_forstok["Cancelled Date"] = orstat_forstok["Cancelled Date"].str.replace('WIB', '')

    orstat_forstok['Order Date'] = pd.to_datetime(orstat_forstok['Order Date'])
    for i in range(orstat_forstok.shape[0]):
        if int(orstat_forstok['Order Date'][i].strftime('%d')) <= 12:
            orstat_forstok['Date'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
            orstat_forstok['Month'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            orstat_forstok['Year'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            orstat_forstok['Date'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).day
            orstat_forstok['Month'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).month_name()
            orstat_forstok['Year'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).year

    temp = orstat_forstok.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    orstat_forstok['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
    temp['Hour'] = pd.to_datetime(orstat_forstok['Order Date']).dt.hour
    temp['Minute'] = pd.to_datetime(orstat_forstok['Order Date']).dt.minute
    temp['Second'] = pd.to_datetime(orstat_forstok['Order Date']).dt.second
    orstat_forstok['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])

    print("Update Order Status")
    date_min  = orstat_forstok['True datetime'].min()
    orstat_forstok = orstat_forstok[['Sales Order ID','Fulfillment Status']].drop_duplicates('Sales Order ID')
    shopee_bp = data_all[data_all['Customer Name'] == 'Shopee Brand Portal']
    data_all = data_all[data_all['Customer Name'] != 'Shopee Brand Portal']
    data_all['True datetime1'] = pd.to_datetime(data_all['True datetime'])
    temp_all = data_all[data_all['True datetime1'] >= pd.to_datetime(date_min)][~data_all[data_all['True datetime1'] >= pd.to_datetime(date_min)]['Channel'].isin(['Nutrimart', 'L-Men Store Blibli', 'Order Online', 'Order Online Jakarta', 'Order Online Surabaya'])]
    orstat_forstok['Sales Order ID'] = orstat_forstok['Sales Order ID'].astype(str)
    temp_all['Sales Order ID'] = temp_all['Sales Order ID'].astype(str)
    temp_all = temp_all.merge(orstat_forstok, how = 'left', on = 'Sales Order ID')
    temp_all['Fulfillment Status'] = temp_all['Fulfillment Status'].fillna('Canceled')
    temp_all['Order Status'] = temp_all['Fulfillment Status']
    temp_all = temp_all.drop('Fulfillment Status', axis = 1)
    data_all['Order #'] = data_all['Order #'].astype(str)
    temp_all['Order #'] = temp_all['Order #'].astype(str)
    data_all = data_all[~data_all['Order #'].isin(temp_all['Order #'])]
    data_all = data_all.append(temp_all, ignore_index = True, sort = False)
    data_all = data_all.append(shopee_bp, ignore_index = True, sort = False)
    print("Export Master Data")

Starting Magento
Opening Chrome
Login Magento
Go To Sales Form
2024-01-25 16:07:08.247257
2024-02-06 16:07:08.247257
Input Date
Download Magento
bulk-order-download-template (1).csv
bulk-order-download-template (1).csv


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)


Import Data ====== 1/10


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_magentoUser = data_magentoUser.append(data_magentoUser2, ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:259: FutureWarning: The default value of regex will change from True to False in a future version.
  data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^\+62", "0")
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:264: FutureWarning: The default value of regex will change from True to False in a future version.
  data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("(021)", "021")
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:282: FutureWarning: The default value of regex will change from True to False in a future versi

--- 3.974240303039551 seconds ---
Formatting Data ====== 2/10
--- 4.154117584228516 seconds ---
Fulfilling SKU ====== 3/10
--- 4.1766722202301025 seconds ---
Listing SKU Missing ====== 4/10


...

Blibli Done
Unbundling Paket J


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:778: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forstok_all = forstok_all.append(tes1, ignore_index = True)


Done


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:785: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_all = data_all.append(forstok_all, ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_15676\592636022.py:786: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_all = data_all.append(data_magento, ignore_index = True, sort = False)


6272-072
62-0
6282-082
6280-080
6246-046
6255-055
6288-088
6259-059
6228-028
6286-086
6227-027
6267-067
6218-018
6233-033
6263-063
6208-008
6213-013
6296-096
6260-060
6277-077
6299-099
6266-066
6244-044
6278-078
6281-081
6221-021
6231-031
6242-042
6271-071
6283-083
6206-006
6230-030
6293-093
6265-065
6270-070
6229-029
6256-056
6268-068
6204-004
6235-035
6205-005
6275-075
6225-025
6273-073
6269-069
6262-062
6222-022
6236-036
6248-048
6203-003
6224-024
6212-012
6209-009
6264-064
6297-097
6241-041
6207-007
6291-091
6226-026
6253-053
6223-023
6215-015
6219-019
done
--- 586.861421585083 seconds ---
Read Data Settlement
Opening Chrome
Login Forstok


In [7]:
### if sku missing di magento
### to do : listing di tatanama rerun kode atas Run -4 
print(data_magento[~data_magento['SKU'].isin(data_SKU['SKU'])][['SKU','Product Name']].drop_duplicates())
print(data_magento[~data_magento['SKU'].isin(data_SKU['SKU'])]['Product Name'].unique())

               SKU                                 Product Name
543  2101150180P12  Tropicana Slim Milk Skim Coffee 500gr (12D)
['Tropicana Slim Milk Skim Coffee 500gr (12D)']


array(['Blibli', 'L-Men Store Blibli'], dtype=object)

In [7]:
print('done')

done


In [ ]:
#ORAMI - Seminggu Sekali

In [138]:
#ORAMI RUN 1 - Seminggu Sekali, Export 3 Minggu Sebelumnya
import time
import pandas as pd
import numpy as np
import math

import smtplib
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os

orami = pd.read_csv(r"C:\Users\steven.nathanael\Downloads\OramiSeller_01092023-30092023.csv", sep = ",",converters = {'Phone' : str, 'AWB' : str})
orami.drop(columns = 'Total', inplace = True)
orami.rename(columns = {'Tanggal pesanan' : 'Order Date',
                        'Nomor pesanan' : 'Order #',
                        'Provinsi' : 'City',
                        'Nama produk' : 'Product Name',
                        'No. Resi' : 'AWB',
                        'Nama customer' : 'Customer Name',
                        'Alamat customer' : 'Address',
                        'Kodepos' : 'Zip Code',
                        'Metode pengiriman' : 'Shipping Courier',
                        'Kuantitas' : 'Qty. Invoiced',
                        'Voucher' : 'Coupon Code',
                        'Total harga produk' : 'Total',
                        'Status pesanan' : 'Order Status'}, inplace = True)
orami['Qty. Invoiced'] = orami['Qty. Invoiced'].astype(int)
orami['Total'].fillna(0, inplace = True)
orami['Total'] = orami['Total'].astype(int)
orami['Day'] = orami['Order Date'].str.slice(0, 3).astype(int)
orami['Month'] = orami['Order Date'].str.slice(3, 6)
orami['Month'] = orami['Month'].apply(lambda x : 'September' if x == 'Sep' else
                                                 ('October' if x == 'Okt' else
                                                 ('November' if x == 'Nov' else 
                                                 ('December' if x == 'Des' else
                                                 ('January' if x == 'Jan' else
                                                 ('February' if x == 'Feb' else
                                                 ('March' if x == 'Mar' else
                                                 ('April' if x == 'Apr' else
                                                 ('May' if x == 'Mei' else
                                                 ('June' if x == 'Jun' else
                                                 ('July' if x == 'Jul' else 'August')))))))))))
orami['Year'] = orami['Order Date'].str.slice(6, 11).astype(int)
orami['month'] = orami['Month'].apply(lambda x : 9 if x == 'September' else
                                                 (10 if x == 'October' else
                                                 (11 if x == 'Novemer' else 
                                                 (12 if x == 'December' else
                                                 (1 if x == 'January' else
                                                 (2 if x == 'February' else
                                                 (3 if x == 'March' else
                                                 (4 if x == 'April' else
                                                 (5 if x == 'May' else
                                                 (6 if x == 'June' else
                                                 (7 if x == 'July' else 8)))))))))))
orami['Hour'] = orami['Order Date'].str.slice(12, 15).astype(int)
orami['Minute'] = orami['Order Date'].str.slice(16, 19).astype(int)
orami['True datetime'] = pd.to_datetime(orami[['Year','month','Day','Hour','Minute']])
orami.rename(columns = {'Day' : 'Date'}, inplace = True)
orami['Real SKU'] = orami['Product Name'].str.split('|').str[1].str.strip().astype(str)
orami['SKU'] = orami['Real SKU']
orami['Selling Price'] = orami['Total'] / orami['Qty. Invoiced']
orami[['Store','Channel']] = 'Orami Shopping'

orami.loc[orami['Product Name'].isin(['Free Spider Bottle - L-Men PlantProtein Ogura 2pcs']), 'Real SKU'] = 'PL24(2)G26'
orami.loc[orami['Product Name'].isin(['Parsel Wonderful Sweetness']), 'Real SKU'] = '(J)71110127'
orami.loc[orami['Product Name'].isin(['BUY 1 GET 1 - Tropicana Slim Mint Cocoa']), 'Real SKU'] = '2104148164P2'
orami.loc[orami['Product Name'].isin(['Free Tupperware - HiLo Platinum Swiss Chocolate 420g']), 'Real SKU'] = 'PH22G184'
orami.loc[orami['Product Name'].isin(['Buy 1 Get 1 Free - HiLo School Bubble Gum 500g']), 'Real SKU'] = '2101459180P2'
orami.loc[orami['Product Name'].isin(['Free HiLo School RTD - HiLo Teen Strawberry Milkshake 500g']), 'Real SKU'] = 'PH78B104'
orami.loc[orami['Product Name'].isin(['Free L-Men 2 GO - L-Men Lose Weight Chocolate Cereal 300g']), 'Real SKU'] = 'PL7B103'
orami.loc[orami['Product Name'].isin(['HiLo Active Es Teler 175gr']), 'Real SKU'] = '2101413144'
orami.loc[orami['Product Name'].isin(['Hampers Idul Fitri Lebaran NutriSari 25 Rasa (75s)']), 'Real SKU'] = '(J)71110126'
orami.loc[orami['Product Name'].isin(['BUY 1 GET 1 - HiLo Multigrain Original 500 gram']), 'Real SKU'] = '2101469180P2'
orami.loc[orami['Product Name'].isin(['Free Spider Bottle - L-Men Platinum Vanilla Caramel 800g']), 'Real SKU'] = 'PL30G26'

orami.loc[orami['Real SKU'].isin(['1101672318']), 'Real SKU'] = '1101665318'

data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
orami['Real SKU'] = orami['Real SKU'].astype(str)
orami = orami.merge(data_SKU[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', on = 'Real SKU')
orami['Total Net'] = orami['Price List NFI'].astype(int) * orami['Qty. Invoiced'].astype(int)

orami.loc[orami['Brand'].isin(["W'dank"]), 'Brand'] = 'WDANK'

indeks = orami[orami['Brand'] == 'Bundle'].index.to_list()
orami['Bundle Flag'] = np.nan
orami['Bundle Flag'][indeks] = 'Bundle'

orami['SKU'] = orami['SKU'].astype(str)
list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Cost 7')+1].to_list()
orami = orami.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
orami = orami.drop(['SKU_y'], axis = 1)
orami  = orami.rename(columns = {'SKU_x':'SKU'})

data_orami = orami.copy()

print("Unbundling")
data_bundle1 = data_orami[~data_orami['Produk 1'].isnull()]
data_bundle1['Bundle Name'] = data_bundle1['Product Name']
data_bundle1['Product Name'] = data_bundle1['Produk 1']
data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']*data_bundle1['Qty. Invoiced']
data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
data_bundle1['Bundle Flag'] = np.nan

data_bundle2 = data_orami[~data_orami['Produk 2'].isnull()]
data_bundle2['Bundle Name'] = data_bundle2['Product Name']
data_bundle2['Product Name'] = data_bundle2['Produk 2']
data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']*data_bundle2['Qty. Invoiced']
data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
data_bundle2['Total Net'] = data_bundle2['Price List NFI 2']
data_bundle2['Bundle Flag'] = np.nan

data_bundle3 = data_orami[~data_orami['Produk 3'].isnull()]
data_bundle3['Bundle Name'] = data_bundle3['Product Name']
data_bundle3['Product Name'] = data_bundle3['Produk 3']
data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']*data_bundle3['Qty. Invoiced']
data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
data_bundle3['Total Net'] = data_bundle3['Price List NFI 3']
data_bundle3['Bundle Flag'] = np.nan

data_bundle4 = data_orami[~data_orami['Produk 4'].isnull()]
data_bundle4['Bundle Name'] = data_bundle4['Product Name']
data_bundle4['Product Name'] = data_bundle4['Produk 4']
data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']*data_bundle4['Qty. Invoiced']
data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
data_bundle4['Total Net'] = data_bundle4['Price List NFI 4']
data_bundle4['Bundle Flag'] = np.nan

data_bundle5 = data_orami[~data_orami['Produk 5'].isnull()]
data_bundle5['Bundle Name'] = data_bundle5['Product Name']
data_bundle5['Product Name'] = data_bundle5['Produk 5']
data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']*data_bundle5['Qty. Invoiced']
data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
data_bundle5['Bundle Flag'] = np.nan

data_bundle6 = data_orami[~data_orami['Produk 6'].isnull()]
data_bundle6['Bundle Name'] = data_bundle6['Product Name']
data_bundle6['Product Name'] = data_bundle6['Produk 6']
data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']*data_bundle6['Qty. Invoiced']
data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
data_bundle6['Bundle Flag'] = np.nan

data_bundle7 = data_orami[~data_orami['Produk 7'].isnull()]
data_bundle7['Bundle Name'] = data_bundle7['Product Name']
data_bundle7['Product Name'] = data_bundle7['Produk 7']
data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']*data_bundle7['Qty. Invoiced']
data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
data_bundle7['Bundle Flag'] = np.nan

data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
data_bundle['SKU'] = data_bundle['SKU'].astype(str)
data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

orami = orami.append(data_bundle, ignore_index = True, sort = False)
orami = orami.merge(data_SKU[['Real SKU', 'Harga Cost']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')
orami['Total Harga Cost'] = pd.to_numeric(orami['Harga Cost'], errors = 'coerce') * orami['Qty. Invoiced']
print(orami[orami['Harga Cost'] == 0][['SKU','Product Name']].drop_duplicates())
orami['Total'] = orami['Selling Price'] * orami['Qty. Invoiced']
orami['Subtotal'] = orami['Selling Price'] * orami['Qty. Invoiced']
orami = orami.rename(columns = {'Real SKU_x' : 'Real SKU'})

columns_da = data_all.columns
columns_o = orami.columns
orami = orami[[x for x in columns_o if x in columns_da]]
orami['Sales Order ID'] = orami['Order #']

Unbundling
Empty DataFrame
Columns: [SKU, Product Name]
Index: []


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_16836\1850596331.py:167: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_16836\1850596331.py:172: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  orami = orami.append(data_bundle, ignore_index = True, sort = False)


In [28]:
#EROR
#ORAMI RUN 2 - Cek Order ID yang ada di data_all

oldoi = data_all[data_all['Store'].isin(['Orami Shopping'])]['Order #'].unique()

#Sukses
temp = orami[orami['Order Status'].isin(['Selesai'])]['Order #'].unique()
orderid1 = [x for x in temp if x in oldoi]
data_all.loc[data_all['Order #'].isin(orderid1),['Order Status']] = 'Selesai'

#Dibatalkan
temp = orami[orami['Order Status'].isin(['Dibatalkan'])]['Order #'].unique()
orderid2 = [x for x in temp if x in oldoi]
data_all.loc[data_all['Order #'].isin(orderid2),['Order Status']] = 'Dibatalkan'

#Pesanan Tiba
temp = orami[orami['Order Status'].isin(['Pesanan Tiba'])]['Order #'].unique()
orderid3 = [x for x in temp if x in oldoi]
data_all.loc[data_all['Order #'].isin(orderid3),['Order Status']] = 'Pesanan Tiba'

#New Order
temp = orami['Order #'].unique()
orderid4 = (set(temp) - set(oldoi))
append = orami[orami['Order #'].isin(orderid4)]
data_all = data_all.append(append, ignore_index = True)

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_16836\820702064.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_all = data_all.append(append, ignore_index = True)


In [62]:
orami[orami['Brand'].isin(['TS','NS','WDANK','L-Men','HiLo']) & orami['Order Status'].isin(['Selesai','Pesanan Tiba'])].groupby(['Month']).agg({'Total Net' : 'sum'})

,Total Net
Month,
July,78864.0
June,162060.0


In [139]:
orami = orami.append(orami1, ignore_index = True)
#data_all = data_all[~data_all['Store'].isin(['Orami Shopping'])]
data_all = data_all.append(orami, ignore_index = True)

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_16836\3400291211.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  orami = orami.append(orami1, ignore_index = True)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_16836\3400291211.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_all = data_all.append(orami, ignore_index = True)


In [140]:
data_all[data_all['Store'].isin(['Orami Shopping']) & data_all['Brand'].isin(['TS','NS','WDANK','L-Men','HiLo']) & data_all['Order Status'].isin(['Selesai','Pesanan Tiba'])].groupby(['Month']).agg({'Total Net' : 'sum'})

,Total Net
Month,
July,157728.0
June,324120.0
September,148740.0


In [7]:
## run-5
indeks = data_all[data_all['Qty. Invoiced'].isnull()].index.to_list()
data_all['Qty. Invoiced'][indeks] = 0
data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

In [9]:
##run-6
order_online_cond = False
shopee_scrap = False
shopee_done = False

In [17]:
##run-7
order_online_jatim = False
order_online_jkt = False
order_online_jateng = False
order_online_bali = False
order_online_makasar = False
order_online_samarinda = False
order_online_medan = False
order_online_lampung = False
order_online_pekanbaru = False
order_online_banjarmasin = False
order_online_jabar = False

In [35]:
## run 8
## until all region true
## error : add missing prod to code line jatim jkt jateng, other to "D:\Masterdata\Order Online\SKU buat andra updated maret 2021.xlsx"

start_time = time.time()
if not order_online_jatim:

    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_jatim')
    ##komen sementara

    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")
    
    time.sleep(30)

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customer@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("jatimhomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()

    time.sleep(50)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    
    #komen sementara
    # data_order=pd.read_csv(r"D:\Masterdata\Input Data\orderonline_orders_all_products_08-07-2022_MOoUrPq6Pzq0pu2y.csv")
    driver.close()
    product_order = pd.read_csv(r'data_supp/orderonline_products_Surabaya.csv')
    
    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 'L-Men Hi Protein 2 Go Chocolate (24 pcs)', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)

    
    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch')
                        
    product_order['title'] = product_order['title'].str.strip().str.replace('  ', ' ')
    product_order['title'] = product_order['title'].str.strip().str.replace('L-Men Gain Mass Chocolate 500 gr', 'L Men Gain Mass Chocolate 500 gr')


    product_order = product_order.append(pd.DataFrame([['Nutrisari Mango Smoothie 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (6pcs)', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate (12sch)', 5500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Honey (50sch)', 39500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sirup Orange 750ml', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 225gr', 57500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gainmass Taro 225gr', 69600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500 gr', 139200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria (40 sch)', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 750gr + Free Kertas Gambar', 85800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Jeruk Peras (40sch x 2) + Nutrisari Mangga Gandaria (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Blewah (40sch x 2) + Nutrisari Jeruk Maroko (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari American Sweet Orange (40sch x 2) + Nutrisari Milky Orange (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 500gr + Free Kertas Gambar', 117600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Chocolate 750gr', 127100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 Sch)', 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Cincau 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sunflower Oil 946 ml', 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Lose Weight Chocolate Cereal 300gr", 118800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Platinum Choco Latte Dus 6 sch", 80000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 pcs)", 264000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 10500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Gain Mass Banana 225gr", 61000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Protein Bar Chocolate 12 sch", 105600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo Chocolate Taro (10 sch)", 14700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - W'Dank Empon Empon 10 Sachet Renceng", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Nanas 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Semangka 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(FREE) W'Dank Empon-Empon 10 sch", 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Chocolate 1000 gr", 136400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-MEN Gain Mass Taro 225 gr", 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Whey Daily Dark Chocolate 250gr", 49500]], columns = ['title', 'price']), ignore_index = True, sort = False)                                        
    product_order = product_order.append(pd.DataFrame([["HiLo Es Teler 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo Es Ketan Hitam 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Honey 500gr", 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Vanilla 750gr", 102300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)', 52000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Sweetener Jahe 50 sch", 38500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Korean Garlic Butter Cookies 5 Sch", 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sambal Terasi 200 gr', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2', 107800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)', 67500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate 12 sch', 105600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Crunch BBQ Beef (20gr)', 9900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Avocado Coffee 4 Sch x 2', 24200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 28900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 20700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Cocopandan 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Shirataki Noodles 71gr', 28100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Saus Tiram 200ml', 39600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Noodles 71gr', 14200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Saus Tiram 200ml', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim White Coffee (4sch)', 17600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hampers Idul Fitri Tropicana Slim', 99000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Takjil HiLo Dessert', 67500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Ramadhan NutriSari 2021', 60000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Kehangatan WDank - Ramadhan Edition', 60000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Platinum Kacang Hijau 800g', 275000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - SUGAR FREE', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Whey Advanced Cappuccino 250gr', 79200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Minyak Kanola 946ml - 100% Pure Canola Oil - Free Korean Garlic Butter Cookies 1 sch', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Minyak Kanola 946ml', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(EACH)Tropicana Slim Korean Garlic Butter Cookies - Sampling', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Platinum Ketan Hitam 800g (25g protein / serving) - Suplemen Tinggi Whey Protein Rendah Lemak', 302500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Whey Advanced Choco Vanilla 500gr', 148500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Susu Skim Chocolate 500gr - Bantu Turunkan Kolesterol', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Cafe Latte 10 Sachet 14gr - Kopi Susu Nikmat Tanpa Gula Pasir', 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Platinum Choco Latte 800gr (25gr protein)', 302500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Susu Skim Plain 1000gr - Bantu Turunkan Kolesterol', 180400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Susu Skim Coffee 500gr - Bantu Turunkan Kolesterol', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - HiLo Active Es Teler 175gr', 38000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Es Teler 175gr', 19000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Apel Jeruk 40 Sachet', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Teen Popcorn Caramel 500 gram x 2', 99000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch', 57800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Rose Vanilla 50 sch', 28900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Chocolate 500gr', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Coffee 500gr', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml - [Free] Tropicana Slim Shirataki Noodles 71gr', 83500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 750g Susu Tinggi Kalsium Lebih Rendah Lemak - [FREE] HiLo Active Ketan Hitam 175g', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim DIABTX (100 sch) - [FREE] Tropicana Slim Avocado Coffee 4 Sch', 76000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Original 1kg', 187700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs', 22600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak Jeruk Bali 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD)', 5650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Mayonnaise 200 g x 2', 34800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Mayonnaise 200 g - Bantu Dukung Hidup Sehat', 17400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Chocolate 1000 gr', 115500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: L-Men Daily Popcorn Caramel 250gr x 2', 78200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml', 83500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Daily Popcorn Caramel 250gr', 39100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 - HiLo Gold Sweet Potato 500 gram', 73200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Jeje Si Jeli', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Asin 200 ml', 24600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Sweet Potato 500 gram', 73200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) HiLo Gold Sweet Potato 500 gram', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE HiLo Joint Plus 140gr', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Joint Plus Orange 140 gram', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) HiLo Joint Plus Orange 140 gram', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Korean Goguma Cookies (5 Sch) x 2', 32200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Mba Nana Si Penuh Pesona', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Popcorn Caramel 500gr', 68600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 1000 gr', 141900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Special Bundle - Tropicana Slim Sweetener Rose Vanilla & Tropicana Slim Korean Goguma Cookies', 55000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 6850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 6850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Rose Vanilla 50 sch', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Andaliman 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Tape Ketan 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Plain 1000 gr', 128200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Mas Kaka yang Banyak Akal', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo School Strawberry Cheesecake 500 gram x 2', 110600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Strawberry Cheesecake 500 gram', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Joint Plus Orange 140 gram x 2', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla Vegiberi 750gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Kurma Pack (4R)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Jeju [40 Sachet]', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak (4R)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Nipis (40 sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Lemon (40 Sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea (10 sch)', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml (4pcs)', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Swiss Chocolate (10 sch) Renceng', 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Teen Coffee Tiramisu Ready To Drink Susu UHT [4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Teen Chocolate Ready To Drink Susu [4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Vanilla Vegiberi Ready To Drink Susu [200ml/4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea 40 sch', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Yuzu Orange 40 sch', 46200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Manis 500gr', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Gold Sweet Potato 500 gram x 2', 105600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Andaliman 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Sweet Potato 500 gram', 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Tape Ketan 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim DIABTX (100 sch)', 76100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Nutty Chocolate Cookies 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Markisa (40 Sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Madu Jeruk (40sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari NUTRI C1000 Jeruk 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Chocolate 500gr', 61800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bajigur (10 sch)", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener I Sweet', 21700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Kacang Hijau 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - Lokalate Kopi Tape Ketan 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Classic Refill 500gr', 98400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Fiber Pro Plain 500gr', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Vanilla Caramel 500gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim High Fiber High Calcium Milk Chocolate 500gr', 112100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - Lokalate Kopi Andaliman 10 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Almond Milk Coconut 200gr x 2', 70000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Almond Milk Coconut 200gr', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['HiLo Yoghurt Smoothie Bowl Strawberry (8 sch)', 62900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bandrek 10 sachet Renceng", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Chocolate Candy (10 sch)", 18300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500320 (CI160)", 72160]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500336 (CS)", 45100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["NutriSari Jeruk Maroko (40 Sch) FREE Lokalate Andaliman (10 Sch)", 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Jeruk Peras Refill 500 gr", 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Maroko (40 sch)", 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(FREE) Lokalate Kopi Andaliman 10 Sachet", 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack:HiLo Teen Strawberry Milkshake 500g x 2", 100000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Strawberry Milkshake 500g", 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Chocolate Ready To Drink (4 tetrapack)", 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School RTD Coklat 200ml", 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Sirup Leci 750ml", 30900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["NutriSari Mango Smoothie 200ml (4 Pcs)", 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari RTD Jeruk Madu 200 ml (4 tetrapack)", 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari RTD Mango Smoothie 200 ml", 6900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari RTD Jeruk Madu 200 ml", 5750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Active Chocolate 750gr", 89200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate x 4 pcs", 38400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate", 9600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Honey 50 sachet', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Triple Pack - Tropicana Slim Klepon Cookies (5 Sch) - Snack Bebas Gula, 100 Kalori', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Klepon Cookies (5 Sch)', 16700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Platinum Swiss Chocolate 420gr', 91500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 250g (Health & Green Science Competition)', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Shirataki Rice 72 g', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Rice 72 g', 25000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Nipis Jahe 40 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Vanilla 500gr', 65200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Yoghurt Banana 250gr', 38400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['TS SHIRATAKI RICE RASA NASI UDUK 24PX72G', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Cotton Candy 500g', 62500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Kuwud Nipis', 37500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea Refill 500 gram', 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea Ready to Drink 200ml x 4pcs', 23600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo RTD Thai Tea 200ml', 5900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo School Bubble Gum 500g', 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo School RTD Coklat 200ML', 6270]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 49500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim BROWNIES  Instant Powder Cake Mix 230 g', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Caramel Latte 500g', 50500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Avocado Chocolate (10 Sch)', 127160]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Bundle - HiLo Teen Taro 500gr & HiLo Teen Strawberry Milkshake 500g', 11400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 63600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Strawberry Milkshake 500g', 63600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)Bundle L-Men Platinum Ketan Hitam 800g & L-Men Platinum Kacang Hijau 800g', 314600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)L-Men Platinum Ketan Hitam 800g ', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)L-Men Platinum Kacang Hijau 800g', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)Twinpack: L-Men Platinum Kacang Hijau 800g', 314600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)L-Men Platinum Kacang Hijau 800g', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Es Ketan Hitam 500g - Powder Drink Tinggi Kalsium', 44000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Biscuit Caramel 500gr', 65000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Plain (Original) 500gr', 77800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lemon Tea 500 gram', 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Pisang Bakar 10 sch', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Pisang Bakar Epe 10 sch', 8750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Bubble Gum 500g', 80100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack/B1G1 HiLo Multigrain Original 500g', 176000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Multigrain Original 500g', 88000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Lemon-C (25 sch)', 25200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Belgian Chocolate (10 sch)', 49200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["HiLo Chocolate Polos 10's", 12000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo White Chocolate (10 Sch)', 12000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Cookies Paket Hampers Idul Fitri Parcel Lebaran', 131600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Banana 10 sch', 12000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese Cookies', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Nutty Chocolate Cookies 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Garlic Butter Cookies 5 Sch', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Klepon Cookies (5 Sch)', 21500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Gift - Paper Bag Ramadhan Tropicana Slim', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['HiLo Es Pisang Ijo 10 Sch x 2 pcs', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Mangga 500gr', 132700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 - Tropicana Slim Mint Cocoa - Minuman Cokelat Mint Nikmat Tanpa Gula Pasir', 20000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE L-Men Lose Weight Mango Sticky Rice 300gr', 145200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE Tropicana Slim Susu Low Fat Macchiato Coffee 500gr', 143000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Diabetamil Milk Vanilla 150 gram', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 30900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-MEN PLANTPROTEIN OGURA 6Dx216G', 103900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr - Near ED', 16150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Choco Series Paket Hampers Idul Fitri Parcel Lebaran', 139600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Chocolate 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 104800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 115500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Vanilla 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 86650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Es Pisang Ijo 10 Sch', 14300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Soy Latte - Kopi Plant Based Rendah Gula', 36900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Mint Cocoa 4 Sachet', 20000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Taro 500g x 2 pcs', 133900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo Gold Biscuit Cereal 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Alpukat 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea Refill 500g', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Soy Latte - Kopi Susu Kacang Bebas Gula', 36900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['2102900319 (D)', 90400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Biscuit Cereal 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Anggur Hijau 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Gula Asem 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - Tropicana Slim Low Fat Milk Korean Strawberry 500g', 159900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Low Fat Milk Korean Strawberry 500g', 79900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese Cookies 100gr', 22500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Wdank Madu Temulawak 10 Sachet', 16600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Es Ketan Hitam Refill 500g - Powder Drink Tinggi Kalsium', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["(Near ED) Lokalate Kopi Tape Ketan 10's", 7500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo Joint Plus Orange 140 gram', 19150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) BUY 1 GET 1 L-Men Gain Mass Mangga 500gr', 132700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo Teen Taro 500gr', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Bundle - Lokalate Kopi Alpukat Refill 500 gram & Kopi Berondong Refill 500 gram', 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea RTD 200ml x 4pcs', 24000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - W'Dank Temulawak Madu", 33300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['TRIPLE HEMAT - HiLo Thai Tea Refill 500g', 100900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['TRIPLE HEMAT - NutriSari Lemon Tea 500 gram', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'dank Sarabba Extra 10 Sachet", 18870]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'dank Madu Temulawak 10 Sachet", 16650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Nutrisari Blewah 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Nipis 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari American Sweet Orange 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Peras 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Milky Orange 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Chocolate Polos 10 sch", 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Madu Lemon 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Nutrisari Florida Orange 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Anggur Hijau 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari NUTRI C1000 Jeruk 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Markisa 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Kuwud Nipis 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE HEMAT - NutriSari Lychee Tea Refill 500 gram", 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Maroko 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Apel Jeruk 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Chocolate Taro 10 sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Avocado Chocolate 10 sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - HiLo Platinum Original 360 gram", 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE DEALS - NutriSari Lychee Tea & Lemon Tea 500 gram Refill", 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Gula Asem 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Madu Jeruk 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Lemon Tea Refill 500 gram", 34600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE HEMAT - HiLo Klepon Latte Refill 500g", 109100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twinpack Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 29000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Madu Kurma Pack 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo White Chocolate 10 sch", 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Taro Latte Refill 500g", 32700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Rujak 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Rujak Jeruk Bali 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE DEALS - HiLo Thai Tea , Klepon Latte , Taro Latte Refill 500g", 103900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Buy 3 Get 3 - HiLo Milky Vanilla Cookies Ready to Drink 200ml", 60000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twinpack Lokalate Kopi Alpukat Refill 500 gram", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Twinpack Lokalate Kopi Berondong Refill 500 gram", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Jeju 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["NutriSari Jeruk Nipis Jahe 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500320 (CI160) TS SWT 160S IND", 76590]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["HiLo Platinum Original 360g (12 Sachet)", 127500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 2 GET 2 - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C", 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Klepon Latte Refill 500g", 25950]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Lokalate Kopi Andaliman 10 Sachet", 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Active Caramel Latte 500g", 32600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 Free - HiLo School 3+ Strawberry Pop 400g", 199800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE HEMAT - HiLo Taro Latte Refill 500g", 145454]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 29000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School 3+ Strawberry Pop 400g", 99900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Near ED - HiLo Almond Milk Coconut 200gr", 51500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Orange Cocoa 4 Sachet - Minuman Cokelat Jeruk Nikmat Tanpa Gula Pasir", 16650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 Free - HiLo School Original 400g - Susu Tinggi Kalsium Lebih Rendah Lemak", 191400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 Free - HiLo Teen Original 400g - Susu Tinggi Kalsium Lebih Rendah Lemak", 191400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack: HiLo Platinum Swiss Chocolate (12 Sch)", 189300]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([["HiLo Choco Hazelnut (10 Sch)", 12210]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Strawberry 10 Sachet - Susu Tinggi Kalsium Lebih Rendah Lemak & Gula", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C", 24000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twinpack HiLo Platinum Original 360gram", 230900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Sweet Orange Sugar Free 10sachet", 20200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["L-Men Lose Weight Mango Sticky Rice 300gr", 127000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C", 144100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - Tropicana Slim Orange Cocoa 4 Sachet - Minuman Cokelat Jeruk Nikmat Tanpa Gula Pasir", 33300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 2 GET 2 - Tropicana Slim RTD Almond Drink Chocolicious 190 ml", 33400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 - Tropicana Slim Bumbu Kaldu Ayam dan Jamur 100 gram", 48500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hampers Lebaran Fancy Tote Bag Tropicana Slim", 186500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 12 GET 12 - Tropicana Slim RTD Almond Drink Chocolicious 190 ml", 199800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 - L-Men Gain Mass Klepon Latte", 277100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - Tropicana Slim Salted Caramel Cocoa 4 Sachet", 33300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim French Butter Souffle Coffee 4 Sachet", 16200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teh Tarik (10 Sachet) - Teh Tarik Tinggi Kalsium Bervitamin", 16050]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Premium Jus Mangga 420g - Minuman Juice Kental Rasa Mangga Asli Vitamin C", 46600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Salted Caramel Cocoa 4 Sachet", 26000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY1GET 1 - Tropicana Slim French Butter Souffle Coffee 4 Sachet", 32400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Lemon Tea 40 sch", 47300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - Tropicana Slim Peanut Almond Butter Spread Jam 300 gram", 184800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram", 24200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY1 GET1 - Tropicana Slim Sweetener Luo Han Guo 50 sachet - Pemanis Alami Natural untuk Batasi Gula", 115400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim RTD Almond Drink Chocolicious 190 ml", 8700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) L-Men Gain Mass Taro 500g (Ags 23)", 66950]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 12 Get 12 Free - L-Men UHT PlantProtein 2GO Ogura / Kacang Merah 190ml", 319680]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Multigrain Original 500g (Ags 23)", 49650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Less Sugar Belimbing 40 sachet", 62000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Platinum Bubble Gum 800 gram", 375200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo RTD Milky Vanilla Cookies 200ml (Ags 23)", 3850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - L-MEN Gain Mass Taro 225 gr", 39250]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 2 Get 2 Free - L-Men UHT PlantProtein 2GO Ogura / Kacang Merah 190ml", 61600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo School Cotton Candy 200ml - Ready to Drink (24 Pcs) / ED Sept 2023", 171800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram / ED Nov 23", 85400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - HiLo Teen Chocolate 1000 gr", 161600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Gold Chocolate 1000g / ED Okt 23", 147800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) NutriSari Gula Asem 40 Sachet / ED Nov 23", 47300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - HiLo Teen Chocolate 1000 gr / ED Okt 23", 161600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) NutriSari Gula Asem 40 Sachet", 47300]], columns = ['title', 'price']), ignore_index = True, sort = False)
   
    product_order = product_order.append(pd.DataFrame([["2102900319 (D) TS SWT DIABETICS INDUSTRIAL 150S", 82500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula / ED Sept 23", 48500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Es Ketan Hitam Refill 500g", 46200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula", 48500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Teen Biscuit Caramel 500gr / ED Nov 23", 87700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Premium Jus Mangga 10 Sachet", 25000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 - Tropicana Slim Sweetener Gula Aren 50 sachet - Pemanis untuk Bantu Batasi Gula", 90000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Lokalate Kopi Berondong 500 gram", 48500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) L-Men Lose Weight Mango Sticky Rice 300gr / ED Nov 23", 152400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 2 Get 2 Free - L-Men UHT PlantProtein 2GO Ogura / Kacang Merah 190ml (4pcs) with 12g Protein & 1,6g BCAA / Serving - Suplemen Tinggi Protein Nabati Siap Minum", 61600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Isotonik 40 Sachet – Minuman Isotonik Buah Jeruk Segar", 62000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Bundle Special Pack Isi 6 pcs - NutriSari RTD Squeezed Orange 200 ml", 35700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Isopower Stargizing 30 Sachet with Creatine & Vitamin B", 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Isotonik 40 Sachet – Minuman Isotonik Buah Jeruk Segar", 62000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 12 GET 12 - Tropicana Slim Oat Drink Japanese Cantaloupe Melon 190 ml (RTD)", 240000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 2 GET 2 - Tropicana Slim Oat Drink Japanese Cantaloupe Melon 190 ml (RTD)", 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 Free - HiLo Chocolate Mint 10 Sachet", 25530]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Tropicana Slim Lemon-C (25 sch) / ED Okt 23", 26600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - Tropicana Slim Gula Buah 50 Sch - Alami untuk Batasi Gula", 138000]], columns = ['title', 'price']), ignore_index = True, sort = False)
   
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - Tropicana Slim 7 Fruits Fiber Daily 12 Sachet", 217560]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) NutriSari Madu Lemon (40 Sch)", 45510]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Nutrisari Jeruk Madu Jeruk 40'sachet (4 Renceng)", 45510]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) - NutriSari Lychee Tea 40 sch", 45510]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Nutrisari Florida Orange (40 Sch)", 45510]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) NutriSari Madu Kurma 40 sch", 45510]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo School Vanilla 1000g", 149850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - NutriSari RTD Squeezed Orange 200 ml", 5772]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram", 24420]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - HiLo Teen Vanilla Caramel 500gr", 84360]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Gold Biscuit Cereal 500g", 81030]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Peanut Almond Butter Spread Jam 300 gram", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Active Chocolate 1000 gr", 129870]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Wdank Madu Temulawak 10 Sachet", 17316]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Tropicana Slim Sweetener Gula Aren 50 sachet", 43290]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Tropicana Slim Soy Latte - Kopi Susu Kacang Bebas Gula", 37740]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Tropicana Slim BROWNIES Instant Powder Cake Mix 230 g", 33300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) W'Dank Bandrek 10 sachet Renceng - Khusus Homdel", 17316]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram", 82140]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Tropicana Slim Low Fat Milk Korean Strawberry 500g", 84360]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School 3+ Soya Multigrain Vanilla Malt 400g", 99900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Gold Chocolate 1000g", 142080]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) L-Men Lose Weight Mango Sticky Rice 300gr", 146520]], columns = ['title', 'price']), ignore_index = True, sort = False)
   

    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Teen Biscuit Caramel 500gr", 87700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Chocolate Mint 10 Sachet", 18300]], columns = ['title', 'price']), ignore_index = True, sort = False) 
    
    price = product_order[product_order['title'] == 'Tropicana Slim Goldenmil Vanilla (6sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Lokalate Kopi Kawista (10sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Lokalate Kopi Kawista (10sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Tropicana Slim Kecap Manis 200ml']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Manis 200m', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    
    #Komen Testing
    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    #data_SKU = pd.read_excel(r'Order Online\SKU buat andra.xlsx')
    data_SKU = data_SKU.rename(columns = {'Unnamed: 0' : 'SKU', 'ref.1' : 'product', 'Pricelist Nutrimart' : 'Price List NFI', 'Harga jua Homdel' : 'Harga Display'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip()

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)
    product_order['title'] = product_order['title'].astype(str)

    data_order = data_order.merge(data_SKU[['SKU', 'product']].drop_duplicates('product'), how = 'left', on = 'product')
    data_order = data_order.merge(product_order[['title', 'price']].drop_duplicates('title'), how = 'left', left_on = 'product', right_on = 'title').drop('title', axis = 1)

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    
    
    
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Surabaya'

        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        indeks = data_order[data_order['Item Name'].astype(str).str.contains('pcs')].index.to_list()
        temp = data_order.iloc[indeks].copy()
        product = temp['Item Name'].str.split("\(",1, expand = True)

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        index = order_all[order_all['SKU'].astype(str) == '2306551174'].index.to_list()
        order_all['SKU'][index] = '2306592173'
        order_all['Real SKU'][index] = '2306592173'
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan

        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Price' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})
        
        
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        
        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']
        
        order_all['Selling Price'] = order_all['Regular Price'].astype(int)
        
        data_SKU3 = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
        data_SKU3 = data_SKU3.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
        data_SKU3['SKU'] = data_SKU3['SKU'].astype(str).str.replace('.0', '', regex = False)
        data_SKU3 = data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0'][data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0']['Harga Coret'].notnull()]
        data_SKU3['product'] = data_SKU3['product'].str.strip()
        data_SKU3 = data_SKU3.reset_index(drop = True)
        
        for i in range(data_SKU3.shape[0]):
            if data_SKU3['SKU'][i] in order_all['SKU'].values:
                indeks = order_all[order_all['SKU'].astype(str) == i].index.to_list()
                order_all['Regular Price'][indeks] = data_SKU3['Harga Display'][i]
                order_all['Selling Price'][indeks] = data_SKU3['Harga Coret'][i]
        
        
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan

        order_all['Warehouse Name'] = 'Order Online Surabaya Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
         'Product Name',
         'Customer Name',
         'Phone',
         'Address',
         'Region',
         'City',
         'Zip Code',
         'payment_status',
         'Regular Price',
         'Shipping Courier',
         'Shipping Cost',
         'Subtotal',
         'Qty. Invoiced',
         'SKU',
         'Order #',
         'Invoice Number',
         'Shipping Name',
         'Shipping Address2',
         'Country',
         'AWB',
         'Channel',
         'Order date',
         'Real SKU',
         'Real Nama Produk',
         'Brand',
         'Sub Brand',
         'Parent Item',
         'Parent SKU',
         'Produk 1',
         'SKU Produk 1',
         'PCS Produk 1',
         'Price List NFI 1',
         'Subtotal Produk 1',
         'Harga Display 1',
         'Harga Cost 1',
         'Harga Organik 1',
         'Produk 2',
         'SKU Produk 2',
         'PCS Produk 2',
         'Price List NFI 2',
         'Subtotal Produk 2',
         'Harga Display 2',
         'Harga Cost 2',
         'Harga Organik 2',
         'Produk 3',
         'SKU Produk 3',
         'PCS Produk 3',
         'Price List NFI 3',
         'Subtotal Produk 3',
         'Harga Display 3',
         'Harga Cost 3',
         'Harga Organik 3',
         'Produk 4',
         'SKU Produk 4',
         'PCS Produk 4',
         'Price List NFI 4',
         'Subtotal Produk 4',
         'Harga Display 4',
         'Harga Cost 4',
         'Harga Organik 4',
         'Produk 5',
         'SKU Produk 5',
         'PCS Produk 5',
         'Price List NFI 5',
         'Subtotal Produk 5',
         'Harga Display 5',
         'Harga Cost 5',
         'Harga Organik 5',
         'Produk 6',
         'SKU Produk 6',
         'PCS Produk 6',
         'Price List NFI 6',
         'Subtotal Produk 6',
         'Harga Display 6',
         'Harga Cost 6',
         'Harga Organik 6',
         'Produk 7',
         'SKU Produk 7',
         'PCS Produk 7',
         'Price List NFI 7',
         'Subtotal Produk 7',
         'Harga Display 7',
         'Harga Cost 7',
         'Harga Organik 7',
         'Bundle Flag',
         'Date',
         'Month',
         'Year',
         'Quarter',
         'Week',
         'True datetime',
         'Total',
         'Price List NFI',
         'Total Net',
         'Selling Price',
         'Kecamatan',
         'Kelurahan',
         'Bundle SKU',                             
         'Bundle Name',
         'Harga Cost',
         'Total Harga Cost',
         'Seller Discount',
         'Shipping',
         'Promo',
         'Discount MC',
         'Warehouse Name',
         'Customer Email',
         'Order Status',
         'Payment Channel',
         'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_jatim = True
        del file_name

if not order_online_jkt:
    import pandas as pd 
    import numpy as np
    import requests
    import os
    
    print('order_online_jkt')
    #komen sementara
    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    time.sleep(8)
    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("nutrimart.nfi@gmail.com")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("sumselhomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(50)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    product_order = pd.read_csv(r'data_supp/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch')

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Unnamed: 0' : 'SKU', 'ref.1' : 'product', 'Pricelist Nutrimart' : 'Price List NFI', 'Harga jua Homdel' : 'Harga Display'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip()
    product_order['title'] = product_order['title'].str.strip().str.replace('  ', ' ')
    product_order['title'] = product_order['title'].str.strip().str.replace('L-Men Gain Mass Chocolate 500 gr', 'L Men Gain Mass Chocolate 500 gr')

    product_order = product_order.append(pd.DataFrame([['Nutrisari Mango Smoothie 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (6pcs)', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)', 264000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate (12sch)', 5500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Honey (50sch)', 39500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sirup Orange 750ml', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 225gr', 57500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gainmass Taro 225gr', 69600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Milk Brown Sugar RTD 200ml (6pcs)', 33000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Lose Weight Chocolate Cereal (12sch)', 99000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500 gr', 139200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Strawberry Jam 375gr', 72600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria (40 sch)', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Cegah Diabetes (+Kaos)', 116100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 250gr + Free Kertas Gambar', 40500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 750gr + Free Kertas Gambar', 85800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Jeruk Peras (40sch x 2) + Nutrisari Mangga Gandaria (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Blewah (40sch x 2) + Nutrisari Jeruk Maroko (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari American Sweet Orange (40sch x 2) + Nutrisari Milky Orange (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 500gr + Free Kertas Gambar', 117600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Ngopi Lokalate', 136000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Chocolate 750gr', 127100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Florida Orange (40sch x 2) + Nutrisari Markisa (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml', 82000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)', 94000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 Sch)', 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Platinum Choco Latte Dus 6 sch", 80000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 104000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 75000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Semangka 40 sch", 42500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Cincau 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Nanas 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 180400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Es Teler 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo Es Ketan Hitam 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Nipis (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Lemon (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Sweet Mango (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate 12 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Crunch BBQ Beef (20gr)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Chocolate Taro (10 sch)', 11000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Markisa (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)', 67500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Avocado Coffee 4 Sch x 2', 18200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Cocopandan 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sambal Terasi 200 gr', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lemon Tea 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Vanilla 750gr', 102300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo RTD Kacang Hijau 200ml (6pcs)', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Chocolate 250gr', 29150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Milky Orange (40sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 28900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)', 52000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Vanilla 500gr', 66000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Jahe 50 sch', 38500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["HiLo Gold Plain (Original) 750gr", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Vanilla Vegiberi 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Lose Weight Chocolate Cereal 300gr', 104500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Honey 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Garlic Butter Cookies 5 Sch', 20400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 211200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-MEN Gain Mass Taro 225 gr', 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Chocolate 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sunflower Oil 946 ml', 57200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["L-Men Whey Daily Dark Chocolate 250gr", 49500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Vanilla 200gr', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Chocolate 1000 gr', 136400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - W'Dank Empon Empon 10 Sachet Renceng", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Banana 225gr', 60500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo RTD Kacang Hijau 200 ml', 4950]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate', 8800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(FREE) W'Dank Empon-Empon 10 sch", 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Khusus Homdel - HiLo Chocolate Banana 10 sch', 11000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Strawberry 40 Sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Vanila Vegiberi 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Saus Tiram 200ml', 39600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Shirataki Noodles 71gr', 28100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Vanila Vegiberi 200ml', 6100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Saus Tiram 200ml', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Noodles 71gr', 14100]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['Nutrisari American Sweet Orange (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Kurma Pack (4R)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Milky Orange (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Chocolate (10 sch)', 11000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Swiss Chocolate (10 sch) Renceng', 16500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Leci 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Sweet Guava (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Paket Hilo - Lebaran Edition', 56100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Ramadhan NutriSari 7 Rasa (7 x 10 sch)', 294000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Peras (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Lebaran Tropicana Slim', 96300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Lokalate untuk Sobatku - Idul Fitri Edition', 54120]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - SUGAR FREE', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Florida Orange (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo RTD Milky Brown Sugar 200 ml', 4950]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Belgian Chocolate (10 sch)', 41800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Mangga Gandaria (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Asin 200 ml', 23650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Apel Jeruk 40 Sachet', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Minyak Jagung 1lt', 69300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Coffee Tiramisu 200ml (6pcs) - Ready to Drink', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Madu Jeruk 40'sachet (4 Renceng)", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen RTD Coffee Tiramisu 200ml', 6050]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Chocolate 500gr', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Mayonnaise 200 g x 2', 34800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs', 22600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Mayonnaise 200 g - Bantu Dukung Hidup Sehat', 17400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD)', 5650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Baru! NutriSari NUTRI C1000 Jeruk 40 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: L-Men Daily Popcorn Caramel 250gr x 2', 78300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Daily Popcorn Caramel 250gr', 39150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE HiLo Joint Plus 140gr', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak Jeruk Bali 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False) 
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Tape Ketan 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False) 
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Korean Goguma Cookies (5 Sch) x 2', 32200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 6850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Andaliman 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Plain 1000 gr', 128200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Mas Kaka yang Banyak Akal', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla Vegiberi 500gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Vanilla Caramel 500gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari NUTRI C1000 Jeruk 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Joint Plus Orange 140 gram x 2', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Popcorn Caramel 500 gram - Susu Tinggi Kalsium Lebih Rendah Lemak', 68600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo School Strawberry Cheesecake 500 gram x 2', 110600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Yuzu Orange 40 sch', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Jeju [40 Sachet]', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Strawberry Cheesecake 500 gram', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Andaliman 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Masak Sehat', 125000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml', 85800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Manis 200ml', 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Santan 5 sachet', 15400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Tape Ketan 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml (4pcs)', 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml', 5750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Paket Ngemil Sehat', 125000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 21200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD)', 7500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - SUGAR FREE', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese Cookies', 21200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['2102900319 (D)', 82500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['2102500336 (classic stick)', 46900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Maroko (40 Sch) FREE Lokalate Andaliman (10 Sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Nutty Chocolate Cookies 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak (4R)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Maroko (40 sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Andaliman 10 Sachet', 13600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 750gr', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Yoghurt Smoothie Bowl Strawberry (8 sch)', 62900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate Ready To Drink (4 tetrapack)', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Teen Chocolate Ready To Drink Susu [4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Coklat 200ml', 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen RTD Coklat 200ml', 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Almond Milk Coconut 200gr x 2', 70000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate x 4 pcs', 38400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Almond Milk Coconut 200gr', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate', 9600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea Ready to Drink 200ml x 4pcs', 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo RTD Thai Tea 200ml', 5750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bajigur (10 sch)", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bandrek 10 sachet Renceng", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Milky Brown Sugar Ready To Drink 200ml (4 Tetrapack)', 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Vanilla Vegiberi Ready To Drink Susu [200ml/4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo RTD Milky Brown Sugar 200 ml", 5725]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School RTD Vanila Vegiberi 200ml", 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Chocolate Candy (10 sch)", 18300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500320 (CI160)", 72160]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 1000 gr', 141900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Triple Pack: Tropicana Slim Klepon Cookies (5 Sch)', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Klepon Cookies (5 Sch)', 16700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Nipis Jahe 40 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Shirataki Rice 72 g', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Rice 72 g', 25000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sirup Leci 750ml', 30900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Mango Smoothie 200ml (4 Pcs)', 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari RTD Mango Smoothie 200 ml', 6900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Es Teler 175gr', 26300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Platinum Swiss Chocolate 420gr', 91500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Kuwud Nipis', 37500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Classic Refill 500gr', 104100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Bundle - HiLo Teen Taro 500gr & HiLo Teen Strawberry Milkshake 500g', 127200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo School Bubble Gum 500g', 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HILO ALMOND MILK COCONUT 12DX200G', 51500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 80100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo School RTD Coklat 200ML', 6900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Strawberry Milkshake 500g', 75500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener I Sweet', 25200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Pisang Bakar 10 sch', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Pisang Bakar Epe 10 sch', 8750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla Vegiberi 750gr', 116700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Coffee Milk Pillow Bag (2 sch)', 10300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea Refill 500 gram', 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Cookies Paket Hampers Idul Fitri Parcel Lebaran', 131600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Lemon-C (25 sch)', 25200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim High Fiber High Calcium Milk Chocolate 500gr', 116700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lemon Tea 500 gram', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Peras Refill 500 gr', 37800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['LOKALATE KOPI PISANG BAKAR EPE', 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 - Tropicana Slim Mint Cocoa - Minuman Cokelat Mint Nikmat Tanpa Gula Pasir', 20000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Es Pisang Ijo 10 Sch x 2 pcs', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '(Near ED)L-Men Platinum Kacang Hijau 800g', 158750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'L-Men Platinum Choco Latte 800 gr - Near ED', 164500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari RTD Jeruk Madu 200 ml', 6000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari Jeruk Manis 500gr', 38100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'L-Men Gain Mass Mangga 500gr', 133900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Avocado Chocolate (10 Sch)', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo White Chocolate (10 Sch)', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Diabetamil Milk Vanilla 150 gram', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Buy 1 Get 1 FREE Tropicana Slim Susu Low Fat Macchiato Coffee 500gr', 143000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Active Chocolate 1000 gr', 121200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Joint Plus Orange 140 gram', 38700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Teen Biscuit Caramel 500gr', 65000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Active Caramel Latte 500g', 65800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Buy 1 Get 1 Free - HiLo Gold Biscuit Cereal 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12DX500G', 76200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo School Bubble Gum 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '2102500336 (CS)', 49200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Thai Tea Refill 500g', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Hilo Teen Coffee Tiramisu Ready To Drink Susu UHT [4 pcs]', 27600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '(Near ED)L-Men Platinum Ketan Hitam 800g', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 46800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari Gula Asem 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari Anggur Hijau 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Es Pisang Ijo 10 Sch', 14300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Buy 1 Get 1 FREE L-Men Lose Weight Mango Sticky Rice 300gr', 145200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE DEALS - NutriSari Lychee Tea & Lemon Tea 500 gram Refill', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - NutriSari Lemon Tea 500 gram', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - HiLo Thai Tea Refill 500g', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - HiLo Klepon Latte Refill 500g', 109100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Bundle - Lokalate Kopi Alpukat Refill 500 gram & Kopi Berondong Refill 500 gram', 46200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - NutriSari Lychee Tea Refill 500 gram', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '(Near ED)L-Men Gain Mass Mangga 500gr', 66350]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Lokalate Kopi Alpukat 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 - Tropicana Slim Bumbu Kaldu Ayam dan Jamur 100 gram", 48500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - HiLo Platinum Original 360 gram", 255000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo School Chocolate 1000 gr", 148700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - HiLo Teen Chocolate 1000 gr", 148700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Joint Plus Orange 140 gram", 19150]], columns = ['title', 'price']), ignore_index = True, sort = False)
   
    product_order = product_order.append(pd.DataFrame([["Twinpack Lokalate Kopi Berondong Refill 500 gram", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - Tropicana Slim Soy Latte - Kopi Plant Based Rendah Gula", 40900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twinpack Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 29000]], columns = ['title', 'price']), ignore_index = True, sort = False)       
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Active Caramel Latte 500g", 65200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Active Chocolate 1000 gr", 120100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 12 GET 12 - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C", 240000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Lokalate Kopi Pisang Bakar Epe 10 sch", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Buy 3 Get 3 - HiLo Milky Vanilla Cookies Ready to Drink 200ml', 60000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) - Tropicana Slim Sweetener Classic Stick Industrial (100 Sch)', 26640]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 115000]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo Teen Original 400g - Susu Tinggi Kalsium Lebih Rendah Lemak', 191400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Near ED - HiLo Gold Plain 1000 gr', 144300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo School Original 400g - Susu Tinggi Kalsium Lebih Rendah Lemak', 191400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Peanut Almond Butter Spread Jam 300 gram', 184800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Orange Cocoa 4 Sachet - Minuman Cokelat Jeruk Nikmat Tanpa Gula Pasir', 16650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twinpack HiLo Platinum Original 360gram', 230900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Es Ketan Hitam 500g - Powder Drink Tinggi Kalsium', 46200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 24000]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['HiLo Gold Chocolate 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 147800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Orange Cocoa 4 Sachet - Minuman Cokelat Jeruk Nikmat Tanpa Gula Pasir', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 2 GET 2 - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Taro 500g x 2 pcs', 267800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Taro Latte Refill 500g', 48500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Platinum Swiss Chocolate (12 Sch)', 189300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hampers Lebaran Fancy Tote Bag Tropicana Slim", 186500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 2 GET 2 - Tropicana Slim RTD Almond Drink Chocolicious 190 ml", 33400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo RTD Milky Vanilla Cookies 200ml", 7700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 12 GET 12 - Tropicana Slim RTD Almond Drink Chocolicious 190 ml", 199800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Platinum Bubble Gum 800 gram", 375200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Less Sugar Belimbing 40 sachet", 62000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Wdank Madu Temulawak 10 Sachet", 18000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 2 Get 2 Free - L-Men UHT PlantProtein 2GO Ogura with 12g Protein & 1,6g BCAA", 61600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim RTD Almond Drink Chocolicious 190 ml", 8700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) L-Men Platinum Bubble Gum 800 gram", 375200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 Free - HiLo Teh Tarik (10 Sachet) - Teh Tarik Tinggi Kalsium Bervitamin", 32190]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Isotonik 40 Sachet – Minuman Isotonik Buah Jeruk Segar", 62000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teh Tarik (10 Sachet) - Teh Tarik Tinggi Kalsium Bervitamin", 16095]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 12 GET 12 - Tropicana Slim Oat Drink Japanese Cantaloupe Melon", 240000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Wdank Madu Temulawak 10 Sachet", 17316]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - Wdank Sarabba Extra 10 Sachet", 18870]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram", 82140]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - L-Men Gain Mass Klepon Latte 500g", 146520]], columns = ['title', 'price']), ignore_index = True, sort = False)

    
    price = product_order[product_order['title'] == 'Tropicana Slim Goldenmil Vanilla (6sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Lokalate Kopi Kawista (10sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Lokalate Kopi Kawista (10sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Kawista', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Tropicana Slim Kecap Manis 200ml']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Manis 200m', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    price = product_order[product_order['title'] == 'Tropicana Slim Diabtx 50 sch']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Diabtx (50 sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)
    product_order['title'] = product_order['title'].astype(str)

    data_order = data_order.merge(data_SKU[['SKU', 'product']].drop_duplicates('product'), how = 'left', on = 'product')
    data_order = data_order.merge(product_order[['title', 'price']].drop_duplicates('title'), how = 'left', left_on = 'product', right_on = 'title').drop('title', axis = 1)
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    indeks
    
    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Jakarta'

        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        indeks = data_order[data_order['Item Name'].astype(str).str.contains('pcs')].index.to_list()
        temp = data_order.iloc[indeks].copy()
        product = temp['Item Name'].str.split("\(",1, expand = True)

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        index = order_all[order_all['SKU'].astype(str) == '2306551174'].index.to_list()
        order_all['SKU'][index] = '2306592173'
        order_all['Real SKU'][index] = '2306592173'
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan

        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Price' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)
        
        order_all = order_all.reset_index()

        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        
        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        order_all['Selling Price'] = order_all['Regular Price'].astype(int)

        data_SKU3 = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
        data_SKU3 = data_SKU3.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
        data_SKU3['SKU'] = data_SKU3['SKU'].astype(str).str.replace('.0', '', regex = False)
        data_SKU3 = data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0'][data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0']['Harga Coret'].notnull()]
        data_SKU3['product'] = data_SKU3['product'].str.strip()
        data_SKU3 = data_SKU3.reset_index(drop = True)

        for i in range(data_SKU3.shape[0]):
            if data_SKU3['SKU'][i] in order_all['SKU'].values:
                indeks = order_all[order_all['SKU'].astype(str) == i].index.to_list()
                order_all['Regular Price'][indeks] = data_SKU3['Harga Display'][i]
                order_all['Selling Price'][indeks] = data_SKU3['Harga Coret'][i]

        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan

        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]
        
        order_all_append['True datetime'] = pd.to_datetime(order_all_append['True datetime'])

        indeks = order_all_append[
            (order_all_append['True datetime'] > pd.to_datetime('2021-02-28')) &
            (order_all_append['Channel'] == 'Order Online Jakarta')
        ].index.to_list()

        order_all_append['Channel'][indeks] = 'Order Online Sumsel'

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_jkt = True
        del file_name

if not order_online_jateng:
            
    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_jateng')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customer1@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("jatenghomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(30)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name), error_bad_lines=False, engine="python")
    
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')

    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Tropicana Slim Oat Drink 190 ml \(RTD\) x 4 pcs$', 'Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs (x1)')
    data_order=data_order[data_order['product']!="2102500336 (CS)"]
    data_order=data_order.reset_index(drop=True)
    
    product = data_order['product'].str.split("\(x",1, expand = True)
    product = product[product[0] != 'nan']
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("Rtd", "RTD").str.replace('Turmeric 180gr', 'Turmeric (6 sch)').str.replace('L-Men Gainmass Taro 225gr', 'L-Men Gain Mass Taro 225 gr').str.replace('L-Men Lose Weight Chocolate Cereal (12sch)', 'L-Men Lose Weight Chocolate Cereal 300gr').str.replace(' (+kaos)', ' (+Kaos)', regex = False).str.replace('40 sch', '40sch', regex = False).str.replace('Coklat', 'Chocolate', regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("Rtd", "RTD").str.replace('Turmeric 180gr', 'Turmeric (6 sch)').str.replace('L-Men Gainmass Taro 225gr', 'L-Men Gain Mass Taro 225 gr').str.replace('L-Men Lose Weight Chocolate Cereal (12sch)', 'L-Men Lose Weight Chocolate Cereal 300gr').str.replace(' (+kaos)', ' (+Kaos)', regex = False).str.replace('40 sch', '40sch', regex = False).str.replace('Coklat', 'Chocolate', regex = False)


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Platinum Swiss Chocolate 420gr', 91500, 91500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim White Coffee (4 sch)', 21200, 18400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-MEN Gain Mass Taro 225 gr', 58300, 58300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500ml', 85800, 85800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Peras Refill 500gr', 34300, 34300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500gr', 125800, 125800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Manis Refill 500gr', 37800, 37800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Roasted Sesame Mayonnaise 200g - Bantu Dukung Hidup Sehat', 22900, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Shirataki Rice 72 g', 50000, 50000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Nipis Jahe 40 Sachet', 62500, 35000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Chocolate 250g (Health & green Science Competition)', 38300, 38300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Kacang Hijau Ready to Drink 200ml (4 tetrapack)', 22900, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Shirataki Noodles 71gr', 9700, 9700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Kacang Hijau 200 ml', 2850, 2850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Brown Sugar 200 ml', 2550, 2550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HILO TEEN STRAWBERRY MILKSHAKEÂ\xa0 12Dx500G', 66800, 66800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Cocopandan 40sch', 21300, 21300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Lemon Tea (40sch)', 42600, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Mayonnaise 200 g', 11450, 11450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Caramel Latte 500g', 95000, 50500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Cotton Candy 500g', 88400, 62500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Oat Drink 190ml (RTD)', 9999, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo school Bubble Gum 500g', 154000, 92400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 59400, 59400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 237600, 199000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 59400, 49500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230g - Bantu Dukung Hidup Sehat', 59400, 59400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HILO ALMOND MILK COCONUT 12Dx200G', 51500, 51500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Bubble Gum 500g', 80100, 80100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Lemon Tea 500 gram', 42000, 34300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Teen Biscuit Caramel 500gr', 92400, 65000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE L-Men Lose Weight Mango Sticky Rice 300gr', 290400, 145200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 - Tropicana Slim Mint Cocoa - Minuman Cokelat Mint Nikmat Tanpa Gula Pasir', 40000, 20000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Es Pisang Ijo 10 sch x 2 pcs', 35000, 17500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NuTrilogi Seri Mas Kaka yang Banyak Akal', 70000, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE Tropicana Slim Susu Low Fat Macchiato Coffee 500gr', 257400, 143000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Lokalate Kopi Pisang Bakar 10 sch', 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Choco Series Paket Hampers Idul Fitri Parcel Lebaran', 185300, 139600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Brownies Instant Powder Cake Mix 230g - Bantu Dukung Hidup Sehat', 31200, 31200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Soy Latte - Kopi Plant Based Rendah Gula', 81800, 36900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 75000, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Taro 500gr - Khusus Homdel', 80800, 40400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Jeruk Madu 200 ml (4 tetrapack)', 24000, 24000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Bandrek 10 sachet Renceng - Khusus Homdel", 17300, 8650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Empon-Empon 10 sch - Khusus Homdel", 15000, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 1 Get 1 Free - HiLo Gold Biscuit Cereal 500g", 161600, 80800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula", 59400, 44400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) HiLo Joint Plus Orange 140 gram", 38300, 19150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) HiLo Active Es Teler 175gr", 29700, 14850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) Lokalate Kopi Tape Ketan 10's", 14900, 7450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sweetener Jahe (50 sch)", 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Anggur Hijau 40 Sachet", 62000, 43290]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Gula Asem 40 Sachet", 62000, 43290]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack - Tropicana Slim Low Fat Milk Korean Strawberry 500g", 257400, 159900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Temulawak Madu", 42000, 33300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack - Tropicana Slim Low Fat Milk Korean Strawberry 500g", 257400, 159900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Temulawak Madu", 42000, 33300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([["TRIPLE HEMAT - HiLo Taro Latte Refill 500g", 145500, 101800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["TRIPLE HEMAT - HiLo Thai Tea Refill 500g", 144000, 100900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["TRIPLE DEALS - NutriSari Lychee Tea & Lemon Tea 500 gram Refill", 103900, 72700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twinpack Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 58000, 29000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Near ED - Lokalate Kopi Andaliman 10 Sachet", 15000, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo RTD Chocolate Avocado 200ml (4pcs)", 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 1 Get 1 - L-Men Gain Mass Klepon Latte", 277100, 138500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G", 86600, 86600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram", 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo school Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium", 28600, 28600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 23100, 23100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Wdank Madu Temulawak 10 Sachet", 17300, 17300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Pisang Bakar Epe 10 sch', 15000, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C", 24000, 17500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Caramel Latte 500g', 71600, 35800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['Twinpack HiLo Platinum Original 360gram', 230900, 129200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Strawberry Milkshake 500g', 87700, 43850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Klepon Latte Refill 500g', 51900, 15570]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 25000, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - NutriSari Jeruk Nipis Jahe 40 Sachet', 47300, 14190]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Sweetener Rose Vanilla 50 sch', 45000, 13500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Multigrain Original 500g', 99300, 49650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Gula Aren 50 sachet', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 144100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Es Ketan Hitam Refill 500g', 46200, 23100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Less Sugar Belimbing 40 sachet', 62000, 45510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim French Butter Souffle Coffee 4 Sachet', 16200, 16200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - L-MEN Gain Mass Taro 225 gr', 78500, 39250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Lokalate Kopi Berondong Refill 500 gram', 48500, 14550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Alpukat Refill 500 gram', 48500, 14550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Vanilla 1000g', 146600, 43980]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Goldenmil Vanilla (6 sch)', 33500, 16750]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Salted Caramel Cocoa 4 Sachet', 26000, 16650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 85400, 42700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 1000 gr', 135100, 40530]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Es Pisang Ijo 10 sch', 16700, 8350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)', 88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Korean Garlic Butter Cookies (5 sch)', 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese Cookies (5 sch)', 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Gold Plain (Original) 500gr', 78600, 23500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Jeruk Peras Refill 500 gr', 36300, 18150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Soy Latte - Kopi Susu Kacang Bebas Gula', 37300, 18650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 24500, 7700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Gula Asem 40 Sachet', 47200, 14200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Apel Jeruk 40 Sachet', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Santan Less Fat (5 sch)', 18500, 18500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Choco Hazelnut (10 sch)', 13300, 13300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Biscuit Caramel 500gr', 87700, 26310]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Teen Coffee Tiramisu Ready To Drink Susu UHT [4 pcs]', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate x 4 pcs', 50800, 50800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([["W'dank Bajigur (10 sch)", 18000, 18000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Taro Latte Refill 500g', 46620, 14000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Cocopandan (40sch)', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Biscuit Cereal 500g', 81030, 24400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Bandrek 10 sachet Renceng", 17316, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Low Fat Milk Korean Strawberry 500g', 84360, 59100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Wdank Madu Temulawak 10 Sachet', 17316, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Es Kuwud Nipis', 45520, 31900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Chocolate Ready To Drink (4 tetrapack)', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Vanilla Vegiberi Ready To Drink Susu [200ml/4 pcs]', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False) 

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')
    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Jateng'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        
        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})
        
        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'
        

        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)

        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']
        
        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan
        

        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU',                           
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_jateng = True
        del file_name

if not order_online_bali:

    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_bali')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customerbali@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("balihomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(30)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()
#         data_order = pd.read_csv(r'Input Data/orderonline_orders_all_products_Jakarta.csv')
#             product_order = pd.read_csv(r'Input Data/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD")
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD")


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 38300, 38300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Extra Virgin Olive Oil 500 ml', 42900, 42900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Active Vanilla 500gr', 32600, 32600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Gold Plain (Original) 500gr', 36600, 36600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Chocolate 750gr', 55500, 55500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 750gr', 55500, 55500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Gain Mass Chocolate 500gr', 62900, 62900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Platinum Choco 800 gr - Near ED', 157300, 157300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Chocolate 500gr', 38300, 38300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Brown Sugar 200 ml', 2574, 2574]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Thai Tea 200ml - Near ED', 2860, 2860]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 - L-Men Gain Mass Klepon Latte', 277100, 138500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Garlic Butter Cookies 5 sch - Khusus Homdel', 11150, 11150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 1000 gr', 60050, 60050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men PlantProtein Ogura 216g', 57700, 57700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Caramel Latte 500g', 32600, 32600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 99000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 25000, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 24000, 13400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Twinpack HiLo Platinum Original 360gram', 230800, 129200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 6', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - L-MEN Gain Mass Taro 225 gr', 78500, 39250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Platinum Swiss Chocolate 420gr', 94700, 47350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Blewah (40 sch)', 47300, 23635]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Oat Drink 190 ml (RTD)', 8100, 4050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Milk Skim Chocolate 500gr', 117700, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Jeruk Peras (40 sch)', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Avocado Chocolate (10 sch)', 13650, 6650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari American Sweet Orange (40 sch)', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Florida Orange (40 sch)', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Klepon Latte Refill 500g', 51900, 25950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Biscuit Caramel 500gr', 87700, 43850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 85400, 42700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Goldenmil Vanilla (6 sch) - Near ED', 33500, 16750]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Less Sugar Belimbing 40 sachet', 62000, 45510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)', 88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo RTD Chocolate Avocado 200ml', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Jeruk Peras Refill 500 gr', 190500, 19050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari RTD Jeruk Madu 200 ml', 6000, 3000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Es Ketan Hitam 500g - Powder Drink Tinggi Kalsium', 46200, 46200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Leci 40 sch', 42600, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml (4pcs)', 25200, 25200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Strawberry Jam 375gr', 77300, 38650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Avocado Coffee 4 sch', 15540, 7800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Proteinmix Coffee 6 sachet x 33gr', 61050, 18300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - L-Men Gain Mass Klepon Latte 500g', 146520, 102564]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Low Fat Milk Korean Strawberry 500g', 84360, 42180]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Kanola 946ml', 77700, 54400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen RTD Coklat 200ml - Khusus Homdel', 6882, 3400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Protein Bar Chocolate 12 sch', 102564, 71800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - HiLo Swiss Chocolate (10 sch) Renceng', 19425, 9700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School RTD Coklat 200ML', 6882, 3400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen RTD Coffee Tiramisu 200ml - Khusus Homdel', 6882, 3400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 24420, 7300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sunflower Oil 946 ml', 84360, 25408]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Vanilla 500gr - Khusus Homdel', 81030, 40500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Lemon Tea 500 gram', 34965, 17483]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Lokalate Kopi Berondong Refill 500 gram', 46620, 14000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari NUTRI C1000 Jeruk 40 Sachet', 45510, 22800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Vanilla 1000g', 149850, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel', 46620, 23300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Madu Lemon (40 sch)', 45510, 22800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) NutriSari Es Rujak 40's", 45510, 13650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Markisa (40 sch)', 45510, 22800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Lemon-C (25 sch)', 25530, 7700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Es Kuwud Nipis', 45510, 13650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sweetener Gula Aren 50 sachet', 43290, 13000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Es Pisang Ijo 10 sch', 16095, 4829]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) Tropicana Slim Madu 350ml", 66600, 46600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Thai Tea (10sch) - Near ED', 16095, 11300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Salted Caramel Cocoa 4 Sachet', 26000, 16650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml - Near ED', 6105, 1832]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Choco Hazelnut (10 sch)', 13300, 13300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Anggur (40 sch)', 42600, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Sweet Mango (40 sch)", 42600, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Diabtx (25 sch)', 22900, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Strawberry 40 sch', 42600, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Teen Chocolate 250gr', 39500, 39500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Nipis (40 sch)', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'dank Bajigur (10 sch)", 18000, 18000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Vanilla 750 gr', 122100, 85500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Es Ketan Hitam 10 sch', 16095, 11300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Es Rujak Jeruk Bali 40 Sachet', 22300, 22300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Sarabba Ekstra", 46000, 37740]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Jeruk Extra Manis (40 sch)", 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Plain 750gr', 117600, 82400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Gold Plain 1000 gr', 142080, 82400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Sirsak (40 sch)', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Near ED - NutriSari Apel Jeruk 40 Sachet", 45510, 13650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Bandrek 10 sachet Renceng - Khusus Homdel", 17316, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Soy Latte - Kopi Susu Kacang Bebas Gula', 37740, 18000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Chocolate 1000 gr', 149850, 74900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Platinum Original 360g (12 Sachet)', 111000, 77700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Near ED - NutriSari Anggur Hijau 40 Sachet", 45510, 31900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) NutriSari Milky Orange (40 sch)", 45510, 31900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Kecap Asin 200 ml', 25530, 17900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Wdank Madu Temulawak 10 Sachet', 17316, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Madu Kurma 40 sch', 45510, 13650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Orange Cocoa 4 sachet', 16650, 11700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Lemon Tea 40 sch', 45510, 22800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Classic Industrial (125 Sachet)', 74370, 37200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
   
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Bali'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})


        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_bali = True
        del file_name

if not order_online_makasar :
                    
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_makasar')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customermakassar@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("makassarhomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(30)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()
#         data_order = pd.read_csv(r'Input Data/orderonline_orders_all_products_Jakarta.csv')
#             product_order = pd.read_csv(r'Input Data/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)
    data_order['product'] = data_order['product'].str.replace("Tropicana Slim Goldenmil Vanilla Turmeric 180gr", 'Tropicana Slim Goldenmil Vanilla Turmeric (6 sch)')
    data_order['product'] = data_order['product'].str.replace("NutriSari Mango Smoothie Ready to Drink Jus 200ml (6pcs)", 'NutriSari Mango Smoothie 200ml (6pcs)', regex = False)
    
    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml', 6000, 6000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (24pcs)', 152400, 152400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Wdank Sarabba Extra 10 Sachet', 19600, 19600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Wdank Madu Temulawak 10 Sachet', 17300, 9700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 80700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Milky Vanilla Cookies 200ml', 7700, 7700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (4pcs)', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Twinpack HiLo Platinum Original 360gram', 230900, 129200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 3', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 4', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 17760, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Less Sugar Belimbing 40 sachet', 62000, 45510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Thai Tea 200ml - Near ED', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Vanilla Cookies 200ml', 7700, 3850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml - Near ED', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School RTD Cotton Candy 200ml', 7200, 3600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo RTD Chocolate Avocado 200ml', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen RTD Coffee Tiramisu 200ml - Khusus Homdel', 7200, 3600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School RTD Vanilla Vegiberi 200ml', 7200, 3600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Brown Sugar 200 ml', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School RTD Coklat 200ML', 7200, 3600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Lemon Tea 40 sch', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen RTD Coklat 200ml - Khusus Homdel', 7200, 3600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Hi Protein 2 Go Chocolate - Khusus Homdel', 12700, 3810]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari RTD Jeruk Madu 200 ml', 6000, 1800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)', 88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Es Ketan Hitam Refill 500g', 46200, 13860]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-MEN PROTEIN CRUNCH BBQ BEEF 20Bx20G', 12700, 3810]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Bundle Special Pack Isi 6 pcs - NutriSari RTD Squeezed Orange 200 ml', 35700, 35700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Chocolate Ready To Drink (4 Tetrapack)', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Milky Orange (40sch)', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Alpukat Refill 500 gram', 48500, 14550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 24600, 24600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Goguma Cookies (5 sch)', 7000, 7000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Lose Weight Mango Sticky Rice 300gr', 44000, 44000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Thai Tea Refill 500g', 46620, 23310]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Chocolate Spread 300g - Near ED', 42735, 42735]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sweetener Honey 50 sachet', 43290, 21645]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Strawberry Jam 375gr', 74370, 37185]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari NUTRI C1000 Jeruk 40 Sachet', 45510, 22755]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim White Coffee', 9713, 9713]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Jeruk Peras Refill 500 gr', 17200, 17203]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Kanola 946ml', 54400, 54400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Belgian Chocolate (10 sch) - Near ED', 51060, 25530]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Susu Vanilla 10 sch - Near ED', 46620, 23310]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Protein Bar Chocolate 12 sch', 71800, 51282]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Makassar'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_makasar = True
        del file_name

if not order_online_medan:
                        
    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_medan')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customermedan@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("medanhomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(30)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()
#         data_order = pd.read_csv(r'Input Data/orderonline_orders_all_products_Jakarta.csv')
#             product_order = pd.read_csv(r'Input Data/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Tropicana Slim Oat Drink 190 ml \(RTD\) x 4 pcs$', 'Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs (x1)')
    
    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men', case = False).str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)
    data_order['product'] = data_order['product'].str.replace("NutriSari Jeruk Nipis 40'sachet (4 Renceng)", 'NutriSari Jeruk Nipis (40 sch)', regex = False)
    data_order['product'] = data_order['product'].str.replace("Tropicana Slim Sweetener Honey 50 sachet", 'Tropicana Slim Sweetener Honey (50sch)', regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Teen Coklat 250gr", 'HiLo Teen Chocolate 250gr', regex = False)
    data_order.loc[data_order['product']=='HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G','product']="HiLo Teen Strawberry Milkshake 500gr"
    


    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men', case = False).str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Garlic Butter Cookies 5 sch - Khusus Homdel', 11500, 11500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Active Vanilla 500gr', 35450, 35450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Jagung 946ml - Khusus Homdel', 50800, 50800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari RTD Jeruk Madu 200 ml', 5900, 2950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Teen Vanilla Caramel 500gr', 43300, 43300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - NutriSari Jeruk Nipis Jahe 40 Sachet', 44600, 22300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Lokalate Kopi Andaliman 10 Sachet', 7450, 7450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (24 Pcs)', 171800, 171800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (4pcs)', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - HiLo School Cotton Candy 500g', 80100, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 16650, 14153]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 750gr', 97200, 48600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Caramel Latte 500g', 65200, 35600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Twinpack HiLo Platinum Original 360gram', 230800, 129200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Strawberry Milkshake 500g', 86600, 43300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Santan 5 sachet', 16600, 8300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Sweetener Rose Vanilla 50 sch', 44600, 22300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Lose Weight Avocado Coffee 300g', 138200, 69250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium', 28600, 28600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 17760, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 99000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 4', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 3', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Gula Aren 50 sachet', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Less Sugar Belimbing 40 sachet', 62000, 45510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School RTD Coklat 200ML', 7200, 7200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (24pcs)', 152400, 152400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED BUY 1 GET 1 Tropicana Slim Extra Virgin Olive Oil 500 ml', 184800, 92400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Gain Mass Chocolate 500gr', 152400, 76200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Kecap Asin 200 ml', 26600, 13300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Vanilla Cookies 200ml', 7700, 3850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Classic Refill 500gr', 109700, 54850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Platinum Swiss Chocolate 420gr', 94700, 47350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Near ED - L-Men Gain Mass Taro 225 gr', 157000, 78500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Shirataki Noodles 71gr', 21400, 10700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Madu 350ml', 69300, 34650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Milk Skim Coffee 500gr', 117700, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 85400, 42700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Stevia (50 sch)', 61200, 30600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Es Kuwud Nipis', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Multigrain Original 500g', 99300, 49650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - (Near ED) Tropicana Slim Shirataki Noodles 71gr', 42800, 21400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari NUTRI C1000 Jeruk 40 Sachet', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim High Fiber High Calcium Milk Chocolate 500gr', 117700, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Chocolate 500gr', 87700, 43850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Biscuit Caramel 500gr', 84600, 42300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School Bubble Gum 500g', 109700, 54850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sweetener Gula Aren 50 sachet', 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Vanilla 500gr - Khusus Homdel', 84300, 42150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Vanilla 1000g', 146600, 73300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Es Ketan Hitam Refill 500g', 46200, 23100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Platinum Choco 800 gr - Near ED', 375200, 187600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim BROWNIES Instant Powder Cake Mix 230 g', 34360, 17300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Kecap Manis 200ml', 28900, 14450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Es Pisang Ijo 10 sch', 16700, 8350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cokelat 30gr (10sch)', 33300, 33300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Biscuit Cereal 500g', 84300, 42150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - Near ED', 30900, 15450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Kecap Manis 200ml', 28900, 14450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)', 88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Lemon Tea 40 sch', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Chocolate Ready To Drink (4 Tetrapack)', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Choco Hazelnut (10 sch)', 13300, 13300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Milky Orange (40sch)', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Nipis (40 sch)', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 51900, 36330]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)', 88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School RTD Vanila Vegiberi 200ml (6pcs)', 43200, 43200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Strawberry Jam 375gr', 77300, 38650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 304800, 304800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 25400, 12700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Diabetamil Milk Vanilla 150 gram', 33500, 16750]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Kanola 946ml', 80800, 40400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 77000, 38500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs', 32400, 32400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Kelapa Muda (40 sch)', 62000, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Twin Pack: Tropicana Slim Saus Tiram 200ml', 64600, 32300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Avocado Coffee (4 sch)', 16300, 11340]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500 gr', 152400, 152400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Kecap Asin 200ml', 26600, 26600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Kacang Hijau 200 ml - Near ED', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Es Rujak Jeruk Bali 40 Sachet', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Gold Sweet Potato 500 gram', 42150, 42150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Protein Crunch BBQ Beef (20gr)', 12700, 6350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Chocolate 750gr', 127000, 63500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Plain 750gr', 122400, 61200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Yoghurt Smootie Bowl Strawberry 8s x 30g', 68100, 34050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Chocolate Candy (10 sch) - Near ED', 23100, 11550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Milk Skim Original 1kg', 207800, 103900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Gold Sweet Potato 500 gram', 42150, 42150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack HiLo RTD Thai Tea 200ml - Near ED', 25200, 12600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Chocolate Spread 300g - Near ED', 76600, 38300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Thai Tea 200ml (6pcs) - Ready to Drink', 35400, 35400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Chocolate 250gr', 40000, 20000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Teen Chocolate Ready To Drink Susu [4 pcs]', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Hokkaido Cheese Cookies', 24200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Jagung 946ml - Khusus Homdel', 101600, 50800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Es Ketan Hitam 500g - Powder Drink Tinggi Kalsium', 46200, 32340]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School 3+ Soya Vanilla Malt 400g', 83250, 58300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'dank Sarabba Ekstra", 18870, 13290]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Sarabba Ekstra", 18870, 13290]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Milk Low Fat Vanilla 500gr', 82140, 41070]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Wdank Madu Temulawak 10 Sachet', 17316, 8658]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 43290, 21645]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Saus Tiram 200ml', 31080, 15540]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sunflower Oil 946 ml', 84360, 59100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')

    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)


    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Medan'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        
        
        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        #order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_medan = True
        del file_name

if not order_online_samarinda:
                            
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_samarinda')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customersamarinda@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("samarindahomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(10)

    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(30)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Lose Weight Avocado Coffee 300g', 69250, 69250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari American Sweet Orange (40 sch)', 45000, 43000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Twinpack HiLo Platinum Original 360gram', 230800, 129200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 24000, 13400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 80700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (4pcs)', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Gula Aren 50 sachet', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 25000, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Gula Aren 50 sachet', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium', 28600, 28600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Salted Caramel Cocoa 4 Sachet', 26000, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Chocolate 1000g', 147800, 73900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - L-MEN Gain Mass Taro 225 gr', 78500, 23550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim French Butter Souffle Coffee 4 Sachet', 16200, 16200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Multigrain Original 500g', 99300, 49650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Multigrain Original 500g', 99300, 99300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Nipis (40 sch)', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'dank Empon-Empon 10 sch", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500 gr', 152400, 152400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs', 29700, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Near ED - Tropicana Slim Low Fat Milk Korean Strawberry 500g", 84360, 42200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Biscuit Cereal 500g', 81030, 24300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Platinum Original 360g (12 Sachet)', 111000, 55500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Alpukat Refill 500 gram', 46620, 14000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Vanilla 1000g', 149850, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 82140, 24600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
   
       
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)


    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Samarinda'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})


        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)
        data_all_aft_order = data_all.copy()
        del file_name
        order_online_samarinda = True
        
if not order_online_lampung:
                            
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_lampung')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customerlampung@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("lampunghomdel")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(30)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()


    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch').str.replace('6 tetrapack', '6 Tetrapack').str.replace("Diabetx 100s", "Diabtx (100 sch)").str.replace("Alpukat 10's", "Alpukat (10sch)").str.replace("Kawista 10 sch", "Kawista (10sch)")




    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch').str.replace('6 tetrapack', '6 Tetrapack').str.replace("Diabetx 100s", "Diabtx (100 sch)").str.replace("Alpukat 10's", "Alpukat (10sch)").str.replace("Kawista 10 sch", "Kawista (10sch)")


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel', 45800, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel', 45800, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel', 45800, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - NutriSari Lychee Tea 40 sch', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Orange Cocoa 4 Sachet - Minuman Cokelat Jeruk Nikmat Tanpa Gula Pasir', 40000, 33300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 290000, 22200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) NutriSari RTD Jeruk Madu 200 ml", 5900, 2950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Kecap Asin 200 ml', 13150, 13150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Chocolate 500gr', 86600, 43300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Caramel Latte 500g', 65200, 32600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 1000 gr', 120100, 60050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 25000, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Teen Vanilla Caramel 500gr', 86600, 43300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Pisang Bakar Epe 10 sch', 15000, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Goldenmil Vanilla (6 sch) - Near ED', 57700, 28850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Stevia (50 sch)', 33500, 16750]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 80700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) Lokalate Kopi Berondong 10's", 14900, 7450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 24000, 13400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - HiLo School Cotton Candy 500g', 80800, 40400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   

    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium', 28600, 28600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari NUTRI C1000 Jeruk 40 Sachet', 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Soy Latte - Kopi Susu Kacang Bebas Gula', 36900, 18450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Garlic Butter Cookies 5 sch - Khusus Homdel', 24200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Thai Tea 200ml - Near ED', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Santan 5 sachet', 18500, 9250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Twin Pack: HiLo Platinum Swiss Chocolate (12 sch)', 189300, 169300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Choco Hazelnut (10 sch)', 12210, 6105]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 750gr', 109700, 54850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Whey Advanced Choco Vanilla 500gr', 184700, 92350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Strawberry Milkshake 500g', 86600, 43300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Platinum Bubble Gum 800 gram', 375200, 187600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium', 28600, 28600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Vanilla Cookies 200ml', 7700, 3850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Lychee Tea Refill 500 gram', 36400, 18200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Vanilla 1000g', 146600, 73300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Avocado Coffee 4 sch', 16200, 8100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim White Coffee', 20200, 10100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Goguma Cookies (5 sch)', 24200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Bundle Special Pack Isi 6 pcs - NutriSari RTD Squeezed Orange 200 ml', 35700, 35700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Klepon Cookies (5 sch)', 24200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Less Sugar Belimbing 40 sachet', 62000, 45510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Active Vanilla 500gr', 79700, 39850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Platinum Swiss Chocolate 420gr', 94700, 47350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School Chocolate 250g', 38300, 19150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Lose Weight Mango Sticky Rice 300gr', 152400, 76200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim French Butter Souffle Coffee 4 Sachet', 16200, 16200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - L-MEN Gain Mass Taro 225 gr', 62900, 31450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - NutriSari Jeruk Nipis Jahe 40 Sachet', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School RTD Cotton Candy 200ml',7200, 7200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Oat Drink 190 ml (RTD)',8100, 8100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Klepon Latte Refill 500g',51900, 25950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Wdank Madu Temulawak 10 Sachet',18000, 18000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Kanola 946ml',80800, 40400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Milk Skim Chocolate 500gr',117700, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Es Ketan Hitam Refill 500g',46200, 46200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Chocolate 1000g',147800, 73900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sirup Leci 750ml',33500, 33500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Saus Tiram 200ml',32300, 16150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Extra Virgin Olive Oil 500 ml',92400, 92400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sirup Cocopandan 750ml',33500, 33500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sirup Orange 750ml',33500, 33500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sirup Orange 750ml',33500, 33500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)',88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Mint Cocoa 4 Sachet', 17300, 8650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Alpukat Refill 500 gram', 48500, 24250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Lokalate Kopi Alpukat (10sch) - Near ED', 14900, 7450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Chocolate Spread 300g - Near ED', 80100, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - Near ED',34600, 17300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Hi Protein 2 Go Chocolate - Khusus Homdel',11500, 6350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - NutriSari Lychee Tea 40sch', 23650, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED -Tropicana Slim Sweet Orange Sugar Free 10 sch',21900, 10950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Nutty Chocolate Cookies 200gr',48500, 48500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Strawberry Jam 375gr',77300, 77300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School RTD Coklat 200ML',3600, 3600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen RTD Coffee Tiramisu 200ml - Khusus Homdel',7200, 7200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Jeruk Peras Refill 500 gr',38100, 19050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Platinum Kacang Hijau 800g',314600, 94400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sunflower Oil 946 ml', 87700, 87700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Nanas 40sch', 47300, 47300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen RTD Coklat 200ml - Khusus Homdel',7200, 7200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Hokkaido Cheese Cookies', 24200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Jagung 946ml - Khusus Homdel', 101600, 50800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Apel Jeruk 40 Sachet',45510, 31900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) NutriSari Jeruk Nipis 40'sachet (4 Renceng)", 45510, 31900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Jeruk Peras (40sch)', 45510, 31900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Vanilla 1000g', 149850, 104900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Chocolate Banana 10 sch',45510, 31900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) HiLo Gold Vanilla 500gr - Khusus Homdel", 84300, 42150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Avocado Chocolate (10 sch)', 12765, 8900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Biscuit Caramel 500gr', 84360, 25300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School RTD Vanilla Vegiberi 200ml', 6882, 2100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
   
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Lampung'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})


        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'

        order_all.loc[data_SKU['SKU'].isin(['(E)2HH1407250']),'Price List NFI'] = '6105'
        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)
#         data_all_aft_order = data_all.copy()
        del file_name
        order_online_lampung = True
        

        
if not order_online_pekanbaru:
                            
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_pekanbaru')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("customerpekanbaru@nutrimart.co.id")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("pekanbaruhomdel1")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(35)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()


    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order.loc[data_order['product']=='HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G','product']="HiLo Teen Strawberry Milkshake 500gr"
    
    product = data_order['product'].str.split("\(x",1, expand = True)

    product.loc[product[0]=="HiLo Teen Taro 500gr",1]="1)"
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - HiLo School Cotton Candy 500g', 38850, 38850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 1000 gr', 61500, 61500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sweetener Jahe 50 sch', 44600, 22300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Lose Weight Avocado Coffee 300g', 138500, 69250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Strawberry Milkshake 500g', 86600, 43300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Klepon Latte Refill 500g', 51900, 25950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 25000, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Milk Skim Chocolate 500gr', 117700, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School Vanilla Vegiberi 750gr', 117700, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Avocado Coffee 4 sch', 16200, 8100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Stevia (50 sch)', 57700, 28850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Garlic Butter Cookies 5 sch - Khusus Homdel', 23100, 11500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) NutriSari Es Rujak 40's", 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam', 32100, 32100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) - Tropicana Slim Sweetener Rose Vanilla 50 sch", 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Santan 5 sachet', 16700, 8350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) Tropicana Slim Klepon Cookies (5 sch)", 23100, 11550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Milk Skim Coffee 500gr', 117000, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   

    data_SKU = data_SKU.append(pd.DataFrame([['Wdank Madu Temulawak 10 Sachet', 17300, 9700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) - NutriSari Jeruk Nipis Jahe 40 Sachet", 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 80700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 24000, 13400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   

    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Lokalate Kopi Andaliman 10 Sachet', 15000, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Whey Advanced Choco Vanilla 500gr', 173200, 86600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Platinum Choco 800 gr - Near ED', 357900, 178950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Milky Vanilla Cookies 200ml x 4 pcs', 30900, 30900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   

    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Bajigur 10 sch", 18000, 9000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Goguma Cookies (5 sch)', 24200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Platinum Swiss Chocolate 420gr', 94700, 47350]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (4pcs)', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Madu 350ml', 69300, 34650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Kecap Manis 200ml', 28900, 14450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 2', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Platinum Bubble Gum 800 gram', 375200, 187600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Active Vanilla 500gr', 79700, 39850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Salted Caramel Cocoa 4 Sachet', 33300, 16650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 8', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Lemon-C (25 sch)', 26600, 13300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Gain Mass Chocolate 500gr', 152400, 76200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim French Butter Souffle Coffee 4 Sachet', 16200, 16200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Lose Weight Mango Sticky Rice 300gr', 152400, 76200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium', 28600, 28600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Less Sugar Belimbing 40 sachet', 62000, 45510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Vanilla 1000g', 146600, 73300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml - Near ED', 6300, 4410]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo RTD Chocolate Avocado 200ml', 6300, 4410]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Lychee Tea Refill 500 gram', 36400, 18200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Vanilla Cookies 200ml', 7700, 3850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim High Fiber High Calcium Milk Chocolate 500gr', 117700, 82390]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Es Ketan Hitam Refill 500g', 46200, 23100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Mint Cocoa 4 Sachet', 17300, 12110]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim White Coffee', 20200, 10100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Goldenmil Vanilla (6 sch) - Near ED', 33500, 23450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School RTD Cotton Candy 200ml', 7000, 3500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Thai Tea 200ml - Near ED', 6300, 3150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Gula Asem 40 Sachet', 47300, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Cocopandan 40sch', 23650, 23650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Peanut Almond Butter 300 gram', 66600, 33300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Kecap Asin 200 ml', 26600, 18620]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Taro Latte Refill 500g', 46620, 23300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 24500, 12250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Diabetamil Milk Vanilla 150 gram', 33500, 10050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Biscuit Caramel 500gr', 87700, 26310]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Platinum Refill Choco Latte 800g', 321900, 257500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - L-MEN Gain Mass Taro 225 gr', 78500, 23550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)', 88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 51900, 25950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED -Tropicana Slim Sweet Orange Sugar Free 10 sch', 21900, 6570]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Extra Virgin Olive Oil 500 ml', 92400, 27720]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Vanilla Vegiberi 500gr', 84300, 25290]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Milk Skim Original 1kg', 207800, 62340]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 24200, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - NutriSari Lychee Tea 40sch', 47300, 33110]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sirup Orange 750ml', 33500, 16750]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Bandrek 10 sachet Renceng - Khusus Homdel", 18000, 5400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 304800, 304800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack HiLo Chocolate Mint 10 Sachet Minuman dengan Cokelat Tinggi Kalsium Praktis dalam Sachet", 25530, 12765]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 12 GET 12 - Tropicana Slim Oat Drink Japanese Cantaloupe Melon', 240000, 199800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Classic Refill 500gr', 109700, 32910]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel", 45800, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Susu Vanilla 10 sch - Near ED', 22900, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Diabtx 100s", 83100, 83100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 77000, 38500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School Strawberry Cheesecake 500 gram', 80100, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Chocolate Taro RTD 200ml (4pcs)", 25200, 25200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Vanilla Vegiberi Ready To Drink Susu [200ml/4 pcs]', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Teen Coffee Tiramisu Ready To Drink Susu UHT [4 pcs]', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Teen Chocolate Ready To Drink Susu [4 pcs]', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Thai Tea Ready to Drink 200ml x 4pcs', 25200, 25200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Protein Bar Chocolate 12 sch', 99500, 49750]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Milk Low Fat Vanilla 500gr', 74400, 37200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_SKU = data_SKU.append(pd.DataFrame([["Near ED - Tropicana Slim Low Fat Milk Korean Strawberry 500g", 87700, 43850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sweetener Gula Aren 50 sachet', 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) Tropicana Slim Cafe Latte (10 sch)", 30600, 10300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Teen Vanilla Caramel 500gr', 87700, 43850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Chocolate 1000g', 147800, 73900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED)HiLo Teen Chocolate 500gr", 87700, 43850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Soy Latte - Kopi Susu Kacang Bebas Gula', 28800, 28800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Es Kuwud Nipis', 47300, 33110]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 85400, 42700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - HiLo Platinum Original 360 gram', 255000, 111000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) HiLo Teen RTD Coklat 200ml - Khusus Homdel", 4800, 4800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Hokkaido Cheese Cookies', 16300, 16300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen RTD Coffee Tiramisu 200ml - Khusus Homdel', 4800, 4800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari American Sweet Orange (40sch)', 47300, 22800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sunflower Oil 946 ml', 80800, 59100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Es Pisang Ijo 10 sch', 11300, 11300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Alpukat Refill 500 gram', 48500, 24250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Gain Mass Banana 225gr', 78500, 39250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Minyak Jagung 946ml - Khusus Homdel', 68400, 68400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari RTD Jeruk Madu 200 ml', 4000, 4000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Hi Protein 2 Go Chocolate - Khusus Homdel', 8500, 3663]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo School 3+ Strawberry Pop 400g', 56700, 24309]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Lemon Tea 500 gram', 36400, 10490]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Manis 500gr - Near ED', 25600, 10989]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Gold Chocolate 500gr', 74400, 24309]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   

    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')
    data_order.loc[data_order['product']=='HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G','product']="HiLo Teen Strawberry Milkshake 500gr"


    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)



    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Pekanbaru'

        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})
        
        order_all = order_all.reset_index(drop = True)

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        del file_name
        order_online_pekanbaru = True

if not order_online_banjarmasin:

    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_banjarmasin')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("banjarmasinpromosi.nhd2022@gmail.com")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("nhdcuan")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(35)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)

    product.loc[product[0]=="HiLo Teen Taro 500gr",1]="1)"
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sweetener Jahe 50 sch - Khusus Homdel', 44600, 22300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Multigrain Original 500g', 92400, 92400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Almond Milk Coconut 200gr', 51900, 51900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Squeezed Orange 200 ml x 24 pcs - Minuman Jeruk Peras Vitamin C', 144100, 80700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 24000, 13400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Platinum Choco 800 gr - Near ED', 326000, 163000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Gain Mass Klepon Latte 500g', 152400, 152400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Bumbu Kaldu Ayam Jamur 100 gram', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Sweetener Rose Vanilla 50 sch', 22500, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Gula Aren 50 sachet', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Lokalate Kopi Alpukat Refill 500 gram', 48500, 33950]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim French Butter Souffle Coffee 4 Sachet', 16200, 16200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim 6', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Multigrain Original 500g', 99300, 69510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (4pcs)', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (24pcs)', 152400, 152400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (24 Pcs)', 171800, 171800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 25000, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim RTD Almond Drink Chocolicious 190 ml', 8325, 4200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - HiLo Platinum Original 360g (12 Sachet)', 111000, 77700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - NutriSari Gula Asem 40 Sachet', 45510, 13700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - Tropicana Slim Sweetener Gula Aren 50 sachet', 43290, 13000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Gold Biscuit Cereal 500g', 81030, 24300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Vanilla 1000g', 149850, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 1000 gr', 129870, 39000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    data_SKU = data_SKU.append(pd.DataFrame([['Near ED - L-MEN Gain Mass Taro 225 gr', 75480, 22600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
    #             data_order = data_order.merge(product_order[['title', 'price']].drop_duplicates('title'), how = 'left', left_on = 'product', right_on = 'title').drop('title', axis = 1)
    # data_order = data_order[data_order['SKU'].notnull()]
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    #         s = requests.Session()
    #         s.get("http://tatanama.pythonanywhere.com")
    #         s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    #         r = s.get("http://tatanama.pythonanywhere.com/download")

    #         with open(r'C:\Users\andra.miftah\Demo 9\SKU_File/Master tatanama.xlsx', 'wb') as output:
    #             output.write(r.content)

    #         if os.path.isfile(r'C:\Users\andra.miftah\Demo 9\SKU_File/Master tatanama.xlsx') :    
    #             SKU_append = pd.read_excel(r'C:\Users\andra.miftah\Demo 9\SKU_File/Master tatanama.xlsx')
    #             SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    #             data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    #             data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    # to_excel = data_SKU.to_excel(r'C:\Users\andra.miftah\Demo 9\SKU_File/data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Banjarmasin'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        order_all = order_all.reset_index(drop = True)

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        del file_name
        order_online_banjarmasin = True

if not order_online_jabar:

    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_jabar')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath(r"Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element(By.NAME, "email")
    username.clear()
    username.send_keys("nutrimartbandung1@gmail.com")

    password = driver.find_element(By.NAME, "password")
    password.send_keys("nhdcuancuan")
    password.click()

    driver.find_element(By.CLASS_NAME, "btn-submit").click()
    time.sleep(20)
    #Click Order
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a/span/span'))).click()
    time.sleep(20)
    #Click Order List
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/nav/div/div/ul/li[3]/div/a[1]/span').click()
    time.sleep(20)
    #Click Calendar
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div/span').click()
    time.sleep(20)
    #Click Last 7 Days
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    #Click Apply
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(20)
    #Clicl Export
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/button/span').click()
    time.sleep(20)
    #Click Export to CSV
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[1]/div[3]/div[1]/div/button[1]').click()


    time.sleep(30)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)

    product.loc[product[0]=="HiLo Teen Taro 500gr",1]="1)"
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r"T:\08. Learning Project\Project eCom\Proposal Andra\Order Online\Input Data\SKU buat andra.xlsx")
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (24pcs)', 152400, 152400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Chocolate Avocado 200ml (4pcs)', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml - Near ED', 6300, 3800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Thai Tea 200ml - Near ED', 6300, 3800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['4 pack - NutriSari RTD Squeezed Orange 200 ml - Minuman Jeruk Peras Vitamin C', 24000, 13400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hampers Lebaran Fancy Tote Bag Tropicana Slim', 186500, 130000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam', 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sweetener Jahe 50 sch - Khusus Homdel', 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - Tropicana Slim Sweetener Rose Vanilla 50 sch', 45000, 22500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim French Butter Souffle Coffee 4 Sachet', 16200, 16200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Salted Caramel Cocoa 4 Sachet', 26000, 16650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium', 28600, 28600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Less Sugar Belimbing 40 sachet', 62000, 45510]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Caramel Latte 500g', 65200, 32600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Premium Jus Mangga 10 Sachet', 25000, 17760]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['2102500320 (CI160)', 88800, 88800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 51900, 36330]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'dank Empon-Empon 10 sch", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim RTD Almond Drink Chocolicious 190 ml", 8700, 8700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)


    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Jabar'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        order_all = order_all.reset_index(drop = True)

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Bundle SKU'] = data_bundle1['SKU']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['Bundle SKU'] = data_bundle2['SKU']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Bundle SKU'] = data_bundle3['SKU']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Bundle SKU'] = data_bundle4['SKU']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Bundle SKU'] = data_bundle5['SKU']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Bundle SKU'] = data_bundle6['SKU']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Bundle SKU'] = data_bundle7['SKU']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle SKU', 
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)
        
        
        del file_name
        order_online_jabar = True
        order_online_cond = True
        
print('OK')

order_online_pekanbaru


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7266: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7269: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7276: FutureWarning: The frame.append method is deprecated and will be rem

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7306: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7307: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7308: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pand

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7338: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Milk Skim Coffee 500gr', 117000, 58850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7340: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(pd.DataFrame([['Wdank Madu Temulawak 10 Sachet', 17300, 9700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7341: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7363: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Lemon-C (25 sch)', 26600, 13300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7364: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Gain Mass Chocolate 500gr', 152400, 76200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:7365: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

...

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:8386: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:8443: FutureWarning: The default value of regex will change from True to False in a future version.
  order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)


Unbundling
Pricing


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:8563: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:8581: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  order_all = order_all.append(data_bundle, ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_9660\3724863462.py:8593: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_order = data_order.append(temp , ignore_index = True, sort = False)
C:\Users\steven.nathanael\AppData\Local\Temp\ipyke

order_online_jabar


KeyError: 0

In [16]:
print('done')

done


In [24]:
### optional cek item yang error
### to do: add to code jatim jkt jateng
### add to sheet sisanya
order_all[order_all['Regular Price'].isnull()][['Product Name']].drop_duplicates().values#.to_excel('nhd ed.xlsx',index=False)#.values
#order_all[order_all['Price List NFI'].isnull()][['SKU','Product Name']].drop_duplicates().values#.to_excel('nhd ed.xlsx',index=False)#.values

array([], shape=(0, 1), dtype=object)

In [36]:
order_online_jabar = True

In [37]:
### optional cman utk cek ada kota homdel yang belum ke run atau tidak
for n in [order_online_jatim,order_online_jkt,
          order_online_jateng,order_online_bali,
          order_online_makasar, order_online_medan,
          order_online_samarinda,order_online_lampung,
          order_online_pekanbaru,order_online_banjarmasin,order_online_jabar]:
    print (n)

True
True
True
True
True
True
True
True
True
True
True


In [11]:
for i in data_bundle1.columns:
    print(i)

Order #
No. Order Item L-Men Blibli
AWB
Order date
Customer Name
Kode SKU
Blibli SKU
SKU
Product Name
Qty. Invoiced
Selling Price
Shipping Courier
Kode Merchant
Channel
Order Status
Warehouse Name
Real SKU
Real Nama Produk
Brand
Sub Brand
Parent Item
Parent SKU
Price List NFI
Total Net
Bundle Flag
Date
Month
Year
Quarter
Week
True datetime
Store
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Harga Organik 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Harga Organik 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Harga Organik 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4
Subtotal Produk 4
Harga Display 4
Harga Cost 4
Harga Organik 4
Produk 5
SKU Produk 5
PCS Produk 5
Price List NFI 5
Subtotal Produk 5
Harga Display 5
Harga Cost 5
Harga Organik 5
Produk 6
SKU Produk 6
PCS Produk 6
Price List NFI 6
Subtotal Produk 6
Harga D

In [39]:
#TES NEW PKP
tes = pd.read_csv(r"C:\Users\steven.nathanael\Downloads\Main Sheet (timo) (2)_Full Data_data (20).csv")
tes[(tes['Year of Invoice Date SO'].isin([2023, 2024]))&(tes['Month of Invoice Date SO'].isin(['December','January']))][['Customer SO Parent Desc','Customer SO Desc']].drop_duplicates()

,Customer SO Parent Desc,Customer SO Desc
406,ECOM RETAIL - EMOS,"ENSEVAL PUTERA MEGATRADING - JAKARTA, PT"
34473,ECOM MP - LAZADA,ECOM NFI - MP LAZADA
42767,ECOM RETAIL - SAYUR BOX,"CIBINONG CENTER INDUSTRIAL ESTATE , PT"
43417,ECOM MP - TIK TOK SHOP,ECOM TIK TOK SHOP
43418,ECOM MP - TOKOPEDIA,ECOM NFI - MP TOKOPEDIA
43675,ECOM RETAIL - SAYUR BOX,"KREASI NOSTRA MANDIRI - BOGOR, PT"
44688,ECOM NFI - NHD,ECOM - ORDER ONLINE KALIMANTAN SELATAN
44789,ECOM MP - ORAMI,ECOM - ORAMI
45043,ECOM MP - JD ID,ECOM NFI - MP JD ID
46276,ECOM MP - SHOPEE,ECOM NFI - MP SHOPEE


In [8]:
# Run - 9
### download file tabi di Main_Sheet_(timo)_(2)
### replace file di var data_nubi
### idea: use sql

import pandas as pd
import numpy as np
import os
import glob

data_all = data_all[(data_all['Store Type'] != 'Retail Online') & (data_all['Year'].isin([2023, 2024]))]
data_nubi = pd.read_csv(r"C:\Users\steven.nathanael\Downloads\Main Sheet (timo) (2)_Full Data_data (5).csv")

data_nubi.loc[data_nubi['Customer SO Desc']=='INDOPASIFIK FARMA MEDIKA INDONESIA, PT',['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - INDOPASIFIK FARMA MEDIKA INDONESIA'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['SUKACITA TEKNOLOGI INDONESIA, PT','COSMART']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - JOYBOX(COSMART)'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['FRESHBOX DEPO JKT']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - FRESHBOX'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['GUDANG HAKUNA MATATA PELAUT']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - HAKUNA MATATA PELAUT'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['GOTO SOLUSI NIAGA - CILINCING, PT']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - TOKO NOW'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['SAYUR UNTUK SEMUA - SERPONG, PT (SEGARI)']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - SEGARI'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['KOPI UTAMA INDONESIA, PT']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - COMMON GROUND'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['PERUT AMAN SEJAHTERA, PT']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - TRUFFLE BELLY'
data_nubi.loc[data_nubi['Customer SO Desc'].isin(['ADK BERKAH JAYA, CV']),['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - DINKES SURABAYA'


data_nubi = data_nubi[data_nubi['Customer SO Group Area'].astype(str).str.contains('ECOM RETAIL')]

data_SKU = pd.read_excel(r'SKU_File\data_SKU.xlsx')

data_nubi['Sales Order ID'] = 'Nubi -' + data_nubi['Customer SO Parent Desc'].astype(str)
data_nubi['Channel Order ID'] = 'Nubi -' + data_nubi['Customer SO Parent Desc'].astype(str)
data_nubi['Customer Name'] = 'Nubi Retail Online'

print("Filling Date ====== 7/10")

data_nubi['Date'] = data_nubi['Day of Invoice Date SO']
data_nubi['Month'] = data_nubi['Month of Invoice Date SO']
data_nubi['Year'] = data_nubi['Year of Invoice Date SO']

quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
        ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
data_nubi = data_nubi.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
data_nubi = data_nubi.drop(['Bulan'], axis = 1)
data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
        {'Bulan' : 'January' , 'Number': 1},
        {'Bulan' : 'February' , 'Number': 2},
        {'Bulan' : 'March' , 'Number': 3},
        {'Bulan' : 'April' , 'Number': 4},
        {'Bulan' : 'May' , 'Number': 5},
        {'Bulan' : 'June', 'Number': 6},
        {'Bulan' : 'July' , 'Number': 7},
        {'Bulan' : 'August', 'Number' : 8},
        {'Bulan' : 'September', 'Number' : 9},
        {'Bulan' : 'October' , 'Number': 10},
        {'Bulan' : 'November' , 'Number': 11}])
temp = data_nubi.copy()
temp['Day'] = temp['Date']
temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
data_nubi['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
data_nubi['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day']])

data_nubi['Order Date'] = data_nubi['Day of Invoice Date SO']
data_nubi['Paid Date'] = data_nubi['Day of Invoice Date SO']
data_nubi['Status'] = 'Delivered'
data_nubi['Channel'] = data_nubi['Customer SO Parent Desc']
data_nubi['Store'] = data_nubi['Customer SO Group Area']
data_nubi['Currency Code'] = "IDR"
data_nubi['Warehouse Name'] = data_nubi['Customer SO Desc']
data_nubi['Regular Price'] = data_nubi['Sales Value Netto SO'] / data_nubi['Sales Qty SO']
data_nubi['Store Type'] = 'Retail Online'

indeks = data_nubi[data_nubi['Regular Price'].isnull()].index.to_list()
data_nubi['Regular Price'][indeks] = 0

data_nubi['Selling Price'] = data_nubi['Sales Value Netto SO'] / data_nubi['Sales Qty SO']
indeks = data_nubi[data_nubi['Selling Price'].isnull()].index.to_list()
data_nubi['Selling Price'][indeks] = 0

indeks = data_nubi[data_nubi['Sales Qty SO'] == 0].index.to_list()
data_nubi['Selling Price'][indeks] = 0
data_nubi['Regular Price'][indeks] = 0

indeks = data_nubi[data_nubi['Sales Value Netto SO'] == 0].index.to_list()
data_nubi['Selling Price'][indeks] = 0
data_nubi['Regular Price'][indeks] = 0

data_nubi['Sub Total'] = data_nubi['Sales Value Netto SO']
data_nubi['Gross Sales'] = data_nubi['Sales Value Netto SO']

data_nubi = data_nubi.rename(columns = {'Item Group Code' : 'SKU', 'Item Group' : 'Item Name', 'Sales Qty SO' : 'Quantity'})

data_nubi = data_nubi[['Sales Order ID', 'Channel Order ID', 'Customer Name', 'Order Date', 'Paid Date','Status','Channel', 'Store', 'Currency Code', 'Warehouse Name', 'Regular Price', 'Selling Price',
              'Sub Total', 'Gross Sales', 'SKU', 'Item Name', 'Quantity', 'Date', 'Month', 'Year', 'True datetime', "Week", 'Brand', 'Category']]

data_nubi['Order Date'] = data_nubi['True datetime']
data_nubi['Quarter'] = pd.to_datetime(data_nubi['True datetime']).dt.quarter

print("Filling Brand ====== 5/10")
data_nubi['SKU'] = data_nubi['SKU'].astype(str)
data_nubi['Item Name'] = data_nubi['Item Name'].astype(str)
data_SKU['Real SKU'] = data_SKU['SKU']
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)


data_nubi = data_nubi.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')
data_nubi['Real SKU'] = data_nubi['Real SKU'].astype(str)
data_nubi = data_nubi.merge(data_SKU[['SKU', 'Sub Brand', 'Parent Item', 'Parent SKU','Price List NFI']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
data_nubi['Parent Item'] = data_nubi['Parent Item'].fillna(data_nubi['Item Name'])
data_nubi = data_nubi.drop(['SKU_y'], axis = 1)
data_nubi = data_nubi.rename(columns = {'SKU_x':'SKU'})

indeks = data_nubi[data_nubi['Real SKU'].astype(str) == 'nan'].index.to_list()
data_nubi['Real SKU'][indeks] = data_nubi['SKU'][indeks]

indeks = data_nubi[data_nubi['Real Nama Produk'].astype(str) == 'nan'].index.to_list()
data_nubi['Real Nama Produk'][indeks] = data_nubi['Item Name'][indeks]

data_nubi['Sub Brand'] = data_nubi['Category']

data_nubi['Total'] = data_nubi['Sub Total']
data_nubi['Total Net'] = np.nan

data_nubi = data_nubi.rename(columns={'Channel Order ID' : 'Order #',
                                        'Status' : 'Order Status',
                                        'Order Date' : 'Order date',
                                        'Item Name' :'Product Name',
                                        'Bundle Name' : 'Bundle',
                                        'Shipping Country' : 'Country',
                                        'Shipping Province' : 'Region',
                                        'Shipping City' : 'City',
                                        'Shipping Zip' : 'Zip Code',
                                        'Shipping Address1' : 'Address',
                                        'Shipping Phone' : 'Phone',
                                        'Quantity' : 'Qty. Invoiced',
                                        'Item Price' : 'Regular Price',
                                        'Sub Total' : 'Subtotal'})


temp = data_nubi[data_nubi['Price List NFI'].isnull()]
skuoff=pd.read_excel('data_supp\item gk tau harga.xlsx')
skuoff=skuoff[['Real SKU','Price List NFI']]
skuoff['Real SKU']=skuoff['Real SKU'].astype(str)#.info()
display(temp[~temp['Real SKU'].isin(skuoff['Real SKU'].unique())])
temp=temp.merge(skuoff,how='left',on='Real SKU').rename(columns=({'Price List NFI_y':'Price List NFI'}))
data_nubi=data_nubi[data_nubi['Price List NFI'].notnull()].append(temp)


data_nubi['Price List NFI'] = pd.to_numeric(data_nubi['Price List NFI']).astype(int)
data_nubi['Qty. Invoiced'] = pd.to_numeric(data_nubi['Qty. Invoiced']).fillna(0).astype(int)

data_nubi['Total Net'] = data_nubi['Price List NFI'] * data_nubi['Qty. Invoiced']

data_nubi = data_nubi.reset_index(drop = True)

data_nubi['True datetime'] = pd.to_datetime(data_nubi['True datetime'])
data_nubi['Promo'] = np.nan
data_nubi['Discount MC'] = np.nan
data_nubi['Store Type'] = 'Retail Online'

for i in data_nubi['Brand'].unique():
    indeks = data_nubi[data_nubi['Brand'] == i].index.to_list()
    if i == 'NUTRISARI':
        data_nubi['Brand'][indeks] = 'NS'
    elif i == 'HI LO':
        data_nubi['Brand'][indeks] = 'HiLo'
        
temp = data_nubi[data_nubi['Store'] == 'WARUNG PINTAR DISTRIBUSI - BOGOR, PT'].copy()
data_nubi = data_nubi[data_nubi['Store'] != 'WARUNG PINTAR DISTRIBUSI - BOGOR, PT']

temp['Store'] = 'WARUNG PINTAR'
temp['Channel'] = 'WARUNG PINTAR DISTRIBUSI, PT'
temp['Warehouse Name'] = 'WARUNG PINTAR DISTRIBUSI - BOGOR, PT'

data_nubi = data_nubi.append(temp, ignore_index = True, sort = False)

indeks = data_nubi[data_nubi['Store'] == 'SHOPEE'].index.to_list()
data_nubi['Store'][indeks] = 'Shopee Sell-In'


Filling Date ====== 7/10
Filling Brand ====== 5/10


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_13268\1512677607.py:59: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data_nubi['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week


,Sales Order ID,Order #,Customer Name,Order date,Paid Date,Order Status,Channel,Store,Currency Code,Warehouse Name,...,Category,Quarter,Real SKU,Real Nama Produk,Sub Brand,Parent Item,Parent SKU,Price List NFI,Total,Total Net


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_13268\1512677607.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_nubi=data_nubi[data_nubi['Price List NFI'].notnull()].append(temp)
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_13268\1512677607.py:174: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_nubi = data_nubi.append(temp, ignore_index = True, sort = False)


In [11]:
#CEK SKU KOSONG
data_nubi[data_nubi['Price List NFI'].isnull()][['Store','Real SKU','Real Nama Produk','Qty. Invoiced','Total']]

,Store,Real SKU,Real Nama Produk,Qty. Invoiced,Total


In [9]:
### ini run
from datetime import date
from datetime import datetime, timedelta
import glob
import os
#CUTOFF 1 BULAN
cutoff=str((date.today().replace(day=1)-timedelta(1)).replace(day=1))

#CUTOFF 2 BULAN
#cutoff=str((((date.today().replace(day=1)-timedelta(1)).replace(day=1))-timedelta(1)).replace(day=1))

data_nubinew=data_nubi[data_nubi['True datetime']>=cutoff]

data_nubi

list_of_files = glob.glob('data_nubi\*.xlsx') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

data_nubi=pd.read_excel(latest_file)
# data_nubi['Total Net Before ]
data_nubi=data_nubi[data_nubi['True datetime']<cutoff]
          
data_nubi=data_nubi.append(data_nubinew)

data_nubi.to_excel(f"data_nubi\data_nubi_{date.today()}.xlsx",index=False)

data_nubi\data_nubi_2024-02-02.xlsx


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_13268\1317888647.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_nubi=data_nubi.append(data_nubinew)


In [10]:
#run satuin nama retail
data_nubi.loc[data_nubi['Store']=='ASTRO TECHNOLOGIES - TANGERANG, PT',['Store']]='ECOM RETAIL - ASTRO'
data_nubi.loc[data_nubi['Store']=='RITEL BERSAMA NASIONAL; PT',['Store']]='ECOM RETAIL - JD ID'
data_nubi.loc[data_nubi['Store']=='EMOS',['Store']]='ECOM RETAIL - EMOS'
data_nubi.loc[data_nubi['Store']=='KREASI NOSTRA MANDIRI - BOGOR, PT',['Store']]='ECOM RETAIL - SAYUR BOX JAKARTA'
data_nubi.loc[data_nubi['Store']=='KREAS I NOSTRA MANDIRI - SURABAYA, PT',['Store']]='ECOM RETAIL - SAYUR BOX SURABAYA'
data_nubi.loc[data_nubi['Store']=='CIBINONG CENTER INDUSTRIAL ESTATE , PT',['Store']]='ECOM RETAIL - SAYUR BOX JAKARTA'
data_nubi.loc[data_nubi['Store']=='KREASI TANI LAKSMI - SURABAYA, PT',['Store']]='ECOM RETAIL - SAYUR BOX SURABAYA'
data_nubi.loc[data_nubi['Store']=='ECOM RETAIL - SWIFT BEKASI',['Store']]='ECOM RETAIL - TOKO NOW'
data_nubi.loc[data_nubi['Store']=='ECOM RETAIL - SWIFT JKT UTARA',['Store']]='ECOM RETAIL - TOKO NOW'
data_nubi.loc[data_nubi['Store']=='INOVASI DIGITAL NIAGA - KALIDERES, PT',['Store']]='ECOM RETAIL - BAYI NINJA JAKARTA'
data_nubi.loc[data_nubi['Store']=='INOVASI DIGITAL NIAGA - SURABAYA, PT',['Store']]='ECOM RETAIL - BAYI NINJA JAKARTA'
data_nubi.loc[data_nubi['Store']=='INDOPASIFIK TEKNOLOGI MED INDONESIA, PT',['Store']]='ECOM RETAIL - INDOPASIFIK FARMA MEDIKA INDONESIA'
data_nubi.loc[data_nubi['Store']=='ECOM RETAIL - LIFEPACK',['Store']]='ECOM RETAIL - INDOPASIFIK FARMA MEDIKA INDONESIA'
data_nubi.loc[data_nubi['Store']=='SAYUR BOX',['Store']]='ECOM RETAIL - SAYUR BOX JAKARTA'
data_nubi.loc[data_nubi['Store']=='TANIHUB',['Store']]='ECOM RETAIL - TANIHUB'

In [14]:
data_nubi['Store'].unique()

array(['MITRA TOKOPEDIA', 'Shopee Sell-In', 'ECOM RETAIL - TANIHUB',
       'ECOM RETAIL - JD ID', 'WARUNG PINTAR', 'ECOM RETAIL - EMOS',
       'ECOM RETAIL - SAYUR BOX JAKARTA', 'CHILIBELI', 'ALGOLAB SOLUTION',
       'CARI SAYUR', 'ECOM RETAIL - BAYI NINJA JAKARTA',
       'ECOM RETAIL - INDOPASIFIK FARMA MEDIKA INDONESIA',
       'ECOM RETAIL - SAYUR BOX SURABAYA', 'ECOM RETAIL - ASTRO',
       'KREASI NOSTRA MANDIRI - SURABAYA, PT', 'ECOM RETAIL - TOKO NOW',
       'ECOM RETAIL - JOYBOX(COSMART)', 'ECOM RETAIL - FRESHBOX',
       'ECOM RETAIL - HAKUNA MATATA PELAUT', 'ECOM RETAIL - SEGARI',
       'ECOM RETAIL - COMMON GROUND', 'ECOM RETAIL - DINKES SURABAYA',
       'ECOM RETAIL - TRUFFLE BELLY'], dtype=object)

In [11]:
## ini run
data_all['Store'] = data_all['Store'].astype(str).str.replace('JD Indonesia - Nutrimart', 'JD Indonesia', regex = False)

data_all = data_all[data_all['Store'].isin(['JD Indonesia', 'Lazada', 'Bukalapak', 'Blibli', 'Tokopedia',
       'Shopee', 'Elevenia', 'Nutrimart', 'Order Online','Aladin Mall','TikTok','Orami Shopping'])]
data_all = data_all.append(data_nubi, ignore_index = True, sort = False)

indeks = data_all[data_all['Store'].isin(['JD Indonesia', 'Lazada', 'Bukalapak', 'Blibli', 'Tokopedia',
       'Shopee', 'Elevenia','Aladin Mall','TikTok','Orami Shopping'])].index.to_list()
data_all['Store Type'][indeks] = 'Marketplace'

indeks = data_all[data_all['Store'].isin(['Nutrimart', 'Order Online'])].index.to_list()
data_all['Store Type'][indeks] = 'Organic'

indeks = data_all[data_all['Order #'].astype(str).str.contains('Nubi')].index.to_list()
data_all['Store Type'][indeks] = 'Retail Online'

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_13268\67016908.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_all = data_all.append(data_nubi, ignore_index = True, sort = False)


In [12]:
## ini run juga
indeks = data_all[data_all['Product Name'] == 'L-Men Bar Crunchy Chocolate (12sch)'].index.to_list()
data_all['Real Nama Produk'][indeks] = 'L-Men Bar Crunchy Chocolate 12sch'
data_all['Parent Item'][indeks] = 'L-Men Bar Crunchy Chocolate (12 Sch)'
data_all['Brand'][indeks] = 'L-Men'
data_all['Sub Brand'][indeks] = 'L-MEN POWDER'
data_all['Real SKU'][indeks] = '2306592173'
data_all['Parent SKU'][indeks] = '2306592173'

indeks = data_all[data_all['Product Name'] == 'Heavenly Blush Yogurt Drink To Go Peach (1 pc)'].index.to_list()
data_all['Brand'][indeks] = 'Heavenly Blush'

indeks = data_all[data_all['Brand'] == 'HB'].index.to_list()
data_all['Brand'][indeks] = 'Heavenly Blush'

In [13]:
## ini run juga
data_all['Region Group'] = np.nan

index = data_all[data_all['Region'].isin(['Dki Jakarta', 'Banten'])].index.to_list()
data_all['Region Group'][index] = 'Jabodetabek'

list_city = ['Kota Bekasi', 'Kabupaten Bekasi', 'Kab. Bekasi', 'Kota Bogor', 'Kabupaten Bogor', 'Kab. Bogor', 'Kota Depok', 'Kota Tangerang',
             'Kota Tangerang Selatan','Kabupaten Tangerang', 'Kab. Tangerang']

index = data_all[data_all['City'].isin(list_city)].index.to_list()
data_all['Region Group'][index] = 'Jabodetabek'

data_all['Region Group'] = data_all['Region Group'].fillna(data_all['Region'])

In [29]:
#APPEND TOKOPEDIA SETTLEMENT (ATAS PERINTAH KO TIMO) - SETIAP BULAN 1X
#First Step : Extract data settlement
s_tok = pd.read_excel(r"C:\Users\steven.nathanael\Downloads\Runningan Tokopedia Nov 22.xlsx",sheet_name = 'Sheet1',converters = {'Phone' : str, 'Order #' : str})
s_tok['Customer Phone']=s_tok['Customer Phone'].str.extract('(\d+)')
s_tok['Real SKU'] = s_tok['Real SKU'].astype(str)
s_tok['Order #'] = s_tok['Invoice']
print('Data Settlement Finish')

all_df = pd.merge(s_tok[['Order #','Month','Order Status']], data_all['Order #'], on=['Order #'], how='left', indicator='exists')
all_df[all_df['exists'] == 'left_only'].sort_values('Month')
mis_inv = all_df[all_df['exists'] == 'left_only']['Order #'].unique()
mis_inv
print('Listing missing invoice')

mis_tok = s_tok[s_tok['Order #'].isin(mis_inv)]
mis_tok = mis_tok[mis_tok['Order Status'].str.contains('Dikirim|Selesai|Tiba', case = False, na = False)]
mis_tok['day'] = mis_tok['Date'].astype(int)
mis_tok['month'] = mis_tok['Month'].apply(lambda x : 9 if x == 'September' else
                                                 (10 if x == 'October' else
                                                 (11 if x == 'Novemer' else 
                                                 (12 if x == 'December' else
                                                 (1 if x == 'January' else
                                                 (2 if x == 'February' else
                                                 (3 if x == 'March' else
                                                 (4 if x == 'April' else
                                                 (5 if x == 'May' else
                                                 (6 if x == 'June' else
                                                 (7 if x == 'July' else 8)))))))))))
mis_tok['True datetime'] = pd.to_datetime(mis_tok[['Year','month','day']])
mis_tok['Phone'] = mis_tok['Customer Phone'].astype(str)
mis_tok['Store Type'] = 'Marketplace'
mis_tok['Store'] = 'Tokopedia'
print('Repairing Table')

kolom = data_all.columns

data_all1 = data_all.append(mis_tok, ignore_index = False)
data_all = data_all1[kolom]
del data_all1
print('Finish Appending TokPed')

Data Settlement Finish
Listing missing invoice
Repairing Table
Finish Appending TokPed


In [101]:
data_all1 = data_all.copy()

In [106]:
data_all = data_all1.copy()

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_36404\137081300.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_all = data_all.append(tes1, ignore_index = True)


In [14]:
print("LET'S TRY FOR EMAIL")

LET'S TRY FOR EMAIL


In [2]:
print("Export Master Data")

#TEMPORARY ORAMI
#data_all = data_all[~(data_all['Store'].isin(['Orami Shopping']))]

data_all['True datetime'] = pd.to_datetime(data_all['True datetime'])
data_all['Price List NFI'] = pd.to_numeric(data_all['Price List NFI'])
data_all['PL Before PPN'] = data_all['Price List NFI']/1.1
data_all.loc[data_all['True datetime']>='2022-04-01','PL Before PPN']= data_all['Price List NFI']/1.11
data_all['Total Net Before PPN'] = data_all['PL Before PPN'] * data_all['Qty. Invoiced']

index = data_all[
    (pd.to_datetime(data_all['True datetime']) > '2020-09-11') &
    (data_all['Real SKU'].astype(str) == '1101909453') 
].index.to_list()

data_all['Real SKU'][index] = '1101909331'

index = data_all[
    (pd.to_datetime(data_all['True datetime']) > '2020-09-11') &
    (data_all['Real SKU'].astype(str) == '1101569453') 
].index.to_list()

data_all['Real SKU'][index] = '1101569326'

index = data_all[
    (pd.to_datetime(data_all['True datetime']) > '2020-09-11') &
    (data_all['Real SKU'].astype(str) == '1101907453') 
].index.to_list()

data_all['Real SKU'][index] = '1101907331'

index = data_all[data_all['Real SKU'] == '(B) 2101656'].index.to_list()
data_all['Real SKU'][index] = '(B)2101656'

index = data_all[data_all['Real SKU'] == '(B)1101989453'].index.to_list()
data_all['Brand'][index] = 'Bonus Produk'

data_all.loc[data_all['SKU'].isin(['(U)2104115163', '(U)2101809250', '(U)2309005305',
                                   '(U)2101845443',"(U)2101947250", '(B)71210295',
                                   '(B)2106323165']),'Brand']='Bonus Produk'


if 'Order Online' not in data_now:
    data_now = data_now + ' With Order Online'
SKU = pd.read_excel(r'SKU_File/data_SKU.xlsx')
SKU_baru = pd.read_excel(r"SKU_File/Subbrand Baru.xlsx")

SKU_final = SKU.copy()
SKU_baru['Item Group Code'] = SKU_baru['Item Group Code'].astype(str).str.replace('.0','', regex = False)
SKU_final['SKU Clean'] = SKU_final['SKU'].astype(str).str.replace('(S)', '', regex = False).str.replace('(R)', '', regex = False).str.replace('(B)', '', regex = False).str.replace('(50%)', '', regex = False).str.replace('(E)', '', regex = False)
SKU_final = SKU_final.merge(SKU_baru.drop_duplicates('Item Group Code'), how = 'left', left_on = 'SKU Clean', right_on = 'Item Group Code')

no_pair = SKU[SKU['SKU'].astype(str).isin(SKU_final[SKU_final['Category Baru'].isnull()]['SKU'].astype(str))]
SKU_final = SKU_final[~SKU_final['SKU'].astype(str).isin(no_pair['SKU'].astype(str))]

no_pair['SKU Clean'] = no_pair['SKU'].astype(str).str.replace('(S)', '', regex = False).str.replace('(R)', '', regex = False).str.replace('(B)', '', regex = False).str.replace('(50%)', '', regex = False).str.replace('(E)', '', regex = False).str.replace('E', '', regex = False)
SKU_baru['Item Group Code'] = SKU_baru['Product Code'].astype(str).str.replace('.0','', regex = False)

index = no_pair[no_pair['SKU'].astype(str) == '1101984451'].index[0]
no_pair['SKU Clean'][index] = '1101984453'

index = no_pair[no_pair['SKU'].astype(str) == '2104393210'].index[0]
no_pair['SKU Clean'][index] = '2104392210'

no_pair = no_pair.merge(SKU_baru.drop_duplicates('Product Code'), how = 'left', left_on = 'SKU Clean', right_on = 'Product Code')

SKU_final = SKU_final.append(no_pair, ignore_index = True, sort = False)

#ERROR EXPORTED PARENT ITEM - INPUT HERE
data_all.loc[data_all['Exported Parent Item'].str.contains('Straw Milkshake', na = False, case = False),'Exported Parent Item']='HILO TEEN STRAWBERRY MILKSHAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101569360'].index[0]
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS JERUK PERAS REF 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101569360'].index[0]
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS JERUK PERAS REF 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101686036','(E)1101686036'])].index[0]
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = "W'DANK LKLT KOPI KAWISTA PLS 12RX10SX15G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(["(E)2104407",'2104407'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS EXTRA VIRGIN OLIVE OIL 12BTLX500ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2305551106'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX33.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2305551106'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX33.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2305551106'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX33.5G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2LH1204172','(B)(R)2LH1204172'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PROTEIN BAR STRONGBERY 6SBX12SX20G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2304034180'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN GM MANGGA 6DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101976454'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MARKISA PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101927454'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS NANAS PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101979454'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MADU KURMA PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101588453', '1101588453', '(B)(R)1101588453','(E)1101588453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MILKY ORANGE PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101588016'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MILKY ORANGE FC 72PX10SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102070454'].index[0]
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS LYCHEE TEA PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1N03301456'].index[0]
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS LESS SUGAR BELIMBING PLS 4PX40SX6G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101987454'].index[0]
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MANGGA GANDARIA PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101452195'].index[0]
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101500195'].index[0]
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD PLAIN 6DX1000G'

index = SKU_final[SKU_final['SKU'].isin(['2101524180','(E)2101524180'])].index[0]
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD BISCUIT CEREAL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101524180'].index[0]
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD BISCUIT CEREAL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101453195'].index[0]
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101651195'].index[0]
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2102546125'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT JAHE 24Dx50SX2.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(B)2102546125'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT JAHE 24Dx50SX2.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2102546125'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT JAHE 24Dx50SX2.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2304034180'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN GM MANGGA 6DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101989453', '1101989453', '(B)(R)1101989453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ES CINCAU PLS 4PX40SX13G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101954453', '1101954453', '(B)(R)1101954453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS SEMANGKA PLS 4PX40SX13G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101954454'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS SEMANGKA PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101927453', '1101927453', '(B)(R)1101927453','(E)1101927453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS NANAS PLS 4PX40SX13G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101538453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ANGGUR HIJAU PLS 4PX40SX11G '

index = SKU_final[SKU_final['SKU'].astype(str) == '2104170104'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS WHITE COFFEE 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101656318', '(B)1101656318', '(B)(R)1101656318','(E)1101656318'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI BERONDONG PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101930453', '(B)(R)1101930453', '1101930453',"(E)1101930453"])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS COCOPANDAN PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['71210158', '(J)71110121'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET NUTRISARI 25 RASA'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(J)71110126'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET NUTRISARI 25 RASA FRESHSTART'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1102574110'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NS MIX PAKET BALI 12Dx10S'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['71210165'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET KEHANGATAN WDANK'

index = SKU_final[SKU_final['SKU'].isin(['(E)2104170164','2104170164'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS WHITE COFFEE 12DX4SX20G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2309005300', '2309005305','(E)2309005300', '(E)2309005305'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PROTEIN CRUNCH BBQ BEEF 20BX20G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)1101688317','1101688317'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'NS W’DANK EMPON-EMPON PLS 12Rx10Sx12G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2306551173'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN BAR CRUNCHY CHOCOLATE 6SBX12SX22G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2306551173'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN BAR CRUNCHY CHOCOLATE 6SBX12SX22G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101813443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES TELER PLS 8RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HH0813443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HI LO DRINK TEH TARIK PLS 8RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HH0817441'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HI LO DRINK CHOCO MALT 8RX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101813443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES TELER PLS 8RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101813036'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES TELER FC 36PX10SX15G'

index = SKU_final[SKU_final['SKU'].isin(['2HH0816450'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HI LO DRINK CHOCO MINT PLS 15RX10SX14G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2101864443','2101864443'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES PISANG IJO PLS 8RX10SX15G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2101845360','2101845360'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES KETAN HITAM REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101845443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES KETAN HITAM PLS 8RX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101845443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES KETAN HITAM PLS 8RX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '71210166'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET LOKALATE UNTUK SOBATKU'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2304008180','(E)2304008180'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN GAINMASS TARO 6DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2305551161'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX38.5G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2305507288',"(E)2305507288"])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM KACANG HIJAU 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str) == '71210163'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET HAMPERS IMLEK NUTRISARI'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104508105'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS KOREAN GARLIC BUTTER COOKIES 12DX5SX20G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2104148164','2104148164'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS MINT COCOA 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104148024'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS MINT COCOA 12DX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2T00803164'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS ORANGE COCOA 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2T00804164'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS SALTED CARAMEL COCOA 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104214210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAMBAL TERASI 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104214210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAMBAL TERASI 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2104115163','2104115163'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS AVOCADO COFFEE 12DX4SX20G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101609180','(E)2101609180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN TARO 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101445144','(E)2101445144'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE KETAN HITAM 12DX175G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2101485195','2101485195'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE VANILLA 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2102526125'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT LEMONGRASS PANDAN 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104394210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAUS TIRAM 24BTLX200ML'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2305545288',"(E)2305545288"])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM KETAN HITAM 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106023014', '(B)2106023014',"(E)2106023014"])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SHIRATAKI NOODLES 40PX71G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2T03201001','(E)2T03201001'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS BUMBU KALDU AYAM JAMUR 24PX100G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106323165','(E)2106323165'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS BUMBU SAUS TELUR ASIN 24Dx3Sx22G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110113'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET HILO'
SKU_final['Unnamed: 3'][index] = 'PAKET TAKJIL HILO DESSERT'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110110'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'HAMPER IDUL FITRI TS'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110129'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'PAKET TS BEAT HYPERTENSION 2022'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110114'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET RAMADHAN NUTRISARI 2021'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101413144'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE ES TELER 12DX175G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101413144'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE ES TELER 12DX175G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110111'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET KEHANGATAN WDANK - RAMADHAN EDITION'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110115'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'GETPLUS HAMPERS IDUL FITRI TS'

index = SKU_final[SKU_final['SKU'].astype(str) == '2102584125'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT ROSE VANILLA 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101642180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN POPCORN CARAMEL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101642180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN POPCORN CARAMEL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101978454'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MADU LEMON PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102611454'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS LEMON TEA PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101543453'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS APEL JERUK PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(B) 2101656'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN RTD COFFEE TIRAMISU 24PX200ML'

index = SKU_final[SKU_final['SKU'].isin(['2HH1409249','(B)2HH1409249'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HI LO DRINK RTD CHOCOFIT 24TPKX190ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HF1401250'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL RTD VEGIBERI 24PX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101816250'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO RTD VANILA COOKIES 24PX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101816250'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO RTD VANILA COOKIES 24PX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2TA0102180'].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS GOLDENMIL VANILLA 12DX500G'

index = SKU_final[SKU_final['SKU'].isin(['2106386249','(E)2106386249'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS RTD OAT DRINK VANILLICIOUS 24PX190ML'

index = SKU_final[SKU_final['SKU'].isin(['(E)2T01402249','2T01402249'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS RTD ALMOND DRINK CHOCOLICIOUS 24TPKX190ML '

index = SKU_final[SKU_final['SKU'].astype(str) == '2T01403249'].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS RTD OAT DRINK MELON 24TPKX190ML'

index = SKU_final[SKU_final['SKU'].isin(['(E)2105028180', '2105028180'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS LOW FAT MILK KOREAN STRAWBERRY 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104241210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS MAYONNAISE ROASTED SESAME 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104241210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS MAYONNAISE ROASTED SESAME 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101996453',"(E)1101996453"])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ES RUJAK JERUK BALI PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101982454'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MADU JERUK PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1N00838454'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ISOTONIK REFRESHING CITRUS PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2300542155'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN DAILY POPCORN CARAMEL 12DX250G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2LM0801029'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN ISOPOWER STARGIZING 6DX30SX7.8G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1102110453','(E)1102110453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS NUTRI C1000 JERUK PLS 4PX40SX6G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110117'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRILOGI SERI MBA NANA SI PENUH PESONA'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110116'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRILOGI SERI MAS KAKA YANG BANYAK AKAL'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110119'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRILOGI SERI JEJE SI JELI'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102560453'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRISARI 4 RASA SUMMER PACKAGE 40 SACHET'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102561453'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRISARI 4 RASA LOCAL PACKAGE 40 SACHET'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2104543105','(E)2104543105'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS KOREAN GOGUMA COOKIES 12DX5SX20G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101651155'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN CHOCOLATE 12DX250G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110118'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET ORANGE ADVENTURE'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101578180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD SWEET POTATO 12DX500G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2101551195','2101551195'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2101700166','2101700166'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO PLATINUM ORIGINAL 12DX12SX30G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101700037'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HI LO PLATINUM ORIGINAL 12DX5SX30G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HG0103050'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HI LO PLATINUM +HMB Vanilla 12DX8SX42G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101710110'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO JOINT PLUS 12DX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101710110'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO JOINT PLUS 12DX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101693318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI TAPE KETAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101693318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI TAPE KETAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101694318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ANDALIMAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101653318','(E)1101653318'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = "NS W'DANK SARABBA EXTRA PLS 12RX10SX15G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101618318','(E)1101618318'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = "NS W'DANK MADU TEMULAWAK PLS 12RX10SX11G"

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110120'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET LOKALATE #RASALOKAL'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101444180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL STRAW CHEESECAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101486195','(E)2101486195'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL VANILLA 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101487148', '(B)2101487148','(E)2101487148'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ALMOND MILK COCONUT 12DX200G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2101469180','2101469180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO MULTIGRAIN ORIGINAL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101683180', '(B)2101683180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN STRAWBERRY MILKSHAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2304097159','2304097051'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLANTPROTEIN OGURA 6Dx216G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2304097159'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLANTPROTEIN OGURA 12DX216G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106395308','(E)2106395308'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SHIRATAKI RICE RASA NASI UDUK 24PX72G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2T02603203'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS BERAS MERAH ORGANIK 12PCHX1000G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1102070350','(E)1102070350'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS TEA LYCHEE TEA REF 12BAGX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1N00804229'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS JERUK MANIS PITCHER 12BTLX300G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1100534034'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS PREMIUM JUS MANGGA REF 12BAGX420G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101505453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS JERUK NIPIS JAHE PLS 4PX40SX11G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)1102574110'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS MIX PAKET BALI 12Dx10S"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)1101505453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS JERUK NIPIS JAHE PLS 4PX40SX11G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1N01402250','(E)1N01402250'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS RTD'
SKU_final['Unnamed: 3'][index] = "NS RTD SQUEEZED ORANGE 24TPKX200ML"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101595453','(E)1101595453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS ES KUWUD NIPIS PLS 4PX40SX11G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)1101591453','1101591453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS GULA ASEM PLS 4PX40SX11G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2104589105','(B)2104589105', '(E)2104589105'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = "TS COOKIES KLEPON 12DX5SX20G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2304515112'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = "L-MEN LOSE WEIGHT AVOCADO COFFEE 6DX12SX25G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2LD0905181','(E)2LD0905181'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = "L-MEN GAINMASS KLEPON LATTE 6DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2LC0903181'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = "L-MEN ADVANCE GOLD VANILLA 6DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101453607','(E)2101453607'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = "HILO SCHOOL CHOCOLATE 12DX250G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101459180','(E)2101459180','(B)2101459180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = "HILO SCHOOL BUBBLE GUM 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101478180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = "HILO ACTIVE CARAMEL LATTE 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2101478180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = "HILO ACTIVE CARAMEL LATTE 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110124'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'PARSEL NATAL TAHUN BARU'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101479180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL COTTON CANDY 12DX500G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2HF1403250','2HF1403250'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL RTD COTTON CANDY 24TPKX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101692318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI PISANG BAKAR EPE PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].isin(['(E)1101656360','1101656360'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI BERONDONG 12BAGX500G'

index = SKU_final[SKU_final['SKU'].isin(['(E)1101685360','1101685360'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ALPUKAT 12BAGX500G'

index = SKU_final[SKU_final['SKU'].isin(['1NC1402055'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'NS LOKALATE RTD KOPI BRNDONG 24TPKX185ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101686036'].index[0]
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = "W'DANK LKLT KOPI KAWISTA PLS 12RX10SX15G"

index = SKU_final[SKU_final['SKU'].astype(str) == '2101888360'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO KLEPON LATTE REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2101847360','2101847360']) ].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO THAI TEA REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2101819360', '2101819360'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO TARO LATTE REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101481118'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL CHOCOLATE CANDY 12SBX20SX8G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106317152','(E)2106317152'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS INSTANT CAKE MIX BROWNIES 12Dx230G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2305559288','2305559288'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM BUBBLE GUM 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2305584288','2305584288'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM VANILLA CARAMEL 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2305551028'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE REF 6DX800G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2304576112','(E)2304576112'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN LW MANGO STICKY RICE 6DX12SX25G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2105055180','2105055180'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS LOW FAT MILK MACCHIATO COFFEE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2101618180','2101618180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN BISCUIT CARAMEL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2154084141','(E)2154084141'])].index.to_list()
SKU_final['Category Baru'][index] = 'DIABETAMIL'
SKU_final['Unnamed: 3'][index] = 'DIABETAMIL MILK VANILLA 12Dx150G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2104119110','(E)2104119110'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS SOY LATTE 12Dx10Sx15G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2T02703229','2T02703229'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS PEANUT ALMOND BUTTER 12BTLX300G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2T00806053','(B)2T00806053','(B)(R)2T00806053'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS 7 FRUITS FIBER DAILY 12DX12SX15G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2T01812126','2T01812126'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT GULA AREN 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].isin(['2T01814125'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT GULA BUAH 24DX50SX2.5G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1102611360','(E)1102611360'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS TEA LEMON TEA REF 12BAGX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2HH1407250','(E)2HH1407250'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO DRINK RTD CHOC AVOCADO 24TPKX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101694318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ANDALIMAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101888360'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO KLEPON LATTE REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2102526125'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT LEMONGRASS PANDAN 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2102584125'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT ROSE VANILLA 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101983454'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS JERUK MAROKO PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101543453'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS APEL JERUK PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102611337'].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = 'NS LEMON TEA REF 12BAGX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102611337'].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = 'NS LEMON TEA REF 12BAGX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101907335'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS FLORIDA ORANGE PLS 18PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101907017'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS FLORIDA ORANGE FC 72OX10SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101930454'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS COCOPANDAN PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102070337'].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = 'NS LYCHEE TEA REF 12BAGX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101989454'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ES CINCAU PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101500195'].index[0]
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD PLAIN 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101444180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL STRAW CHEESECAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104394210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAUS TIRAM 24BTLX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101651195'].index[0]
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101694108'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ANDALIMAN 12DX8SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HF0111003'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HI LO SCHOOL STRAWBERRY PLS 12RX10SX27G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101686318','(E)1101686318'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI KAWISTA PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101453195'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].isin(['2HF0112021','(E)2HF0112021'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HI LO SCHOOL 3+ SOYA VANILLA MALT 12DX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2300542155'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN DAILY POPCORN CARAMEL 12DX250G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101578180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD SWEET POTATO 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104508105'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS KOREAN GARLIC BUTTER COOKIES 12DX5SX20G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101656108'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI BERONDONG 12Dx8Sx15G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2104115110','2104115110'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS AVOCADO COFFEE 12DX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2T01705164'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS FRENCH BUTTER SOUFFLE COFFEE 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2T01705024'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS FRENCH BUTTER SOUF COFFEE 12DX10SX15G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2101479180','(B)2101479180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL COTTON CANDY 12DX500G'

index = SKU_final[SKU_final['SKU'].isin(['(E)2HF0113021','2HF0113021'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HI LO SCHOOL 3+ STRAWBERRY POP 12DX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HF0114021'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HI LO SCHOOL ORIGINAL 12DX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104393210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SALTY SOY SAUCE 24BTLX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101452195'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2304515112'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN LOSE WEIGHT AVOCADO COFFEE 6DX12SX25G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2LG0906288'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM STRAWBERRRY 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110130'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'PAKET TROPICANA SLIM CHILL OUT KIT 2022'

index = SKU_final[SKU_final['SKU'].astype(str) == '2102000155'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT CLASSIC 12DX250G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2T01813126'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT LUO HAN GUO 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101683180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN STRAWBERRY MILKSHAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HC0113021'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HI LO TEEN HiProtein Melon 12DX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HC0109021'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HI LO TEEN ORIGINAL 12DX400G'

index = SKU_final[SKU_final['SKU'].isin(['2HC0112021','2HC0110021'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HI LO TEEN Berries Club 12DX400G'

index = SKU_final[SKU_final['SKU'].isin(['2101684195'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HI LO TEEN VANILLA CARAMEL 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110131'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET NUTRISARI 35 RASA'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110133'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET NUTRISARI VALENTINE 2023'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101692318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI PISANG BAKAR EPE PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].isin(['2LL1402249','(E)2LL1402249'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN RTD'
SKU_final['Unnamed: 3'][index] = 'L-MEN PROTEIN 2GO RTD OGURA 24TPKX190ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110143'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET L-MEN'
SKU_final['Unnamed: 3'][index] = 'GREEN PACKAGING L-MEN 2GO CHOCOLATE'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110141'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'GREEN PACKAGING OAT DRINK RTD'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110139'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'GREEN PACKAGING MINYAK KANOLA'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110144'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET HILO'
SKU_final['Unnamed: 3'][index] = 'GREEN PACKAGING HILO SCHOOL CHOCOLATE RTD'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110145'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET HILO'
SKU_final['Unnamed: 3'][index] = 'GREEN PACKAGING HILO TEEN CHOCOLATE RTD'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110142'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'GREEN PACKAGING NS SQUEEZED ORANGE RTD'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110134'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET LOKALATE'
SKU_final['Unnamed: 3'][index] = 'PAKET SOBAT MELEK SEGITIGA LOKALATE 2023'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110155'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'VB SSI - TS ALMOND DRINK CHOCOLICIOUS 4 PCS'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110154'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'VB SSI - TS OAT DRINK VANILICIOUS 4 PCS'

index = SKU_final[SKU_final['SKU'].isin(['(J)71110153','LN00000002'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET NUTRISARI 45 RASA 2023'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2T00805043'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS COLLAGEN DRINK STRAWBERRY 12KLRX200G'

SKU = SKU.merge(SKU_final[['SKU', 'Category Baru', 'Unnamed: 3']].drop_duplicates('SKU'), how = 'left', on = 'SKU')
SKU = SKU.rename(columns = {'Unnamed: 3' : 'Exported Parent Item'})

index = data_all[data_all['Real SKU'] == '2101492P24'].index.to_list()
data_all['Brand'][index] = 'Bundle'
index = data_all[data_all['SKU'].isin(['71210111', '71210112'])].index.to_list()
data_all['Brand'][index] = 'Bundle'
index = data_all[data_all['Real SKU'] == '2306592173'].index.to_list()
data_all['Brand'][index] = 'L-Men'
indeks = data_all[data_all['Brand'] == 'TS'][data_all[data_all['Brand'] == 'TS']['Real SKU'].astype(str) == '2101453180'].index.to_list()
data_all['Real SKU'][indeks] = data_all['SKU'][indeks]
data_all['Parent Item'][indeks] = 'Tropicana Slim Milk Low Fat Vanilla 500gr'

index = data_all[data_all['Brand'].isin(['NS']) & data_all['Sub Brand'].str.contains('WDANK', case = False, na = False)].index.to_list()
data_all['Brand'][index] = 'WDANK'

index = data_all[data_all['Brand'] == "W'dank"].index.to_list()
data_all['Brand'][index] = 'WDANK'

data_all['Real SKU'] = data_all['Real SKU'].astype(str).str.replace('.0', '', regex = False)

no_name = data_all[data_all['Exported Parent Item'].isnull()]
no_name = no_name.merge(SKU[['SKU', 'Category Baru', 'Exported Parent Item']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(no_name.index)

data_all['Category Baru'][no_name.index] = no_name['Category Baru_y']
data_all['Exported Parent Item'][no_name.index] = no_name['Exported Parent Item_y']

data_SKU = SKU.copy()
data_SKU['Category Baru'] = data_SKU['Category Baru'].fillna(data_SKU['Sub Brand'])
data_SKU['Category Baru'] = data_SKU['Category Baru'].fillna(data_SKU['Brand'])
data_SKU['Sub Brand'] = data_SKU['Category Baru']
data_all['Sub Brand'] = data_all['Category Baru']

index = data_all[data_all['Real SKU'].astype(str) == '2102000155'].index.to_list()
data_all['Category Baru'][index] = 'TS KUNING - SWT POWDER'
data_all['Exported Parent Item'][index] = 'TS SWT CLASSIC 12DX250G'

index = data_all[data_all['Real SKU'].astype(str) == '2153000314'].index.to_list()
data_all['Category Baru'][index] = 'TS KUNING - SWT POWDER'

index = data_all[data_all['Real SKU'].astype(str) == '2304097159'].index.to_list()
data_all['Category Baru'][index] = 'L-MEN POWDER'
data_all['Sub Brand'][index] = 'L-MEN POWDER'

index = data_all[data_all['Real SKU'].astype(str) == '2101651155'].index.to_list()
data_all['Category Baru'][index] = 'HILO TEEN'
data_all['Exported Parent Item'][index] = 'HILO TEEN CHOCOLATE 12DX250G'

temp = data_all.copy()
no_name['Real SKU'] = no_name['Real SKU'].astype(str).str.replace('(S)','',regex = False).str.replace('.0','',regex = False)
data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('(S)','',regex = False).str.replace('.0','',regex = False)
data_all['Sub Brand'][no_name.index] = no_name.merge(data_SKU[['SKU', 'Sub Brand']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(no_name.index)['Sub Brand_y']

index = data_all[data_all['Real Nama Produk'].isin(['NS FLORIDA ORANGE FC 72OX10SX11G'])].index.to_list()
data_all['Sub Brand'][index] = 'NS TRADITIONAL'

index = data_all[data_all['Brand'] == "WDANK"].index.to_list()
data_all['Sub Brand'][index] = 'WDANK'

print('Filter Status')
index = data_all[data_all['Customer Name'] == 'Shopee Brand Portal'].index.to_list()
data_all['Order Status'][index] = 'Delivered'
osf=pd.read_excel(r"data_supp\order filter.xlsx")
data_success =data_all[data_all['Order Status'].isin(osf['order filter'].unique()) & (~data_all['Store'].isin(['Orami Shopping']))]

# data_success =data_all[data_all['Order Status'].isin([
#         '"Delivered"', '"Ready to Ship"', '"Open"', '"Printed"',
#        '"Shipped"', '"Delivered, Shipped"', '"Shipped, Ready to Ship"',
#        '"Shipped, Delivered"', '"Ready to Ship, Delivered"',
#        '"Ready to Ship, Shipped"', '"Delivered, Ready to Ship"',
#        '"Delivered, Open"', '"Open, Delivered"', 'Delivered',
#        '"Ready to Ship, Open"', '"Shipped, Open"','complete', 'payment_confirmed',
#        'completed_to_wms', 'processing', 
#        'ready_to_ship', 
#        '"Open, Ready to Ship"', '"Open, Shipped, Ready to Ship"',
#        'Sudah Settlement', '"Delivered, Shipped, Ready to Ship"',
#        '"Ready to Ship, Shipped, Delivered"',
#        '"Ready to Ship, Delivered, Shipped"', '"Open, Shipped"',
#        '"Printed, Open"', '"Shipped, Printed"', '"Delivered, Printed"',
#        '"Open, Printed"', '"Printed, Shipped"', '"Printed, Delivered"',
#        '"Open, Delivered, Printed"', '"Open, Shipped, Printed"',
#        '"Shipped, Open, Printed"', '"Ready to Ship, Printed"',
#        '"Printed, Delivered, Open"', '"Delivered, Printed, Shipped"',
#        '"Printed, Shipped, Open"', '"Delivered, Shipped, Open"',
#        '"Shipped, Open, Ready to Ship"',
#        '"Delivered, Open, Ready to Ship"', 
#        '"Ready to Ship, Open, Shipped"', 
#        'Sudah Isi Pickup Time', '"Shipped, Ready to Ship, Open"',
#        '"Payout"',  '"Printed, Ready to Ship"',
#        'Ready to Ship', 'Open',
#        'Shipped', 'Delivered, Shipped', 
#        'Delivered, Open', 'Printed', 'cod_sent',
#        'entri_verified', 'incoming', 'completed', 'Open, Ready to Ship',
#        'Open, Shipped', 'Ready to Ship, Shipped', 'completed_with_awb','Packed','Shipped','Delivered','Ready to Ship','Completed',
#        '"Return Waiting for Approval"', '"Return"', 'bt_payment_review',
#        'closed', 'Order Baru', 'ready_to_cod', 'Pesanan Dikirim',
#        'Pesanan Dikomplain', 'Pesanan Selesai', 'Pesanan Tiba',
#        'Resi Diubah', 'Pesanan Baru', 'Selesai'
#     ])]


all_single = data_success[data_success['Brand'].isin(['NS', 'TS', 'L-Men', 'HiLo','WDANK'])]
all_single = all_single[all_single['Exported Parent Item'] != "W'DANK LKLT KOPI DURIAN PLS 12RX10SX15G"]

index = all_single[all_single['Brand'] == "WDANK"].index.to_list()
all_single['Brand'][index] = 'NS'

all_single = all_single[~all_single['Store'].isin(['ECOM RETAIL - JD ID','MITRA TOKOPEDIA','ECOM RETAIL - SHOPEE'])]
indeks = all_single[all_single['Store'].isin(['CARI SAYUR',  'EMOS', 'INOVASI DIGITAL NIAGA - KALIDERES, PT',
                                              'RITEL BERSAMA NASIONAL; PT', 'SAYUR BOX', 'TANIHUB'])].index.to_list()
all_single['Exported Parent Item'][indeks] = all_single['Product Name'][indeks]

indeks = all_single[(all_single['Store'].isin(['CARI SAYUR',  'EMOS', 'INOVASI DIGITAL NIAGA - KALIDERES, PT',
       'RITEL BERSAMA NASIONAL; PT', 'SAYUR BOX', 'TANIHUB']))&
                   (all_single['Sub Brand'].isnull())].index.to_list()
all_single['Sub Brand'][indeks] = all_single['Category'][indeks]

all_single['Total Net'] = all_single['Total Net'].astype(float).astype('int64')

bundle_name = data_all[data_all['Real SKU'] == 'PN50N86N87N89G162']['Product Name'].unique()
bundle_name = list(bundle_name) + ['NutriSari Orange Adventure (5 x 10 Sch)','NutriSari Bundle - Orange Adventure']
indeks = all_single[all_single['Bundle Name'].astype(str).isin(bundle_name)].index.to_list()
all_single['Brand'][indeks] = 'NS'
all_single['Sub Brand'][indeks] = 'PAKET NUTRISARI'
all_single['Exported Parent Item'][indeks] = 'PAKET ORANGE ADVENTURE'

all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS SHIRATAKI NOODLES 40Px71G', 'TS SHIRATAKI NOODLES 40PX71G')
all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS KOREAN GARLIC BUTTERCOOKIES12DX5SX20G', 'TS KOREAN GARLIC BUTTER COOKIES 12DX5SX20G')
all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS WHITE COFFEE 12DX4SX20G', 'TS WHITE COFFEE 12DX4SX15G')
all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS AVOCADO COFFEE 12DX4SX20G', 'TS AVOCADO COFFEE 12DX4SX14G')

all_single.loc[all_single['Real SKU'] == '2152051','Exported Parent Item']='TS DM COOKIES CHOCO 12DX10SX20G'#[['Year','Month','Brand','Store','Sub Brand','Real SKU','Real Nama Produk','Bundle Name']]#.drop_duplicates()
all_single.loc[all_single['Real SKU'] == '1101907017','Exported Parent Item']='NS FLORIDA ORANGE FC 72OX10SX11G'#[['Year','Month','Brand','Store','Sub Brand','Real SKU','Real Nama Produk','Bundle Name']]#.drop_duplicates()

all_single.loc[all_single['Real SKU'] == '2152051','Sub Brand']='TS KUNING OTHERS'
all_single=all_single[all_single['Real SKU']!="(U)1102101155"]
all_single=all_single[all_single['Real SKU']!="(U)2307061250"]
all_single=all_single[all_single['Real SKU']!="(B)2T01705164"]
all_single.loc[all_single['Order #']=='1000103864','Region Group']='Kab. Bogor'

zipnotmapped=all_single[(all_single['Region Group'].isnull())&(all_single['Store Type']=='Marketplace')&(all_single['Year']==2022)&(all_single['Zip Code'].notnull())]['Zip Code'].unique()#.to_excel('Gk ke mapping.xlsx',index=False)
for i in zipnotmapped:
    regionvalue=all_single[(all_single['Zip Code']==i)&(all_single['Region Group'].notnull())]['Region Group'].unique()[0]
    all_single.loc[(all_single['Region Group'].isnull())&(all_single['Region Group'].isnull())&(all_single['Store Type']=='Marketplace')&(all_single['Year']==2022),'Region Group']=regionvalue


indeks = all_single[all_single['Exported Parent Item'].isnull()].index.to_list()
indeks = indeks + all_single[all_single['Exported Parent Item'] == 'nan'].index.to_list()

if len(indeks) == 0:
    print("Prepare E-mailing")
    
    ########################################################
    
    table_brand = all_single[all_single['Store']=='Order Online'][['Brand','Sub Brand']].drop_duplicates().reset_index(drop = True)
    all_single['True datetime'] = pd.to_datetime(all_single['True datetime'])
    from datetime import datetime, timedelta
    today = datetime.today()
    yesterday = datetime.today() - timedelta(days=1)
    today = pd.to_datetime(today.strftime('%Y-%m-%d'))
    yesterday = pd.to_datetime(yesterday.strftime('%Y-%m-%d'))
    yesterday_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= yesterday][all_single[all_single['True datetime'] >= yesterday]['True datetime']<today]
    yesterday_sales = yesterday_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    yesterday_sales['Total Net'] = yesterday_sales['Total Net']/1.1
    yesterday_sales = yesterday_sales.rename(columns = {'Total Net' : 'Yesterday Sales'})
    table_brand = table_brand.merge(yesterday_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    if int(today.strftime('%d')) > 1 :
        mtd_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))
    else :
        mtd_date = datetime.today()-timedelta(days=int(yesterday.strftime('%d')))
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))

    mtd_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= mtd_date][all_single[all_single['True datetime'] >= mtd_date]['True datetime']<today]
    mtd_sales = mtd_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    mtd_sales['Total Net'] = mtd_sales['Total Net']/1.1
    mtd_sales = mtd_sales.rename(columns = {'Total Net' : 'Local Sales'})
    table_brand = table_brand.merge(mtd_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    from calendar import monthrange


    if int(today.strftime('%d')) > 1 :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(today.strftime('%d'))-1)
        number_of_days = monthrange(today.year, today.month)[1]
    else :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(yesterday.strftime('%d')))
        number_of_days = monthrange(yesterday.year, yesterday.month)[1]

    table_brand['Projected'] = table_brand['Average This Month'] * number_of_days

    from dateutil import rrule
    from dateutil.relativedelta import relativedelta

    start_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)-relativedelta(months=3)
    start_date = pd.to_datetime(start_date.strftime('%Y-%m-%d'))
    end_date = datetime.today()-timedelta(days=int(today.strftime('%d')))
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        first_date = pd.to_datetime(dt)
        print("First Date " + str(first_date))
        last_date = first_date + relativedelta(months=1)
        print("Last Date " + str(last_date))
        temp_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]
        temp_sales = temp_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
        temp_sales['Total Net'] = temp_sales['Total Net']/1.1
        colname = str(first_date.month_name()) + ' ' + str(first_date.year)
        temp_sales = temp_sales.rename(columns = {'Total Net' : colname})
        table_brand = table_brand.merge(temp_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    cols = list(table_brand)
    cols[len(cols)-1], cols[len(cols)-3] = cols[len(cols)-3], cols[len(cols)-1]
    table_brand = table_brand.loc[:,cols]
    table_brand = table_brand.reset_index(drop = True)
    table_brand = table_brand.drop('Average This Month', axis = 1)

    def highlight_max(x):
        return ['font-weight: bold' if v == x.loc[x.index.max()] else ''
                    for v in x]

    #     table_brand = table_brand[table_brand['Local Sales'].notnull()]
    table_brand['Yesterday Sales'] = table_brand['Yesterday Sales'].fillna(0).astype('int64')
    table_brand['Local Sales'] = table_brand['Local Sales'].fillna(0).astype('int64')
    table_brand = table_brand.sort_values(['Brand','Local Sales'], ascending = [True, False])

    table_ecom = table_brand.copy()

    for i in table_ecom.columns[2:]:
        table_ecom[i] = table_ecom[i].fillna(0).astype('int64')

    table_ecom = table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True)
    # for i in table_ecom.columns[2:]:
    #     table_ecom[i] = table_ecom[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
    #     table_ecom[i][table_ecom.index.max()] = "<b> {} </b>".format(table_ecom[i][table_ecom.index.max()])

    table_ecom = table_ecom.drop('Brand', axis = 1)
    table_ecom['Sub Brand'][table_ecom.index.max()] = '<b>Total E-Commerce (NHD) Sales<b>'
    table_ecom = table_ecom.rename(columns = {'Sub Brand' : ''})

    table_ecom2 = table_ecom.iloc[[-1]].reset_index(drop = True)

    ###############################################################

    table_brand = all_single[all_single['Store']!='Order Online'][['Brand','Sub Brand']].drop_duplicates().reset_index(drop = True)
    all_single['True datetime'] = pd.to_datetime(all_single['True datetime'])

    from datetime import datetime, timedelta

    today = datetime.today()
    yesterday = datetime.today() - timedelta(days=1)

    today = pd.to_datetime(today.strftime('%Y-%m-%d'))
    yesterday = pd.to_datetime(yesterday.strftime('%Y-%m-%d'))

    yesterday_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= yesterday][all_single[all_single['True datetime'] >= yesterday]['True datetime']<today]
    yesterday_sales = yesterday_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    yesterday_sales['Total Net'] = yesterday_sales['Total Net']/1.1
    yesterday_sales = yesterday_sales.rename(columns = {'Total Net' : 'Yesterday Sales'})
    table_brand = table_brand.merge(yesterday_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    if int(today.strftime('%d')) > 1 :
        mtd_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))
    else :
        mtd_date = datetime.today()-timedelta(days=int(yesterday.strftime('%d')))
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))

    mtd_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= mtd_date][all_single[all_single['True datetime'] >= mtd_date]['True datetime']<today]
    mtd_sales = mtd_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    mtd_sales['Total Net'] = mtd_sales['Total Net']/1.1
    mtd_sales = mtd_sales.rename(columns = {'Total Net' : 'Local Sales'})
    table_brand = table_brand.merge(mtd_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    from calendar import monthrange


    if int(today.strftime('%d')) > 1 :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(today.strftime('%d'))-1)
        number_of_days = monthrange(today.year, today.month)[1]
    else :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(yesterday.strftime('%d')))
        number_of_days = monthrange(yesterday.year, yesterday.month)[1]

    table_brand['Projected'] = table_brand['Average This Month'] * number_of_days

    from dateutil import rrule
    from dateutil.relativedelta import relativedelta

    start_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)-relativedelta(months=3)
    start_date = pd.to_datetime(start_date.strftime('%Y-%m-%d'))
    end_date = datetime.today()-timedelta(days=int(today.strftime('%d')))
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        first_date = pd.to_datetime(dt)
        print("First Date " + str(first_date))
        last_date = first_date + relativedelta(months=1)
        print("Last Date " + str(last_date))
        temp_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]
        temp_sales = temp_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
        temp_sales['Total Net'] = temp_sales['Total Net']/1.1
        colname = str(first_date.month_name()) + ' ' + str(first_date.year)
        temp_sales = temp_sales.rename(columns = {'Total Net' : colname})
        table_brand = table_brand.merge(temp_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    cols = list(table_brand)
    cols[len(cols)-1], cols[len(cols)-3] = cols[len(cols)-3], cols[len(cols)-1]
    table_brand = table_brand.loc[:,cols]
    table_brand = table_brand.reset_index(drop = True)
    table_brand = table_brand.drop('Average This Month', axis = 1)

    def highlight_max(x):
        return ['font-weight: bold' if v == x.loc[x.index.max()] else ''
                    for v in x]

    #     table_brand = table_brand[table_brand['Local Sales'].notnull()]
    table_brand['Yesterday Sales'] = table_brand['Yesterday Sales'].fillna(0).astype('int64')
    table_brand['Local Sales'] = table_brand['Local Sales'].fillna(0).astype('int64')
    table_brand = table_brand.sort_values(['Brand','Local Sales'], ascending = [True, False])

    table_ecom = table_brand.copy()

    for i in table_ecom.columns[2:]:
        table_ecom[i] = table_ecom[i].fillna(0).astype('int64')

    table_ecom = table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True)
    # for i in table_ecom.columns[2:]:
    #     table_ecom[i] = table_ecom[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
    #     table_ecom[i][table_ecom.index.max()] = "<b> {} </b>".format(table_ecom[i][table_ecom.index.max()])

    table_ecom = table_ecom.drop('Brand', axis = 1)
    table_ecom['Sub Brand'][table_ecom.index.max()] = '<b>Total E-Commerce (Non NHD) Sales</b>'
    table_ecom = table_ecom.rename(columns = {'Sub Brand' : ''})

    table_ecom1 = table_ecom.iloc[[-1]].reset_index(drop = True)
    # table_ecom=table_ecom1.append(table_ecom2,ignore_index=True,sort=False)
    # table_ecom=table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True,sort=False)

    table_ecom=table_ecom1.append(table_ecom2,ignore_index=True,sort=False)
    table_ecom=table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True)
    table_ecom.loc[(table_ecom[""].isnull()),""]='<b>Total E-Commerce Sales</b>'
    table_ecom

    for i in table_ecom.columns[1:]:
        table_ecom[i] = table_ecom[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_ecom[i][table_ecom.index.max()] = "<b> {} </b>".format(table_ecom[i][table_ecom.index.max()])

    ###############################################################
    from datetime import date
    sales_total=all_single.groupby(['Year','Month','True datetime','Store'],dropna=False)[['Total Net']].sum().reset_index()
    sales_total.loc[sales_total['Store']=='Order Online','Store Group']='<b>Total Ecommerce (NHD) Sales</b>'
    sales_total.loc[sales_total['Store']!='Order Online','Store Group']='<b>Total Ecommerce (non NHD) Sales</b>'

    sales_total['Total Net Before PPN']=sales_total['Total Net']/1.1
    sales_total.loc[sales_total['True datetime']>='2022-04-01','Total Net Before PPN']=sales_total['Total Net']/1.11

    sales_total['Date']=pd.to_datetime(sales_total['True datetime']).dt.date

    col1=sales_total[(sales_total['Date']>=(date.today()-timedelta(1)))&
                        (sales_total['Date']<(date.today()))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Yesterday Sales'}))

    if '<b>Total Ecommerce (NHD) Sales</b>' not in col1['Store Group'].unique():
        col1=col1.append(pd.DataFrame({'Store Group': ["<b>Total Ecommerce (NHD) Sales</b>"], 'Yesterday Sales': [0]}))

    col2=sales_total[(sales_total['Year']==(datetime.now()-timedelta(1)).year)&
                         (sales_total['Month']==(datetime.now()-timedelta(1)).strftime("%B"))&
                         (sales_total['Date']<(date.today()))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Local Sales'}))
    lastmonth1=(date.today().replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth2=(lastmonth1.replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth3=(lastmonth2.replace(day=1)-timedelta(1))
    col3=col2.copy()
    maxday=monthrange((datetime.now()-timedelta(1)).year, (datetime.now()-timedelta(1)).month)[1]
    col3['Projected']=col3['Local Sales']*maxday/((datetime.now()-timedelta(1)).day)

    col4=sales_total[(sales_total['Year']==lastmonth1.year)&
                 (sales_total['Month']==lastmonth1.strftime("%B"))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth1.strftime("%B")} {lastmonth1.year}'}))
    col5=sales_total[(sales_total['Year']==lastmonth2.year)&
                 (sales_total['Month']==lastmonth2.strftime("%B"))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth2.strftime("%B")} {lastmonth2.year}'}))
    col6=sales_total[(sales_total['Year']==lastmonth3.year)&
                 (sales_total['Month']==lastmonth3.strftime("%B"))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth3.strftime("%B")} {lastmonth3.year}'}))
    tblall=col1.merge(col2,how='left').merge(col3,how='left').merge(col4,how='left').merge(col5,how='left').merge(col6,how='left')
    tblall=tblall.fillna(int(0))
    tblall=tblall.sort_values('Store Group',ascending=False).reset_index(drop=True)

    total_row=tblall.sum(numeric_only=True)

    tblall=tblall.append(total_row,ignore_index=True)
    tblall.loc[tblall['Store Group'].isnull(),'Store Group']='<b>Total Ecommerce Sales</b>'
    tblall.rename(columns={'Store Group':""})
    for i in tblall.columns[1:]:
        tblall[i] = tblall[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        tblall[i][tblall.index.max()] = "<b> {} </b>".format(tblall[i][tblall.index.max()])

    tblall = tblall[tblall['Store Group'].isin(['<b>Total Ecommerce Sales</b>'])]   
    
    
    ###############################################################
    sales_region=all_single.groupby(['Year','Month','True datetime','Region Group'],dropna=False)[['Total Net']].sum().reset_index()
    mappingarea=pd.read_excel("data_supp\Database Area untuk Report.xlsx")
    mappingarea=mappingarea.rename(columns=({'Region':'Real Region',"City":'Region Group'}))
    sales_region=sales_region.merge(mappingarea,how='left')
    sales_region.loc[sales_region['Region Group'].isnull(),'Real Region']='Retail Online'
    
    
    sales_region['Total Net Before PPN']=sales_region['Total Net']/1.1
    sales_region.loc[sales_region['True datetime']>='2022-04-01','Total Net Before PPN']=sales_region['Total Net']/1.11
    sales_region['Date']=pd.to_datetime(sales_region['True datetime']).dt.date

    tblreg=sales_region[(sales_region['Date']>=(date.today()-timedelta(1)))&
                        (sales_region['Date']<(date.today()))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Yesterday Sales'}))
    no_ytd_sales=mappingarea[~mappingarea['Real Region'].isin(tblreg['Real Region'].unique())]['Real Region'].unique()
    for i in no_ytd_sales:
        tblreg=tblreg.append(pd.DataFrame({'Real Region': [f"{i}"], 'Yesterday Sales': [0]}))
        tblreg=tblreg.sort_values('Real Region')    
    if 'Retail Online' not in tblreg['Real Region'].unique():
        tblreg=tblreg.append(pd.DataFrame({'Real Region': ["Retail Online"], 'Yesterday Sales': [0]}))

    tblreg2=sales_region[(sales_region['Year']==(datetime.now()-timedelta(1)).year)&
                         (sales_region['Month']==(datetime.now()-timedelta(1)).strftime("%B"))&
                         (sales_region['Date']<(date.today()))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Local Sales'}))
    lastmonth1=(date.today().replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth2=(lastmonth1.replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth3=(lastmonth2.replace(day=1)-timedelta(1))
    tblreg3=tblreg2.copy()
    maxday=monthrange((datetime.now()-timedelta(1)).year, (datetime.now()-timedelta(1)).month)[1]
    tblreg3['Projected']=tblreg3['Local Sales']*maxday/((datetime.now()-timedelta(1)).day)

    tblregshopee=tblreg3[tblreg3["Real Region"]=='Retail Online']
    tblreg3=tblreg3[tblreg3["Real Region"]!='Retail Online']#.copy()

    totprj=tblreg3[tblreg3["Real Region"]!='Retail Online']['Projected'].sum()
    tblreg3['Projected %']=tblreg3['Projected']/totprj
    tblreg3=tblreg3.append(tblregshopee)

    tblreg3=tblreg3[['Real Region','Projected','Projected %']]
    tblreg4=sales_region[(sales_region['Year']==lastmonth1.year)&
                 (sales_region['Month']==lastmonth1.strftime("%B"))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth1.strftime("%B")} {lastmonth1.year}'}))
    tblreg5=sales_region[(sales_region['Year']==lastmonth2.year)&
                 (sales_region['Month']==lastmonth2.strftime("%B"))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth2.strftime("%B")} {lastmonth2.year}'}))
    tblreg6=sales_region[(sales_region['Year']==lastmonth3.year)&
                 (sales_region['Month']==lastmonth3.strftime("%B"))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth3.strftime("%B")} {lastmonth3.year}'}))
    tblreg=tblreg.merge(tblreg2,how='left').merge(tblreg3,how='left').merge(tblreg4,how='left').merge(tblreg5,how='left').merge(tblreg6,how='left')
    tblreg=tblreg.rename(columns={'Real Region':'Region'})

    total_row=tblreg.sum(numeric_only=True)
    sub_total_row=tblreg[tblreg["Region"]!='Retail Online'].sum(numeric_only=True)
    # tblreg.loc[(tblreg["Region"].isnull()),"Region"]='<b>Total Sales</b>'
    res=pd.DataFrame()
    res=res.append(tblreg[tblreg["Region"]!='Retail Online']).append(sub_total_row,ignore_index=True).append(tblreg[tblreg["Region"]=='Retail Online'])
    res.loc[res["Region"].isnull(),'Region']='<b>Sub Total Sales</b>'
    res=res.append(total_row,ignore_index=True)
    res.loc[res["Region"].isnull(),'Region']='<b>Total Sales</b>'

    res.loc[(res['Region']=='Retail Online')&
               (res['Local Sales'].isnull()),'Local Sales']=0
    res.loc[(res['Region']=='Retail Online')&
               (res['Projected'].isnull()),'Projected']=0

    tblreg=res.copy()
    tblreg=tblreg.fillna(0)
    for i in tblreg.columns[1:4]:
        tblreg[i] = tblreg[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        tblreg[i][tblreg.index.max()] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()])

    for i in tblreg.columns[4:5]:
        tblreg[i] = tblreg[i].apply(lambda x: "{:.2f} %".format(x*100))#.str.replace(',','.', regex = False)
        tblreg[i][tblreg.index.max()] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()])


    for i in tblreg.columns[5:]:
        tblreg[i] = tblreg[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        tblreg[i][tblreg.index.max()] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()])

    for i in tblreg.columns[1:]:
        tblreg[i][tblreg.index.max()-2] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()-2])

    tblreg.loc[(tblreg["Projected %"]=="nan %"),"Projected %"]=''


    tblreg

    ###################################################################################

    for i in table_brand.columns[2:]:
        table_brand[i] = table_brand[i].fillna(0).astype('int64')

    table_brand_hilo = table_brand[table_brand['Brand'] == 'HiLo'].reset_index(drop = True)
    table_brand_hilo = table_brand_hilo.append(table_brand_hilo.sum(numeric_only=True), ignore_index=True)
    table_brand_hilo['Sub Brand'][table_brand_hilo.index.max()] = '<b>TOTAL<b>'
    table_brand_hilo['Brand'][table_brand_hilo.index.max()] = ''


    table_brand_ns= table_brand[table_brand['Brand'] == 'NS'].reset_index(drop = True)
    table_brand_ns = table_brand_ns.append(table_brand_ns.sum(numeric_only=True), ignore_index=True)
    table_brand_ns['Sub Brand'][table_brand_ns.index.max()] = '<b>TOTAL<b>'
    table_brand_ns['Brand'][table_brand_ns.index.max()] = ''


    table_brand_lmen = table_brand[table_brand['Brand'] == 'L-Men'].reset_index(drop = True)
    table_brand_lmen = table_brand_lmen.append(table_brand_lmen.sum(numeric_only=True), ignore_index=True)
    table_brand_lmen['Sub Brand'][table_brand_lmen.index.max()] = '<b>TOTAL<b>'
    table_brand_lmen['Brand'][table_brand_lmen.index.max()] = ''


    table_brand_ts = table_brand[table_brand['Brand'] == 'TS'].reset_index(drop = True)
    table_brand_ts = table_brand_ts.append(table_brand_ts.sum(numeric_only=True), ignore_index=True)
    table_brand_ts['Sub Brand'][table_brand_ts.index.max()] = '<b>TOTAL<b>'
    table_brand_ts['Brand'][table_brand_ts.index.max()] = ''


    for i in table_brand_hilo.columns[2:]:
        table_brand_hilo[i] = table_brand_hilo[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_hilo[i][table_brand_hilo.index.max()] = "<b> {} </b>".format(table_brand_hilo[i][table_brand_hilo.index.max()])

    for i in table_brand_lmen.columns[2:]:
        table_brand_lmen[i] = table_brand_lmen[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_lmen[i][table_brand_lmen.index.max()] = "<b> {} </b>".format(table_brand_lmen[i][table_brand_lmen.index.max()])

    for i in table_brand_ns.columns[2:]:
        table_brand_ns[i] = table_brand_ns[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_ns[i][table_brand_ns.index.max()] = "<b> {} </b>".format(table_brand_ns[i][table_brand_ns.index.max()])

    for i in table_brand_ts.columns[2:]:
        table_brand_ts[i] = table_brand_ts[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_ts[i][table_brand_ts.index.max()] = "<b> {} </b>".format(table_brand_ts[i][table_brand_ts.index.max()])

    table_brand = table_brand_hilo.append(pd.Series(), ignore_index = True, sort = False)
    table_brand = table_brand.append(table_brand_lmen, ignore_index = True, sort = False)
    table_brand = table_brand.append(pd.Series(), ignore_index = True, sort = False)
    table_brand = table_brand.append(table_brand_ns, ignore_index = True, sort = False)
    table_brand = table_brand.append(pd.Series(), ignore_index = True, sort = False)
    table_brand = table_brand.append(table_brand_ts, ignore_index = True, sort = False)
    for i in table_brand.columns:
        table_brand[i] = table_brand[i].fillna('')

    table_brand = table_brand[(table_brand == 'Rp 0').sum(1) < 5]
    table_brand = table_brand.reset_index(drop = True)


    table_item = all_single[['Brand', 'Sub Brand', 'Exported Parent Item']].drop_duplicates().reset_index(drop = True)
    all_single['True datetime'] = pd.to_datetime(all_single['True datetime'])
    from datetime import datetime, timedelta

    today = datetime.today()
    yesterday = datetime.today() - timedelta(days=1)

    today = pd.to_datetime(today.strftime('%Y-%m-%d'))
    yesterday = pd.to_datetime(yesterday.strftime('%Y-%m-%d'))

    yesterday_sales = all_single[all_single['True datetime'] >= yesterday][all_single[all_single['True datetime'] >= yesterday]['True datetime']<today]
    yesterday_sales = yesterday_sales.groupby(['Brand', 'Sub Brand', 'Exported Parent Item'])['Total Net'].sum().reset_index()
    yesterday_sales['Total Net'] = yesterday_sales['Total Net']/1.1
    yesterday_sales = yesterday_sales.rename(columns = {'Total Net' : 'Yesterday Sales'})
    table_item = table_item.merge(yesterday_sales, how = 'left', on = ['Brand', 'Sub Brand', 'Exported Parent Item'])

    if int(today.strftime('%d')) > 1 :
        mtd_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))
    else :
        mtd_date = datetime.today()-timedelta(days=int(yesterday.strftime('%d')))
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))

    mtd_sales = all_single[all_single['True datetime'] >= mtd_date][all_single[all_single['True datetime'] >= mtd_date]['True datetime']<today]
    mtd_sales = mtd_sales.groupby(['Brand', 'Sub Brand', 'Exported Parent Item'])['Total Net'].sum().reset_index()
    mtd_sales['Total Net'] = mtd_sales['Total Net']/1.1
    mtd_sales = mtd_sales.rename(columns = {'Total Net' : 'Local Sales'})
    table_item = table_item.merge(mtd_sales, how = 'left', on = ['Brand', 'Sub Brand', 'Exported Parent Item'])

    if int(today.strftime('%d')) > 1 :
        table_item['Average This Month'] = table_item['Local Sales']/(int(today.strftime('%d'))-1)
        number_of_days = monthrange(today.year, today.month)[1]
    else :
        table_item['Average This Month'] = table_item['Local Sales']/(int(yesterday.strftime('%d')))
        number_of_days = monthrange(yesterday.year, yesterday.month)[1]

    table_item['Projected'] = table_item['Average This Month'] * number_of_days

    from dateutil import rrule
    from dateutil.relativedelta import relativedelta

    start_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)-relativedelta(months=3)
    start_date = pd.to_datetime(start_date.strftime('%Y-%m-%d'))
    end_date = datetime.today()-timedelta(days=int(today.strftime('%d')))
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        first_date = pd.to_datetime(dt)
        last_date = first_date + relativedelta(months=1)
        temp_sales = all_single[all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]
        temp_sales = temp_sales.groupby(['Brand', 'Sub Brand', 'Exported Parent Item'])['Total Net'].sum().reset_index()
        temp_sales['Total Net'] = temp_sales['Total Net']/1.11
        colname = str(first_date.month_name()) + ' ' + str(first_date.year)
        temp_sales = temp_sales.rename(columns = {'Total Net' : colname})
        table_item = table_item.merge(temp_sales, how = 'left', on = ['Brand', 'Sub Brand', 'Exported Parent Item'])

    cols = list(table_item)
    cols[len(cols)-1], cols[len(cols)-3] = cols[len(cols)-3], cols[len(cols)-1]
    table_item = table_item.loc[:,cols]

    #     table_item = table_item[table_item['Local Sales'].notnull()]
    table_item['Yesterday Sales'] = pd.to_numeric(table_item['Yesterday Sales'], errors = 'coerce').fillna(0).astype('int64')
    table_item['Local Sales'] = pd.to_numeric(table_item['Local Sales'], errors = 'coerce').fillna(0).astype('int64')
    table_item = table_item.sort_values(['Brand','Local Sales'], ascending = [True, False])

    for i in table_item.columns[3:]:
        table_item[i] = table_item[i].fillna(0).apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)

    table_item = table_item[(table_item == 'Rp 0').sum(1) < 5]
    table_item = table_item.reset_index(drop = True)
    table_item = table_item.drop('Average This Month', axis = 1)

    all_single = all_single.sort_values('True datetime')

    first_product = all_single.groupby('Exported Parent Item')['True datetime'].first().reset_index()

    product_launch = first_product[first_product['True datetime'] > pd.to_datetime(datetime.today() - relativedelta(months=3))]['Exported Parent Item'].unique()
    non_launch =  ["PARSEL NATAL TAHUN BARU",'HAMPER IDUL FITRI TS', 'GETPLUS HAMPERS IDUL FITRI TS', 'PAKET TAKJIL HILO DESSERT',
                   'HILO CHOCOLATE PLS 15RX10SX14G', 'HILO TEEN TARO 12DX500G', 'TS SAMBAL TERASI 24BTLX200G',
                   'TS AVOCADO COFFEE 12DX4SX20G', 'L-MEN PLATINUM KETAN HITAM 6KLRX800G', 'HILO AVOCADO CHOCOLATE PLS 15RX10SX14G', 'TS SHIRATAKI NOODLES 40PX71G',
                   'TS KOREAN GARLIC BUTTERCOOKIES12DX5SX20G','TS AVOCADO COFFEE 12DX4SX14G', 'TS SWT CLASSIC 24DX100G', 'HILO WHITE CHOCOLATE PLS 15RX10SX14G', 
                   'HILO CHOCO HAZELNUT PLS 15RX10SX14G', 'NS ANGGUR PLS 4PX40SX11G',"NS JERUK MANADO PLS 4PX40SX11G",
                   "NS TEA LYCHEE TEA REF 12BAGX500G","NUTRISARI 4 RASA SUMMER PACKAGE 40 SACHET","NUTRISARI 4 RASA LOCAL PACKAGE 40 SACHET",
                   'PAKET NUTRISARI 25 RASA FRESHSTART','PAKET TS BEAT HYPERTENSION 2022','HILO SCHOOL CHOCOLATE CANDY 12SBX20SX8G','HILO ES TELER FC 36PX10SX15G','NS BLEWAH FC 72PX10SX11G','NS JERUK MAROKO 72PX10SX14G',
                   'HILO TEEN STRAWBERRY MILKSHAKE 12DX500G','LOKALATE KOPI KAWISTA PLS 12RX10SX15G','LOKALATE KOPI ANDALIMAN 12DX8SX15G',
                   'LOKALATE KOPI ALPUKAT FC 36PX10SX15G','GREEN PACKAGING HILO TEEN CHOCOLATE RTD','HILO SCHOOL SUSU COKELAT PLS 12RX10SX35G',
                   'GREEN PACKAGING HILO SCHOOL CHOCOLATE RTD','GREEN PACKAGING L-MEN 2GO CHOCOLATE','GREEN PACKAGING NS SQUEEZED ORANGE RTD',
                   'NS ASO PLS 16RX10SX14G','NS MADU LEMON PLS 4PX40SX11G','NS LEMON TEA PLS 4PX40SX11G','GREEN PACKAGING OAT DRINK RTD',
                   'GREEN PACKAGING MINYAK KANOLA','L-MEN PLANTPROTEIN OGURA 6DX216G','NS MANGGA GANDARIA PLS 4PX40SX11G',
                   'NS MADU JERUK PLS 4PX40SX11G','NS LYCHEE TEA PLS 4PX40SX11G', 'NS LEMON TEA REF 12BAGX400G', 'NS LYCHEE TEA REF 12BAGX400G',
                   'NS ES CINCAU PLS 4PX40SX11G', 'NS FLORIDA ORANGE PLS 18PX40SX11G', 'NS SEMANGKA PLS 4PX40SX11G', 
                   'NS FLORIDA ORANGE FC 72OX10SX11G', 'NS MARKISA PLS 4PX40SX11G', 'NS MADU KURMA PLS 4PX40SX11G', 'NS NANAS PLS 4PX40SX11G',
                   'NS JERUK MAROKO PLS 4PX40SX11G', 'TS CORN OIL 16PX1L', 'NS JERUK MANIS REF 12DX250G', 'NS COCOPANDAN PLS 4PX40SX11G']
    product_launch = [x for x in product_launch if x not in non_launch]

    table_launch = table_item[table_item['Exported Parent Item'].isin(product_launch)]
    table_item = table_item[~table_item['Exported Parent Item'].isin(product_launch)]

    table_hilo = table_item[table_item['Brand'] == 'HiLo'].reset_index(drop = True)
    table_ns= table_item[table_item['Brand'] == 'NS'].reset_index(drop = True)
    table_ns = table_ns[table_ns['Exported Parent Item'] != "W'DANK LKLT KOPI DURIAN PLS 12RX10SX15G"]
    table_lmen = table_item[table_item['Brand'] == 'L-Men'].reset_index(drop = True)
    table_ts = table_item[table_item['Brand'] == 'TS'].reset_index(drop = True)

    new_table_launch = pd.DataFrame()
    for i in table_launch['Brand'].unique():
        if i != table_launch['Brand'].unique()[-1]:
            new_table_launch = new_table_launch.append(table_launch[table_launch['Brand'] == i], ignore_index = True, sort = False).append(pd.Series(), ignore_index = True, sort = False)
        else :
            new_table_launch = new_table_launch.append(table_launch[table_launch['Brand'] == i], ignore_index = True, sort = False)

    for i in new_table_launch.columns:
        new_table_launch[i] = new_table_launch[i].fillna('')

#     index = data_all[data_all['Sub Brand'] == 'WDANK'][data_all[data_all['Sub Brand'] == 'WDANK']['Bundle Flag'].isnull()].index.to_list()
#     data_all['Brand'][index] = 'WDANK'


    print("Clean Data Finish")

    print("Clean Data Non Nubi Finish")


    print("Export Data 2020 Finish")


    print("Prepare Emailing")
  
    import win32com.client
    o = win32com.client.Dispatch("Outlook.Application")
    for oacc in o.Session.Accounts:
        if oacc.SmtpAddress == "customer@nutrifood.co.id":
            oacctouse = oacc
            break
    Msg = o.CreateItem(0)
    if oacctouse:
        Msg._oleobj_.Invoke(*(64209, 0, 8, 0, oacctouse))  # Msg.SendUsingAccount = oacctouseif to:

        to = ['steven.nathanael@nutrifood.co.id']

        list_to = ';'.join(to)

        Msg.To = list_to
        Msg.Subject = "Report E-commerce " + \
        str(datetime.today().date().strftime('%B %d, %Y'))


    html = '''
    <html>
    <head>
    <style>

        h2 {
            text-align: center;
            font-family: Helvetica, Arial, sans-serif;
        }
        table { 
            margin-left: auto;
            margin-right: auto;
        }
        table, th, td {
            border: 1px solid black;
            border-collapse: collapse;
        }
        th, td {
            padding: 5px;
            text-align: center;
            font-family: Helvetica, Arial, sans-serif;
            font-size: 90%;
        }
        table tbody tr:hover {
            background-color: #dddddd;
        }
        .wide {
            width: 90%; 
        }
        p

    </style>
    </head>
    <body>
        '''
    html = html + """
        <p> Dear all,</p>
        <p> Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> dengan rincian channel sebagai berikut : </p>
        <p> Channel Marketplace : Blibli, Bukalapak, Elevenia, JD Indonesia, Lazada, Nutrimart, Order Online, Shopee, Tokopedia, Aladin Mall, TikTok </p>
        <p> Channel Retail : Cari Sayur, Emos, Farmaku, Inovasi Digital Niaga, Ritel Bersama Nasional, Sayurbox, Tanihub, Astro, Lifepack, Toko Now   </p>
        <p> Notes: Mohon maaf ada keterlambatan data untuk retail online </p>

    """.format(tanggal = str(datetime.today().date().strftime('%B %d, %Y')))

    html = html + tblall.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')

    html = html + """
        <p> Dan berikut data penjualan E-Com (Before PPN) per Area :</p>
        """

    html = html + tblreg.to_html(classes='wide', escape=False)

    html = html + """
        <p> Dan Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> per Brand :</p>
    """.format(tanggal = str(datetime.today().date().strftime('%B %d, %Y'))) 


    html = html + table_brand.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')



    html = html + """
        <p> Dan berikut data penjualan E-Com (Before PPN) per Item :</p>
        <p> Item Launching : </p>"""

    html = html + new_table_launch.to_html(classes='wide', escape=False)

    html = html + "<p> HiLo : </p>"    

    html = html + table_hilo.to_html(classes='wide', escape=False)

    html = html + "<p> L-Men : </p>"    

    html = html + table_lmen.to_html(classes='wide', escape=False)

    html = html + "<p> NutriSari : </p>"    

    html = html + table_ns.to_html(classes='wide', escape=False)

    html = html + "<p> Tropicana Slim : </p>"    

    html = html + table_ts.to_html(classes='wide', escape=False)

    html = html + """
        <p></p>
        <p> Best regards, </p>
        <p> E-Commerce Nutrifood </p>
    <body>
    
    </html>
    """

    Msg.HTMLBody = html
    Msg.Send()
    print('Email Sent')
else :
    print('Missing Parent Item')

Export Master Data


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all['Real SKU'][index] = '1101909331'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all['Real SKU'][index] = '1101909331'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SKU_final['Category Baru'][index] = 'L-MEN POWDER'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SKU_final['Unnamed: 3'][index] = 'L-MEN GM MANGGA 6DX500G'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SKU_final['Unnamed: 3'][index] = 'TS SWT JAHE 24Dx50SX2.5G'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SKU_final['Unnamed: 3'][index] = 'NS MIX PAKET BALI 12Dx10S'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SKU_final['Category Baru'][index] = 'PAKET WDANK'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

...

First Date 2023-12-01 00:00:00
Last Date 2024-01-01 00:00:00


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:1333: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]


First Date 2024-01-01 00:00:00
Last Date 2024-02-01 00:00:00


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:1333: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:1366: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_ecom['Sub Brand'][table_ecom.index.max()] = '<b>Total E-Commerce (Non NHD) Sales</b>'
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\3388873355.py:1380: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

Clean Data Finish
Clean Data Non Nubi Finish
Export Data 2020 Finish
Prepare Emailing
Email Sent


In [3]:
for i in all_single.columns:
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4

In [30]:
#CEK ORDER STATUS UPDATE
osf=pd.read_excel(r"data_supp\order filter.xlsx")
data_all[(~data_all['Order Status'].isin(osf['order filter'].unique()))&data_all['Month'].isin(['December','November','January'])][['Store','Channel','Order Status']].drop_duplicates()

,Store,Channel,Order Status
29,Blibli,L-Men Store Blibli,Order Batal
169,Nutrimart,Nutrimart,pending
208,Nutrimart,Nutrimart,canceled
435,Shopee,Shopee,Cancelled
583,Lazada,FBL,Cancelled
666,Tokopedia,Tokopedia,Cancelled
738,Lazada,Lazada,Cancelled
742,TikTok,TikTok,Cancelled
1181,Blibli,Blibli,Cancelled
1908,JD Indonesia,JD Indonesia,Cancelled


In [61]:
## ini cek kalau pembagian region sama total (tbl 1 & 2 Beda) update di file area utk report
sales_region[(sales_region['Region Group'].notnull())&(~sales_region['Region Group'].isin(mappingarea['Region Group'].unique()))][['Region Group']].values

array([], shape=(0, 1), dtype=object)

In [17]:
# if error missing parent item
# listing di kode atas, exported parent item liat tatanama akun admin login user&pass: nutrifood
all_single[all_single['Exported Parent Item'] == 'nan'][['Real SKU','Real Nama Produk','Category Baru','Exported Parent Item','Store','Year','Month']].drop_duplicates()#.values
#all_single[all_single['Exported Parent Item'] == 'nan']['Real Nama Produk'].unique()#.values


,Real SKU,Real Nama Produk,Category Baru,Exported Parent Item,Store,Year,Month
2303135,(B)(R)2LH1204172,Sampling - L-Men Bar Strongberry,NaN,nan,Lazada,2024.0,January
2303969,(B)(R)2T00806053,Sampling - Tropicana Slim 7 Fruits Fiber Daily,NaN,nan,Lazada,2024.0,January
2303970,(B)2HH1409249,PBI - HiLo Ready to Drink Chocofit 190ml,NaN,nan,Lazada,2024.0,January


In [29]:
data_nubi[data_nubi['Year'].isin([2024])]

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Invoice Create Date,Fee Amount,Invoice Amount,Payment Received ID,Payment Received Ref. number,Category Baru,Exported Parent Item,Unnamed: 0,Price List NFI_x,Customer Type


In [31]:
for i in data_all.columns:
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4

In [3]:
### run 12
### take out item yang gk masuk email automation
buang = ['PAKET HILO']
table_brand = table_brand[~table_brand['Sub Brand'].isin(buang)]
table_hilo = table_hilo[~table_hilo['Sub Brand'].isin(buang)]

buang = ['PAKET TS']
table_brand = table_brand[~table_brand['Sub Brand'].isin(buang)]
table_ts = table_ts[~table_ts['Sub Brand'].isin(buang)]

buang = ['PAKET L-MEN']
table_brand = table_brand[~table_brand['Sub Brand'].isin(buang)]
table_ts = table_ts[~table_ts['Sub Brand'].isin(buang)]

buang = ['PAKET NUTRISARI']
table_brand = table_brand[~table_brand['Sub Brand'].isin(buang)]
table_ns = table_ns[~table_ns['Sub Brand'].isin(buang)]

buang = ['PAKET LOKALATE']
table_brand = table_brand[~table_brand['Sub Brand'].isin(buang)]
table_ns = table_ns[~table_ns['Sub Brand'].isin(buang)]

In [100]:
data_all['Brand'].unique()

array(['Bundle', 'L-Men', 'Gimmick', 'Bonus Produk', 'TS', 'HiLo', 'NS',
       'WRP', 'WDANK', 'L-MEN'], dtype=object)

In [4]:
### Samain Nama Produk
data_all.loc[data_all['Real Nama Produk'].isin(['Ex Shopee - HiLo Teen Chocolate 500gr']), 'Real Nama Produk'] = 'HiLo Teen Chocolate 500gr'
data_all.loc[data_all['Real Nama Produk'].isin(['Ex Shopee - HiLo Active Chocolate 750gr']), 'Real Nama Produk'] = 'HiLo Active Chocolate 750gr'
data_all = data_all[~data_all['Brand'].isin(['nan'])]

In [ ]:
data_all[data_all['True datetime'] > '2023-08-03'

In [5]:
### run 13
### send email automation final ke user

import win32com.client

o = win32com.client.Dispatch("Outlook.Application")
for oacc in o.Session.Accounts:
    if oacc.SmtpAddress == "customer@nutrifood.co.id":
        oacctouse = oacc
        break

Msg = o.CreateItem(0)
if oacctouse:
    Msg._oleobj_.Invoke(*(64209, 0, 8, 0, oacctouse))  # Msg.SendUsingAccount = oacctouseif to:
    
    to=["mardi@nutrifood.co.id", "Elvina@nutrifood.co.id", "susana@nutrifood.co.id",
        "ignasius.dwi@nutrifood.co.id", "may@nutrifood.co.id", "Meirza@nutrifood.co.id", "christian.jesaya@nutrifood.co.id",
        "didit@nutrifood.co.id", "noviana@nutrifood.co.id","reynald.tjandra@nutrifood.co.id", "elvira@nutrifood.co.id",
        "manthovani.annice@nutrifood.co.id","joshua.suhendro@nutrifood.co.id","stephanie.wonoadi@nutrifood.co.id",
        "lisa.arianti@nutrifood.co.id",'charissa.lungkat@nutrifood.co.id','rosariani@nutrifood.co.id',"zakaria@nutrifood.co.id","geovano.satria@nutrifood.co.id","agus_s@nutrifood.co.id","romi.anggara@nutrifood.co.id",
        "edward@nutrifood.co.id", "elisa.putri@nutrifood.co.id","teddy.andreas@nutrifood.co.id", "muroby.rocky@nutrifood.co.id",
        "evelyn@nutrifood.co.id","sheila.odilia@nutrifood.co.id","timotius.giovandi@nutrifood.co.id",
        'ronaldo.yolanda@nutrifood.co.id','bagus.sindhu@nutrifood.co.id',"hendi.herdiansyah@nutrifood.co.id",'christovertand.wim@nutrifood.co.id',
        'steven.nathanael@nutrifood.co.id','arbie.hasyim@nutrifood.co.id','ariel.christianto@nutrifood.co.id','raedi.noor@nutrifood.co.id','ignatia.benna@nutrifood.co.id', 'james.michael@nutrifood.co.id']
    #to = ['steven.nathanael@nutrifood.co.id']

    list_to = ';'.join(to)

    Msg.To = list_to
    Msg.Subject = "Report E-commerce " + \
    str(datetime.today().date().strftime('%B %d, %Y'))
    




html = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
    }
    table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
    }
    th, td {
        padding: 5px;
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }
    p

</style>
</head>
<body>
    '''
html = html + """
    <p> Dear all,</p>
    <p> Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> dengan rincian channel sebagai berikut : </p>
    <p> Channel Marketplace : Blibli, Bukalapak, Lazada, Nutrimart, Order Online, Shopee, Tokopedia, Aladin Mall, TikTok </p>
    <p> Channel Retail : Cari Sayur, Emos, Farmaku, Inovasi Digital Niaga, Ritel Bersama Nasional, Sayurbox, Tanihub, Astro, Lifepack, Toko Now, Freshbox  </p>
""".format(tanggal = str(datetime.today().date().strftime('%B %d, %Y')))

html = html + tblall.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')

html = html + """
    <p> Dan berikut data penjualan E-Com (Before PPN) per Area :</p>
    """

html = html + tblreg.to_html(classes='wide', escape=False)

html = html + """
    <p> Dan Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> per Brand :</p>
""".format(tanggal = str(datetime.today().date().strftime('%B %d, %Y'))) 


html = html + table_brand.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')



html = html + """
    <p> Dan berikut data penjualan E-Com (Before PPN) per Item :</p>
    <p> Item Launching : </p>"""

html = html + new_table_launch.to_html(classes='wide', escape=False)

html = html + "<p> HiLo : </p>"    

html = html + table_hilo.to_html(classes='wide', escape=False)

html = html + "<p> L-Men : </p>"    

html = html + table_lmen.to_html(classes='wide', escape=False)

html = html + "<p> NutriSari : </p>"    

html = html + table_ns.to_html(classes='wide', escape=False)

html = html + "<p> Tropicana Slim : </p>"    

html = html + table_ts.to_html(classes='wide', escape=False)

html = html + """
    <p></p>
    <p> Best regards, </p>
    <p> E-Commerce Nutrifood </p>
<body>
</html>
"""

Msg.HTMLBody = html
Msg.Send()

In [43]:
for i in data_all.columns:
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4

In [50]:
s_sp = pd.read_excel(r"D:\Masterdata\Settlement\Runningan Shopee Jan 2021 - Nov 22.xlsx",sheet_name = 'Runningan Shopee',converters = {'Phone' : str, 'Order #' : str})
s_sp['Customer Phone']=s_sp['No. Telepon'].str.extract('(\d+)')
s_sp['Real SKU'] = s_sp['Real SKU'].astype(str)
s_sp['Order #'] = s_sp['No. Pesanan']
s_sp['Order Status'] = s_sp['Status Pesanan']
print('Data Settlement Finish')

Data Settlement Finish


In [60]:
#s_sp['Order Status'] = s_sp['Status Pesanan']
s_sp['Order Status'].unique()

array(['Cancel', 'Selesai', 'Batal', 'Sedang Dikirim', 'Perlu Dikirim',
       'Belum Bayar'], dtype=object)

In [57]:
all_df = pd.merge(s_sp[['Order #','Month','Order Status']], data_all['Order #'], on=['Order #'], how='left', indicator='exists')
#all_df[all_df['exists'] == 'left_only'].sort_values('Month')
#all_df['exists'].unique()
mis_inv = all_df[all_df['exists'] == 'left_only']['Order #'].unique()
mis_inv


array(['220124MUQE0B2C', '220124N565JVHM', '220124N6JXYQN6', ...,
       '220209210HYCVE', '220521REYRSDSW', '220509NWCCXX7P'], dtype=object)

In [59]:
mis_sp = s_sp[s_tok['Order #'].isin(mis_inv)]
mis_tok = mis_tok[mis_sp['Order Status'].str.contains('Dikirim|Selesai|Tiba', case = False, na = False)]
mis_tok['day'] = mis_tok['Date'].astype(int)
mis_tok['month'] = mis_tok['Month'].apply(lambda x : 9 if x == 'September' else
                                                 (10 if x == 'October' else
                                                 (11 if x == 'Novebmer' else 
                                                 (12 if x == 'December' else
                                                 (1 if x == 'January' else
                                                 (2 if x == 'February' else
                                                 (3 if x == 'March' else
                                                 (4 if x == 'April' else
                                                 (5 if x == 'May' else
                                                 (6 if x == 'June' else
                                                 (7 if x == 'July' else 8)))))))))))
mis_tok['True datetime'] = pd.to_datetime(mis_tok[['Year','month','day']])
mis_tok['Phone'] = mis_tok['Customer Phone'].astype(str)
mis_tok['Store Type'] = 'Marketplace'
mis_tok['Store'] = 'Tokopedia'
print('Repairing Table')

['left_only', 'both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [ ]:
mis_tok = s_tok[s_tok['Order #'].isin(mis_inv)]
mis_tok = mis_tok[mis_tok['Order Status'].str.contains('Dikirim|Selesai|Tiba', case = False, na = False)]
mis_tok['day'] = mis_tok['Date'].astype(int)
mis_tok['month'] = mis_tok['Month'].apply(lambda x : 9 if x == 'September' else
                                                 (10 if x == 'October' else
                                                 (11 if x == 'Novemer' else 
                                                 (12 if x == 'December' else
                                                 (1 if x == 'January' else
                                                 (2 if x == 'February' else
                                                 (3 if x == 'March' else
                                                 (4 if x == 'April' else
                                                 (5 if x == 'May' else
                                                 (6 if x == 'June' else
                                                 (7 if x == 'July' else 8)))))))))))
mis_tok['True datetime'] = pd.to_datetime(mis_tok[['Year','month','day']])
mis_tok['Phone'] = mis_tok['Customer Phone'].astype(str)
mis_tok['Store Type'] = 'Marketplace'
mis_tok['Store'] = 'Tokopedia'
print('Repairing Table')

kolom = data_all.columns

data_all1 = data_all.append(mis_tok)
data_all = data_all1[kolom]
print('Finish Appending TokPed')

In [17]:
%whos

Variable                 Type              Data/Info
----------------------------------------------------
ActionChains             type              <class 'selenium.webdrive<...>ion_chains.ActionChains'>
By                       type              <class 'selenium.webdriver.common.by.By'>
ChromeDriverManager      type              <class 'webdriver_manager<...>ome.ChromeDriverManager'>
EC                       module            <module 'selenium.webdriv<...>\expected_conditions.py'>
Keys                     type              <class 'selenium.webdriver.common.keys.Keys'>
MIMEApplication          type              <class 'email.mime.application.MIMEApplication'>
MIMEMultipart            type              <class 'email.mime.multipart.MIMEMultipart'>
MIMEText                 type              <class 'email.mime.text.MIMEText'>
NoSuchElementException   type              <class 'selenium.common.e<...>.NoSuchElementException'>
Options                  type              <class 'selenium.webdri

no_name                  DataFrame                                  <...>13703 rows x 214 columns]
no_pair                  DataFrame                           SKU Bra<...>[4030 rows x 130 columns]
non_phone                ndarray           0: 0 elems, type `object`, 0 bytes
non_phone1               DataFrame         Empty DataFrame\nColumns:<...>[Store, Phone]\nIndex: []
np                       module            <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
null_sku                 DataFrame         Empty DataFrame\nColumns:<...>, Toko/Gudang]\nIndex: []
number_of_days           int               29
options                  Options           <selenium.webdriver.chrom<...>ct at 0x000001F187D67D60>
orderItemNo              int               12206729469
orderNo                  str               12147325855
ornum                    DataFrame                    Order # Sales <...>hidin I No.16, RT.00...  
os                       module            <module 'os' from 'C:\\Us<...>

In [18]:
del data_magentoUser, forstok_all

In [3]:
pd. __version__

'1.4.0'

In [ ]:
data_all['Year'].unique()

In [53]:
data_all = data_all[data_all['True datetime'] >= '2022-01-01']

In [12]:
### run 14
### export masterdata
data_all.to_csv(r'D:\Masterdata\Clean Data\data_all_' + data_now + '.csv', sep = ';', index = False)


###OPTIONAL - Temporary Clean Data kalau Python Error
#data_all.to_csv(r'D:\Masterdata\Clean Data\data_all_' + data_now + '_temp.csv', sep = ';', index = False)

In [49]:
import time
import pandas as pd
import numpy as np
import math

In [1]:
###Optional
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time
import pandas as pd
import numpy as np
import math

import smtplib
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os

data_all =pd.read_csv(r"D:\Masterdata\Clean Data\data_all_5 February With Order Online_temp.csv", sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})

data_all['True datetime']=pd.to_datetime(data_all['True datetime'])
data_all['Real SKU']=data_all['Real SKU'].astype(str)
data_all['Phone']=data_all['Phone'].str.extract('(\d+)')

data_now = '5 February With Order Online'

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_11088\425283804.py:20: DtypeWarning: Columns (12,15,20,24,25,27,29,30,31,32,33,34,35,36,37,39,53,54,57,59,60,66,67,73,74,80,81,87,88,94,95,101,102,108,122,123,134,135,136,153,159,161,167,168,169,172,173,174,175,176,180,181,183,196,197,198,199,200,201,202,203,205,209,210) have mixed types. Specify dtype option on import or set low_memory=False.
  data_all =pd.read_csv(r"D:\Masterdata\Clean Data\data_all_5 February With Order Online_temp.csv", sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})


Export Lite Version

In [6]:
mappingarea=pd.read_excel("data_supp\Database Area untuk Report.xlsx")
mappingarea=mappingarea.rename(columns=({'Region':'Real Region',"City":'Region Group'}))
# temp=data_all.merge(mappingarea,how='left')
# temp.loc[temp['Region Group'].isnull(),'Real Region']='Retail Online'
for i in mappingarea['Real Region'].unique():
    print(i)
    reg=mappingarea[mappingarea['Real Region']==i]['Region Group'].unique()
    data_all.loc[data_all['Region Group'].isin(reg),'Real Region']=i
data_all.loc[data_all['Region Group'].isnull(),'Real Region']='Retail Online'

Sumatera 1
BARA
Sumatera 2
Jabodetabek
Jawa Tengah
Kalimantan
Sulawesi
Jawa Timur
Jawa Barat
PUMA


In [ ]:
##NOT USED
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
data_all[(data_all['Year'].isin([2021,2022]))&
         (data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Store Type','Store','Channel','Real Region','Warehouse Name','Order Status','Region Group','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index().to_excel(f'masterdata_{date.today()}_lite.xlsx',index=False)
# data_all.groupby(['Year','Month','Date','Store Type','Store','Channel','Order Status'])[['Order #']].nunique().reset_index().to_excel(f'Cek order gerak {date.today()}.xlsx',index=False)

In [7]:
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
data_all[(data_all['True datetime'] >= '2023-11-01')&
         (data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Hour','Store Type','Store','Channel','Warehouse Name','Real Region','City','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Exported Parent Item'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index().to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_lite new.xlsx',index=False)
# data_all.groupby(['Year','Month','Date','Store Type','Store','Channel','Order Status'])[['Order #']].nunique().reset_index().to_excel(f'Cek order gerak {date.today()}.xlsx',index=False)

In [45]:
data_all[data_all['True datetime']>'2023-01-01'][['Order #','Order Status','Year','Month','Week','Date','Store Type','Store',
                                                  'Channel','Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced',
                                                  'Total Net Before PPN','Total','Coupon Code','Customer Name','Customer Email',
                                                  'Phone','Address','Warehouse Name','City','Region','Region Group','Real Region']].to_excel(f'D:\Masterdata\Masterdata Lite\Masterdata Lite Order {date.today()}.xlsx',index=False)

ValueError: This sheet is too large! Your sheet size is: 1362807, 26 Max sheet size is: 1048576, 16384

In [36]:
data_all[(data_all['True datetime']>'2022-04-01')][['Order #','Order Status','Year','Month','Week','Date','Store Type','Store',
                                                  'Channel','Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced',
                                                  'Total Net Before PPN','Total','Coupon Code','Customer Name','Customer Email',
                                                  'Phone','Address','Warehouse Name','City','Region','Region Group','Real Region']].to_excel(f'Masterdata Lite Order {date.today()}.xlsx',index=False)

ValueError: This sheet is too large! Your sheet size is: 1121077, 26 Max sheet size is: 1048576, 16384

In [9]:
##JANGAN PAKE
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
resb=data_all[(data_all['Year'].isin([2023]))&(data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Store Type','Store','Channel','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Payment Channel','City','Real Region'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum','Total Net' : 'sum'}).reset_index()
resb=resb[resb['Store Type']!='Organic']#['Store'].unique()
resa=data_all[(data_all['Year'].isin([2023]))&(data_all['Order Status'].isin(osf['order filter'].unique()))&
                           (data_all['Store Type']=='Organic')][['Order #','Year','Month','Date','Store Type','Store','Channel','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced','Total','Total Net','Total Net Before PPN','Customer Name','Customer Email','Phone','Payment Channel','City','Real Region']]
print('tinggal append export')
resb.append(resa).to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_custlite v2.xlsx',index=False)

tinggal append export


ValueError: This sheet is too large! Your sheet size is: 1465547, 23 Max sheet size is: 1048576, 16384

In [95]:
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
cust_1 = data_all[(data_all['True datetime'] > '2023-10-01') &(data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Order #','Year','Month','Date','Store Type','Store','Channel','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Customer Name','Customer Email','Phone','Payment Channel','City','Real Region'], dropna = False).agg({'Qty. Invoiced':'sum','Total Net Before PPN':'sum','Total Net' : 'sum','Total' : 'sum'}).reset_index()
#cust_2 = data_all[(data_all['True datetime'] > '2023-08-01')&(data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Order #','Year','Month','Date','Store Type','Store','Channel','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Customer Name','Customer Email','Phone','Payment Channel','City','Region Group','Real Region'], dropna = False).agg({'Qty. Invoiced':'sum','Total Net Before PPN':'sum','Total Net' : 'sum','Total' : 'sum'}).reset_index()
cust_1.to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_custlite v2.xlsx',index=False)
#cust_2.to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_custlite v3.xlsx',index=False)

In [50]:
data_all[(data_all['Store Type']=='Organic')]['Store'].unique()

array(['Nutrimart', 'Order Online'], dtype=object)

In [47]:
##JANGAN PAKE
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
resb=data_all[(data_all['Year'].isin([2023]))&(data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Store Type','Store','Channel','Coupon Code','Payment Channel','City','Real Region'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index()
resb=resb[resb['Store Type']!='Organic']#['Store'].unique()
resa=data_all[(data_all['Year'].isin([2023]))&(data_all['Order Status'].isin(osf['order filter'].unique()))&
                           (data_all['Store Type']=='Organic')][['Order #','Year','Month','Date','Store Type','Store','Channel','Coupon Code','Qty. Invoiced','Total','Total Net','Total Net Before PPN','Customer Name','Customer Email','Phone','Payment Channel','City','Real Region']]
print('tinggal append export')
resb.append(resa).to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_custlite v3.xlsx',index=False)

tinggal append export


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_17104\4093137161.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resb.append(resa).to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_custlite v3.xlsx',index=False)


In [3]:
#WIM
osf=pd.read_excel(r"data_supp\order filter.xlsx")
from datetime import date
data_all.loc[data_all['Store']=='Blibli','Warehouse Name']='Blibli Warehouse'
data_all.loc[data_all['Store'].isin(['Shopee', 'TikTok']),'Warehouse Name']='SSI Warehouse'
data_all[(data_all['True datetime']>='2023-11-01')&(data_all['Order Status'].isin(osf['order filter'].unique()))][['Sales Order ID','Order #','Order Status','Year','Month','Week','Date','Store Type','Store',
                                                  'Channel','Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced',
                                                  'Total Net Before PPN','Total','Coupon Code','Customer Name','Customer Email',
                                                  'Phone','Address','Warehouse Name','City','Region','Region Group','Real Region']].to_excel(f'D:\Masterdata\Masterdata Lite\Masterdata Lite Order {date.today()}.xlsx',index=False)

In [68]:
data_all.loc[data_all['Store']=='Blibli','Warehouse Name']='Blibli Warehouse'

In [71]:
data_all.loc[data_all['Store']=='Blibli']['Warehouse Name'].unique()

array(['Blibli Warehouse'], dtype=object)

In [67]:
for i in data_all.columns :
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4

In [34]:
from datetime import date

data_all[(data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Store Type','Store','Channel','Brand','Sub Brand','Real SKU','Parent Item','Real Nama Produk','Bundle Name','Exported Parent Item'],dropna=False).agg({'Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index().to_excel(f'Data All Aggregat {date.today()}.xlsx',index=False)
# data_all.groupby(['Year','Month','Date','Store Type','Store','Channel','Order Status'])[['Order #']].nunique().reset_index().to_excel(f'Cek order gerak {date.today()}.xlsx',index=False)

In [46]:
coret = data_all[(data_all['Store'].isin(['Shopee','Nutrimart','JD Indonesia']))&
                 (data_all['Order Status'].isin(osf['order filter']))&
                 (data_all['Brand'].isin(['HiLo','TS','NS','WDANK','L-Men']))&
                 (data_all['Year'] == 2022)]
coret['Sales Order ID'].fillna(coret['Order #'], inplace = True)
coret1 = coret.groupby(['Month','Store','Sales Order ID']).agg({'Total Net Before PPN' : 'sum'}).reset_index()
coret2 = coret1[coret1['Total Net Before PPN'] > 100000].groupby(['Store','Month']).agg({'Sales Order ID' : 'nunique'}).reset_index()
coret3 = coret2.groupby('Store').agg({'Sales Order ID' : 'mean'}).reset_index()
coret3

,Store,Sales Order ID
0,JD Indonesia,2609.083333
1,Nutrimart,690.416667
2,Shopee,9558.666667


In [43]:
coret2['Total Net Before PPN'].min()

100400.0

In [ ]:
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
resb=data_all[(data_all['Year'].isin([2023,2022]))&(data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Store Type','Store','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Region','Real Region'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index()
resb=resb[resb['Store Type']!='Organic']#['Store'].unique()
resa=data_all[(data_all['Year'].isin([2022,2023]))&(data_all['Order Status'].isin(osf['order filter'].unique()))&
                           (data_all['Store Type']=='Organic')][['Year','Month','Date','Store Type','Store','Channel','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced','Total Net Before PPN','Region','Real Region']]
print('tinggal append export')
resb.to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_lite_v3.xlsx',index=False)

In [54]:
for i in data_all.columns:
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4

In [ ]:
#CORET2

In [46]:
#DATA BENNA
nm = data_all[data_all['Store'].isin(['Nutrimart']) & (~data_all['Coupon Code'].str.startswith('ORGRSL|BODETARSL', na = False))]
nm = nm[(nm['True datetime'] >= '2022-03-20') & (nm['True datetime'] < '2023-03-21')]
nm['u_cust'] = nm['Customer Email'] + nm['Phone']

NR = nm.drop_duplicates('Order #')[['Order #','Year','Month','Date','u_cust']].sort_values(by = ['Year', 'Month', 'Date'], ascending = True)
NR['cust_time'] = NR.groupby(['u_cust']).cumcount()
nm = pd.merge(nm,
              NR[['Order #','cust_time']], 
              on ='Order #',
              how = 'left')

R = nm.groupby(['u_cust']).agg({'True datetime' : 'max'}).reset_index()
time_A = datetime(2022, 12, 20)
R['Recency'] = R['True datetime'].apply(lambda x : 'Active' if x > time_A else 'Not Active')
nm = pd.merge(nm,
              R[['u_cust','Recency']], 
              on ='u_cust',
              how = 'left')

F = nm.groupby(['u_cust']).agg({'cust_time' : 'max'}).reset_index()
F['Frequency'] = F['cust_time'].apply(lambda x : 'Loyal' if x > 2 else 'Not Loyal')
nm = pd.merge(nm,
              F[['u_cust','Frequency']], 
              on ='u_cust',
              how = 'left')

nm_al = nm[nm['Recency'].isin(['Active']) & nm['Frequency'].isin(['Loyal'])]
nm_al = nm_al[nm_al['Bundle Name'].isnull()]
benna = nm_al.groupby(['Brand','Real Nama Produk']).agg({'u_cust' : 'nunique'}).reset_index()
benna = benna[benna['Brand'].isin(['L-Men', 'NS', 'TS', 'Bundle', 'HiLo', 'WDANK', 'Lain Lain'])].sort_values('u_cust', ascending = False)

benna_2 = nm.groupby('Phone').agg({'Customer Email' : 'nunique'}).reset_index().sort_values('Customer Email', ascending = False)

with pd.ExcelWriter("benna.xlsx") as writer:
    benna.to_excel(writer, sheet_name="Top 10 SKU",index = False)
    benna_2.to_excel(writer, sheet_name="Account Badungan",index = False)    

In [2]:
#Bundle Tokped
tokped = data_all[data_all['Store Type'].isin(['Marketplace']) & data_all['Store'].isin(['Tokopedia']) & data_all['Bundle Name'].isnull()]
tokped = tokped[(tokped['True datetime'] >= '2022-09-01') & (tokped['True datetime'] < '2023-03-01')]
tokped1 = tokped.groupby(['Brand','Real Nama Produk']).agg({'Price List NFI' : 'max'}).reset_index()
tokped1 = tokped1[(tokped1['Price List NFI'] < 100000) & (tokped1['Price List NFI'] >= 50000)]['Real Nama Produk'].unique()
tokped2 = tokped[tokped['Real Nama Produk'].isin(tokped1)]
tokped2

nt = data_all[data_all['Store Type'].isin(['Marketplace']) & (~data_all['Store'].isin(['Tokopedia'])) & data_all['Bundle Name'].isnull()]
nt = nt[(nt['True datetime'] >= '2022-09-01') & (nt['True datetime'] < '2023-03-01')]
nt1 = nt.groupby(['Brand','Real Nama Produk']).agg({'Price List NFI' : 'max'}).reset_index()
nt1 = nt1[(nt1['Price List NFI'] < 100000) & (nt1['Price List NFI'] >= 50000)]['Real Nama Produk'].unique()
nt2 = nt[nt['Real Nama Produk'].isin(nt1)]
nt2

with pd.ExcelWriter("bundle tokped.xlsx") as writer:
    tokped2.to_excel(writer, sheet_name="Tokped",index = False)
    nt2.to_excel(writer, sheet_name="Non-Tokped",index = False)    

In [55]:
#PESANAN > 500K
df = data_all[(data_all['True datetime'] >= '2022-12-01') & (data_all['True datetime'] < '2023-03-01')]
df = df[df['Store Type'].isin(['Marketplace']) & df['Bundle Name'].isnull()]

df1 = df.groupby(['Month','Store','Order #']).agg({'Total Net Before PPN' : 'sum',
                                                          'Total' : 'sum'}).reset_index()
df1 = df1[df1['Total'] >= 500000]

df1.groupby(['Month','Store']).agg({'Order #' : 'nunique'}).reset_index()

,Month,Store,Order #
0,December,Blibli,211
1,December,Bukalapak,33
2,December,JD Indonesia,73
3,December,Lazada,354
4,December,Shopee,465
5,December,TikTok,93
6,December,Tokopedia,347
7,February,Aladin Mall,2
8,February,Blibli,217
9,February,Bukalapak,19


In [56]:
df1.groupby(['Month']).agg({'Order #' : 'nunique'}).reset_index()

,Month,Order #
0,December,1576
1,February,1643
2,January,1632


In [57]:
data_all.head()

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Payment Received Ref. number,Category Baru,Exported Parent Item,Unnamed: 0,Price List NFI_x,Customer Type,Real Region,True datetime1,Payment Date,Shipping Customer Name
0,220101KF9CRRPJ,#SO-115792812,JP9221384040,2022-01-01 00:01:00,Cancelled,2022-01-01 00:00:00,52.0,1.0,January,1.0,...,NaN,HILO TRADITIONAL,HILO CHOCOLATE TARO PLS 15RX10SX14G,NaN,NaN,NaN,Jabodetabek,2022-01-01 00:00:00,NaN,NaN
1,220101KF8V3UF6,#SO-115792804,JP2821530627,2022-01-01 00:01:00,Cancelled,2022-01-01 00:00:00,52.0,1.0,January,1.0,...,NaN,HILO TEEN,HILO TEEN COKLAT 500G (12D),NaN,NaN,NaN,Jawa Barat,2022-01-01 00:00:00,NaN,NaN
2,220101KF86KYT6,#SO-115792769,JP0117026545,NaN,Cancelled,2022-01-01 00:00:00,52.0,1.0,January,1.0,...,NaN,HILO TEEN,HILO TEEN POPCORN CARAMEL 12DX500G,NaN,NaN,NaN,Jabodetabek,2022-01-01 00:00:00,NaN,NaN
3,220101KF99MQ6H,#SO-115792810,JP6646092620,2022-01-01 00:01:00,Cancelled,2022-01-01 00:00:00,52.0,1.0,January,1.0,...,NaN,L-MEN POWDER,L-MEN GM CHOCOLATE 6DX500G,NaN,NaN,NaN,Jawa Tengah,2022-01-01 00:00:00,NaN,NaN
4,220101KF7SYVTJ,#SO-115792738,JP5713456513,2022-01-01 00:05:00,Delivered,2022-01-01 00:00:00,52.0,1.0,January,1.0,...,NaN,L-MEN POWDER,L-MEN GM TARO 12DX225G,NaN,NaN,NaN,Jabodetabek,2022-01-01 00:00:00,NaN,NaN


In [ ]:
#HILO GOLD

In [42]:
df = data_all[data_all['Year'].isin([2023]) & data_all['Store'].isin(['Shopee','TikTok']) & data_all['Month'].isin(['January', 'February', 'March'])][['Order #','Year','Month','Date','Store Type','Store','Channel','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Customer Name','Customer Email','Phone','Payment Channel','City','Real Region','Qty. Invoiced','Total Net Before PPN']]
df = df[df['Real SKU'].isin(['2101524180','2101500180','2101500190','2101551180','2101551190','2101584180','2101500195','2101578180','2101551195',
                            '2101524180P2','2101500180P2','2101500190P2','2101551180P2','2101551190P2','2101584180P2','2101500195P2','2101578180P2','2101551195P2',
                            '2101751112','2101700166','2101751112P2','2101700166P2']) & df['Bundle Name'].isnull()]

In [43]:
df1 = df.groupby(['Month','Store','Order #','Real Nama Produk']).agg({'Total Net Before PPN' : 'sum','Qty. Invoiced' : 'sum'}).reset_index()
df1

,Month,Store,Order #,Real Nama Produk,Total Net Before PPN,Qty. Invoiced
0,February,Shopee,230201TF7RU4E6,HiLo Gold Plain 1000 gr,125000.0,1.0
1,February,Shopee,230201TJ2HPS0H,HiLo Platinum Swiss Chocolate 420gr,82000.0,1.0
2,February,Shopee,230201TMMBJS3S,HiLo Gold Vanilla 500gr,140000.0,2.0
3,February,Shopee,230201TNDS152P,HiLo Gold Chocolate 1000g,118000.0,1.0
4,February,Shopee,230201TNGDXV9U,HiLo Platinum Swiss Chocolate 420gr,82000.0,1.0
...,...,...,...,...,...,...
3877,March,TikTok,577154099018304156,HiLo Platinum Original 360g (12 Sachet),100000.0,1.0
3878,March,TikTok,577154946022672437,HiLo Platinum Swiss Chocolate 420gr,82000.0,1.0
3879,March,TikTok,577155028714752967,Twin Pack: HiLo Platinum Swiss Chocolate (12 Sch),164000.0,1.0
3880,March,TikTok,577155570277583494,HiLo Platinum Swiss Chocolate 420gr,82000.0,1.0


In [44]:
df2 = df1.groupby(['Month','Store','Order #']).agg({'Total Net Before PPN' : 'sum'}).reset_index()
df2

,Month,Store,Order #,Total Net Before PPN
0,February,Shopee,230201TF7RU4E6,125000.0
1,February,Shopee,230201TJ2HPS0H,82000.0
2,February,Shopee,230201TMMBJS3S,140000.0
3,February,Shopee,230201TNDS152P,118000.0
4,February,Shopee,230201TNGDXV9U,82000.0
...,...,...,...,...
3743,March,TikTok,577154099018304156,100000.0
3744,March,TikTok,577154946022672437,82000.0
3745,March,TikTok,577155028714752967,164000.0
3746,March,TikTok,577155570277583494,82000.0


In [50]:
d250 = df2[df2['Store'].isin(['Shopee'])][df2['Total Net Before PPN'] >= 250000]
d250['Order #'].nunique()

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_22732\880581587.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d250 = df2[df2['Store'].isin(['Shopee'])][df2['Total Net Before PPN'] >= 250000]


351

In [52]:
d250 = df2[df2['Store'].isin(['TikTok'])][df2['Total Net Before PPN'] >= 250000]
d250['Order #'].nunique()

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_22732\1205806559.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d250 = df2[df2['Store'].isin(['TikTok'])][df2['Total Net Before PPN'] >= 250000]


32

In [53]:
d200250 = df2[df2['Store'].isin(['Shopee'])][(df2['Total Net Before PPN'] >= 200000) & (df2['Total Net Before PPN'] < 250000)]
d200250['Order #'].nunique()

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_22732\2433765209.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d200250 = df2[df2['Store'].isin(['Shopee'])][(df2['Total Net Before PPN'] >= 200000) & (df2['Total Net Before PPN'] < 250000)]


197

In [54]:
d200250 = df2[df2['Store'].isin(['TikTok'])][(df2['Total Net Before PPN'] >= 200000) & (df2['Total Net Before PPN'] < 250000)]
d200250['Order #'].nunique()

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_22732\3574678150.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d200250 = df2[df2['Store'].isin(['TikTok'])][(df2['Total Net Before PPN'] >= 200000) & (df2['Total Net Before PPN'] < 250000)]


16

In [56]:
d200 = df2[df2['Store'].isin(['Shopee'])][(df2['Total Net Before PPN'] >= 200000)]
d200['Order #'].nunique()

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_22732\4176157121.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d200 = df2[df2['Store'].isin(['Shopee'])][(df2['Total Net Before PPN'] >= 200000)]


548

In [55]:
d200 = df2[df2['Store'].isin(['TikTok'])][(df2['Total Net Before PPN'] >= 200000)]
d200['Order #'].nunique()

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_22732\3249763760.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d200 = df2[df2['Store'].isin(['TikTok'])][(df2['Total Net Before PPN'] >= 200000)]


48

In [51]:
d2 = df1.groupby(['Store','Month','Real Nama Produk']).agg({'Qty. Invoiced' : 'sum'}).reset_index()
d2

,Store,Month,Real Nama Produk,Qty. Invoiced
0,Shopee,February,BUY 1 GET 1 - HiLo Platinum Original 360 gram,67.0
1,Shopee,February,Buy 1 Get 1 Free - HiLo Gold Biscuit Cereal 500g,15.0
2,Shopee,February,HiLo Gold Biscuit Cereal 500g,17.0
3,Shopee,February,HiLo Gold Chocolate 1000g,149.0
4,Shopee,February,HiLo Gold Chocolate 1000g x 2 pcs,45.0
...,...,...,...,...
89,TikTok,March,HiLo Gold Sweet Potato 500 gram,2.0
90,TikTok,March,HiLo Gold Vanilla 500gr,45.0
91,TikTok,March,HiLo Platinum Original 360g (12 Sachet),25.0
92,TikTok,March,HiLo Platinum Swiss Chocolate 420gr,132.0


In [58]:
with pd.ExcelWriter("HiLo Gold Platinum.xlsx") as writer:
    df1.to_excel(writer, sheet_name="Raw Data",index = False)
    d2.to_excel(writer, sheet_name="Unit Sold per Month",index = False)    

In [64]:
all_single[all_single['Brand'].isin(['TS','NS','HiLo','L-Men','WDANK'])].groupby(['Year','Month','Date','Store']).agg({'Total Net Before PPN' : 'sum'}).reset_index().to_excel(f'D:\Masterdata\Masterdata Lite\cek.xlsx')


In [61]:
all_single[all_single['Brand'].isin(['TS','NS','HiLo','L-Men','WDANK'])].groupby(['Year','Month','Date','Store']).agg({'Total Net Before PPN' : 'sum'})

Total Net Before PPN
Year   Month Date Store                                      
2021.0 April 1.0  ECOM RETAIL - TANIHUB          3.883200e+06
                  Shopee Sell-In                 2.511588e+08
             3.0  ECOM RETAIL - TANIHUB          3.376800e+06
             6.0  ECOM RETAIL - TANIHUB          2.712000e+06
             7.0  ECOM RETAIL - TANIHUB          2.983200e+06
...                                                       ...
2023.0 May   31.0 Nutrimart                      5.385176e+06
                  Order Online                   4.501239e+06
                  Shopee                         7.141970e+07
                  TikTok                         7.400273e+07
                  Tokopedia                      8.295434e+07

[2320 rows x 1 columns]

In [ ]:
#MEMBERSHIP LAZADA

In [44]:
osf=pd.read_excel(r"data_supp\order filter.xlsx")
df = data_all[(data_all['True datetime'] >= '2022-04-01') & (data_all['True datetime'] < '2023-04-01') &
              (data_all['Store'].isin(['Lazada'])) &
              (data_all['Order Status'].isin(osf['order filter'].unique())) &
              (data_all['Bundle Name'].isnull())]
itemlist = ['2102040225P2', '2305551028', '2104405', '2104251230', '2101453195P2', '2102200208', '2101453195',
            '2105084180P2', '2101452195', '2101651195P2', '2104228230', '2101459180', '2101452195P2', '2101151180',
            '2105028180']
df

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Payment Received Ref. number,Category Baru,Exported Parent Item,Unnamed: 0,Price List NFI_x,Customer Type,Real Region,True datetime1,Payment Date,Shipping Customer Name
448881,827778692482369,#SO-119116543,JNAP-0102131728,2022-01-04 00:02:00,Delivered,2022-01-04 00:02:00,13.0,1.0,April,2.0,...,NaN,NS TRADITIONAL,NS JERUK PERAS PLS 18PX40SX14G,NaN,NaN,NaN,Jawa Tengah,2022-04-01 00:02:00,NaN,NaN
448887,827787666149552,#SO-119116575,LXAD-2539085327,2022-01-04 00:04:00,Delivered,2022-01-04 00:04:00,13.0,1.0,April,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jabodetabek,2022-04-01 00:04:00,NaN,NaN
448899,834678119603713,#SO-119116663,NLIDAT0003674738,2022-01-04 00:11:00,Delivered,2022-01-04 00:11:00,13.0,1.0,April,2.0,...,NaN,L-MEN POWDER,L-MEN PLATINUM CHOCO LATTE 6KLRX800G,NaN,NaN,NaN,Sumatera 1,2022-04-01 00:11:00,NaN,NaN
448900,834678119603713,#SO-119116663,NLIDAT0003674738,2022-01-04 00:11:00,Delivered,2022-01-04 00:11:00,13.0,1.0,April,2.0,...,NaN,TS KUNING OTHERS,TS KOREAN GOGUMA COOKIES 12DX5SX20G,NaN,NaN,NaN,Sumatera 1,2022-04-01 00:11:00,NaN,NaN
448913,834679921264717,#SO-119116684,JNAP-0102131354,2022-01-04 00:14:00,Delivered,2022-01-04 00:14:00,13.0,1.0,April,2.0,...,NaN,NS TRADITIONAL,NS JERUK PERAS PLS 18PX40SX14G,NaN,NaN,NaN,Jawa Tengah,2022-04-01 00:14:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364928,1129869781744420,SO-130882094,LXAT-8002069211,NaN,Completed,2023-03-31 00:20:00,13.0,31.0,March,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jabodetabek,2023-03-31 00:20:00,2023-03-31 00:20:00,ANGGIA ASMAWI
2364932,1129875703360057,SO-130881775,LXAT-8002068822,NaN,Completed,2023-03-31 00:09:00,13.0,31.0,March,1.0,...,NaN,TS KUNING OTHERS,TS SYRUP ORANGE 12BTLX750ML,NaN,NaN,NaN,Jabodetabek,2023-03-31 00:09:00,2023-03-31 00:09:00,Urip Widodo
2364933,1119587245660057,SO-130881735,LXAT-8002068905,NaN,Completed,2023-03-31 00:08:00,13.0,31.0,March,1.0,...,NaN,TS KUNING OTHERS,TS SYRUP COCOPANDAN 12BTLX750ML,NaN,NaN,NaN,Jabodetabek,2023-03-31 00:08:00,2023-03-31 00:08:00,Urip Widodo
2364934,1129868553833734,SO-130881733,"LXAT-8002068824, LXAT-8002068878",NaN,Completed,2023-03-31 00:08:00,13.0,31.0,March,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jabodetabek,2023-03-31 00:08:00,2023-03-31 00:08:00,yonna


In [63]:
df['Phone'] = df['Phone'].str.replace('="', '')
df['Phone'] = df['Phone'].str.replace('"', '')
df['Phone']

448881     085644827518
448887     085884908776
448899     081276399664
448900     081276399664
448913     081734986553
               ...     
2364928    087788805505
2364932    081953344033
2364933    081953344033
2364934    085881618464
2364935    085881618464
Name: Phone, Length: 196397, dtype: object

In [67]:
df1 = df.groupby(['Phone']).agg({'True datetime' : 'max',
                                 'Total Net Before PPN' : 'sum',
                                 'Order #' : 'nunique'}).reset_index()

In [73]:
with pd.ExcelWriter("data customer laz.xlsx") as writer:
    df1.to_excel(writer,index = False)

In [72]:
df1['Last Order'] = datetime.today() - df1['True datetime']
df1['Last Order'] = df1['Last Order'].dt.days
df1

,Phone,True datetime,Total Net Before PPN,Order #,Last Order
0,0,2023-01-29 23:31:00,15000.0,1,85
1,0213852000,2022-10-24 18:37:00,44000.0,2,182
2,0215533041,2022-12-02 21:11:00,342000.0,1,143
3,0215915811,2022-11-28 14:25:00,56000.0,1,148
4,0217443172,2023-01-28 13:28:00,85000.0,4,87
...,...,...,...,...,...
96094,18632799172,2022-08-18 05:10:00,260000.0,1,250
96095,447795550112,2023-03-27 08:55:00,92500.0,1,29
96096,6582035798,2022-04-30 12:29:00,78000.0,1,360
96097,6586200749,2022-06-02 15:01:00,118000.0,1,327


In [ ]:
#REPAIRING ORAMI

In [42]:
orami2 = data_all[data_all['Store'].isin(['Orami Shopping'])].drop_duplicates(subset = ['Order #','Order Status','Brand','Real Nama Produk','Qty. Invoiced'], keep = 'last')

In [43]:
orami2 = orami2[~orami2['Month'].isin(['May'])]

In [44]:
orami2

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Exported Parent Item,Unnamed: 0,Price List NFI_x,Customer Type,Real Region,True datetime1,Payment Date,Shipping Customer Name,Shipped Date,Note
579847,20498202,20498202,001567505362,NaN,Selesai,NaN,NaN,27.0,March,NaN,...,NaN,NaN,NaN,NaN,Retail Online,2023-03-27 10:27:00,NaN,NaN,NaN,NaN
579848,20497942,20497942,001567505361,NaN,Selesai,NaN,NaN,27.0,March,NaN,...,NaN,NaN,NaN,NaN,Retail Online,2023-03-27 10:15:00,NaN,NaN,NaN,NaN
579849,20497942,20497942,001567505361,NaN,Selesai,NaN,NaN,27.0,March,NaN,...,NS JERUK PERAS REF 12DX500G,NaN,NaN,NaN,Retail Online,2023-03-27 10:15:00,NaN,NaN,NaN,NaN
579850,20497942,20497942,001567505361,NaN,Selesai,NaN,NaN,27.0,March,NaN,...,NaN,NaN,NaN,NaN,Retail Online,2023-03-27 10:15:00,NaN,NaN,NaN,NaN
579851,20419898,20419898,001567505350,NaN,Selesai,NaN,NaN,23.0,March,NaN,...,NaN,NaN,NaN,NaN,Retail Online,2023-03-23 09:03:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612881,20993712,20993712,147802300000448.0,NaN,Selesai,NaN,NaN,28.0,April,NaN,...,HILO THAI TEA PLS 8RX10SX15G,NaN,NaN,NaN,Retail Online,2023-04-28 23:55:00,NaN,NaN,NaN,NaN
612882,21023564,21023564,147952300000276.0,NaN,Selesai,NaN,NaN,30.0,April,NaN,...,HILO SCHOOL RTD COTTON CANDY 24TPKX200ML,NaN,NaN,NaN,Retail Online,2023-04-30 23:57:00,NaN,NaN,NaN,NaN
612884,21016839,21016839,1567505456.0,NaN,Selesai,NaN,NaN,30.0,April,NaN,...,TS RTD ALMOND DRINK CHOCOLICIOUS 24TPKX190ML,NaN,NaN,NaN,Retail Online,2023-04-30 14:45:00,NaN,NaN,NaN,NaN
612885,21015765,21015765,,NaN,Dibatalkan,NaN,NaN,30.0,April,NaN,...,TS RTD ALMOND DRINK CHOCOLICIOUS 24TPKX190ML,NaN,NaN,NaN,Retail Online,2023-04-30 13:10:00,NaN,NaN,NaN,NaN


In [45]:
orami2[orami2['Brand'].isin(['HiLo','L-Men','TS','NS','WDANK']) & orami2['Order Status'].isin(['Selesai'])].groupby(['Year','Month']).agg({'Total Net Before PPN' : 'sum'})

Total Net Before PPN
Year   Month                         
2023.0 April             4.507432e+06
       February          1.729000e+06
       January           1.240100e+07
       March             1.974600e+06

In [119]:
orami2[orami2['Month'].isin(['May'])][['Date','Order #','Order Status','Brand','Real Nama Produk','Qty. Invoiced']].drop_duplicates(subset = ['Order #','Order Status','Brand','Real Nama Produk','Qty. Invoiced'], keep = 'last').sort_values(by = 'Date')

,Date,Order #,Order Status,Brand,Real Nama Produk,Qty. Invoiced
636445,5.0,21091370,Canceled,TS,Tropicana Slim Classic (50 sch),2.0
646743,5.0,21091370,Selesai,TS,Tropicana Slim Classic (50 sch),2.0
668116,14.0,21217267,Canceled,Bundle,Twin Pack: HiLo School Chocolate 1000gr,3.0
668117,14.0,21217267,Canceled,HiLo,HiLo School Chocolate 1000 gr,6.0
676057,14.0,21217267,Selesai,Bundle,Twin Pack: HiLo School Chocolate 1000gr,3.0
676058,14.0,21217267,Selesai,HiLo,HiLo School Chocolate 1000 gr,6.0
724248,23.0,21340223,Canceled,L-Men,L-Men Platinum Kacang Hijau 800g,2.0
778797,23.0,21340223,Selesai,L-Men,L-Men Platinum Kacang Hijau 800g,2.0
724247,26.0,21385948,Canceled,Bundle,Paket HiLo School 3+ - HiLo School 3+ Strawber...,1.0
724249,26.0,21385948,Canceled,HiLo,HiLo School 3+ Strawberry Pop 400g,1.0


In [ ]:
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
orami2[(orami2['True datetime'] >= '2023-01-01')&
         (orami2['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Hour','Store Type','Store','Channel','Warehouse Name','Real Region','City','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index().to_excel(f'D:\Masterdata\Masterdata Lite\orami_{date.today()}_lite new.xlsx',index=False)
# data_all.groupby(['Year','Month','Date','Store Type','Store','Channel','Order Status'])[['Order #']].nunique().reset_index().to_excel(f'Cek order gerak {date.today()}.xlsx',index=False)

In [46]:
data_all = data_all[~data_all['Store'].isin(['Orami Shopping'])]
data_all = data_all.append(orami2, ignore_index = True, sort = False)

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_21228\2713276997.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_all = data_all.append(orami2, ignore_index = True, sort = False)


In [49]:
#CROSS BASKET

In [66]:
osf=pd.read_excel(r"data_supp\order filter.xlsx")
df = data_all[data_all['Order Status'].isin(osf['order filter'].unique()) &
              data_all['Brand'].isin(['HiLo','NS','TS','L-Men','WDANK']) &
              data_all['Store Type'].isin(['Marketplace'])&
              data_all['Store'].isin(['Lazada'])]
#order = df[df['Real SKU'].str.contains('1101909331|1101569326', case = False, na = False)]['Order #'].unique()
order = df[df['Brand'].isin(['NS'])]['Order #'].unique()
df1 = df[(df['True datetime'] >= '2023-05-01') & df['Order #'].isin(order) & (~df['Real SKU'].str.startswith('(E)')) & df['Brand'].isin(['WDANK','HiLo','TS','L-Men'])]
df2 = df1.groupby(['Real Nama Produk']).agg({'Order #' : 'nunique'}).reset_index().sort_values(by = ['Order #'], ascending = False)
#df2[~df2['Real Nama Produk'].str.contains('RTD')].head(12)
df2

,Real Nama Produk,Order #
39,Hilo Chocolate Taro (10 sch),99
108,W'Dank Bajigur (10 sch),92
36,HiLo Thai Tea (10 sch),63
35,HiLo Teh Tarik 10 Sch,59
28,HiLo Swiss Chocolate (10 sch) Renceng,41
...,...,...
84,Tropicana Slim Minyak Jagung 946ml,1
44,L-Men Gain Mass Chocolate 225gr,1
43,L-Men Gain Mass Banana 225gr,1
16,HiLo School Chocolate 500gr,1


In [65]:
df2.head(22)

,Real Nama Produk,Order #
14,Hilo Chocolate Taro (10 sch),32
35,W'Dank Bajigur (10 sch),21
12,HiLo Thai Tea (10 sch),19
11,HiLo Teh Tarik 10 Sch,15
8,HiLo Swiss Chocolate (10 sch) Renceng,11
3,HiLo Es Teler 10 sch,9
15,Hilo Es Ketan Hitam 10 sch,5
36,W'Dank Bandrek 10 sachet Renceng,5
2,HiLo Es Pisang Ijo 10 Sch,5
27,Tropicana Slim DIABTX (50 sch),4


In [ ]:
#L-MEN GOES TO EUROPE

In [48]:
import requests,json
import pandas as pd
osf=pd.read_excel(r"data_supp\order filter.xlsx")  ### jgn lupa ganti yang ini
allorder=requests.get("http://event.l-men.com/apiv1?type=lmen").json()
for i in requests.get("http://event.l-men.com/apiv1?type=lmenvoucheruser").json():
    if len(i['voucher'])>0:
        email=i['email']
        voulist=json.loads(i['voucher'].replace("'",'"'))
        user_ord=[x['orderid'] for x in allorder if x['email']==email]
        ord_detail=data_all[(data_all['Coupon Code'].isin(voulist))&
                            ~(data_all['Order #'].isin(user_ord))&
                            (data_all['Order Status'].isin(osf['order filter'].unique()))&
                            (data_all['Brand']=='L-Men')].groupby(['Order #'])[['Total Net Before PPN']].sum().reset_index()
        for orderid in ord_detail['Order #'].unique():
            val=ord_detail[ord_detail['Order #']==orderid]['Total Net Before PPN'].sum()
            requests.post("http://event.l-men.com/apiv1?id={0}&value={1}&rsl_email={2}".format(orderid,val,email))
            print(email+orderid)    

andrean_gan@yahoo.com1000114414
andrean_gan@yahoo.com1000114524
andrean_gan@yahoo.com1000114677
andrean_gan@yahoo.com1000114951
andrean_gan@yahoo.com1000115067
andrean_gan@yahoo.com1000115168
andrean_gan@yahoo.com1000115297
andrean_gan@yahoo.com1000115421


In [76]:
val

290000.0

In [ ]:
#TOKOPEDIA ERROR

In [6]:
tok=pd.read_excel(r"C:\Users\steven.nathanael\Downloads\Tokopedia_Order_20230530-20230531.xlsx")
tok.rename(columns = {'Nomor Invoice' : 'Order #',
                      'Nomor SKU' : 'Real SKU',
                      'Jumlah Produk Dibeli' : 'Qty. Invoiced',
                      'Nama Produk' : 'Product Name',
                      'Nama Pembeli' : 'Customer Name',
                      'No Telp Pembeli' : 'Phone',
                      'Alamat Pengiriman' : 'Address',
                      'Kota' : 'City',
                      'Provinsi' : 'Region',
                      'Gudang Pengiriman' : 'Warehouse Name',
                      'Nama Kurir' : 'Shipping Courier',
                      'No Resi / Kode Booking' : 'AWB',
                      'Status Terakhir' : 'Order Status',
                      'Total Penjualan' : 'Total',
                      'Harga Jual' : 'Selling Price'}, inplace = True)

tok['Day'] = tok['Tanggal Pembayaran'].str.slice(0, 2).astype(int)
tok['Month'] = tok['Tanggal Pembayaran'].str.slice(3, 5).astype(int)
tok['Year'] = tok['Tanggal Pembayaran'].str.slice(6, 10).astype(int)
tok['Hour'] = tok['Tanggal Pembayaran'].str.slice(11, 13).astype(int)
tok['Minute'] = tok['Tanggal Pembayaran'].str.slice(14, 16).astype(int)
tok['Second'] = tok['Tanggal Pembayaran'].str.slice(17, 20).astype(int)
tok['True datetime'] = pd.to_datetime(tok[['Year','Month','Day','Hour','Minute','Second']])
tok['Month'] = tok['Month'].apply(lambda x : 'September' if x == 9 else
                                            ('October' if x == 10 else
                                            ('November' if x == 11 else 
                                            ('December' if x == 12 else
                                            ('January' if x == 1 else
                                            ('February' if x == 2 else
                                            ('March' if x == 3 else
                                            ('April' if x == 4 else
                                            ('May' if x == 5 else
                                            ('June' if x == 6 else
                                            ('July' if x == 7 else 'August')))))))))))
tok.rename(columns = {'Day' : 'Date'}, inplace = True)


In [78]:
data_all.head()

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Exported Parent Item,Unnamed: 0,Price List NFI_x,Customer Type,Real Region,True datetime1,Payment Date,Shipping Customer Name,Shipped Date,Note
0,12121413238,12170350531,BLI2209226577020,NaN,Sudah Settlement,02/01/2023 09:21,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,Jabodetabek,2023-01-02,NaN,NaN,NaN,NaN
1,12121409053,12170344246,BLI2209227755413,NaN,Sudah Settlement,02/01/2023 08:19,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,Jabodetabek,2023-01-02,NaN,NaN,NaN,NaN
2,12121396731,12170324721,BLIGI00049654681-1,NaN,Delivered,01/01/2023 23:38,52.0,1.0,January,1.0,...,L-MEN PLATINUM CHOCO LATTE 6KLRX800G,NaN,NaN,NaN,Jabodetabek,2023-01-01,NaN,NaN,NaN,NaN
3,12121385758,12170309208,BLI1510222741567,NaN,Sudah Settlement,01/01/2023 19:53,52.0,1.0,January,1.0,...,NaN,NaN,NaN,NaN,Jabodetabek,2023-01-01,NaN,NaN,NaN,NaN
4,12121385635,12170309027,8811622233348591,NaN,Sudah Settlement,01/01/2023 19:51,52.0,1.0,January,1.0,...,L-MEN ADV CAPPUCINO 12DX250G,NaN,NaN,NaN,Kalimantan,2023-01-01,NaN,NaN,NaN,NaN


In [35]:
###Optional
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time
import pandas as pd
import numpy as np
import math

import smtplib
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os

df =pd.read_csv(r"C:\Users\steven.nathanael\Downloads\Nutrifood - Compas Dashboard_Page 4_Table (1).csv", sep = ',')


In [37]:
df.to_excel('compass lazada.xlsx', index = False)

In [60]:
##Customer Collagen Lazada
osf=pd.read_excel(r"data_supp\order filter.xlsx")
df = data_all[data_all['Store'].isin(['Lazada']) & data_all['Brand'].isin(['TS']) &
              data_all['Real Nama Produk'].str.contains('Collagen', na = False, case = False) &
              data_all['Order Status'].isin(osf['order filter'].unique())]
df = df[(df['True datetime'] > '2023-06-25')][['Customer Name','Phone','Customer Email']].drop_duplicates()

In [62]:
df.to_excel('Cust Laz Collagen.xlsx', index = False)

In [51]:
data_all[data_all['Order #'].isin(['1236898690108757'])]

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Shipped Date,Note,Ready to Ship Date,Completed Date,Cancelled Reason,Item Note,Shipping Fee (Cashless),Picklist ID,Package ID,Shipment ID
1261386,1236898690108757,SO-135260117,LXAD-3072139802,NaN,Delivered,2023-08-27 04:42:00,34.0,27.0,August,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,SHP-51958023


In [42]:
data_all[data_all['Store'].isin(['Lazada']) & data_all['Year'].isin([2023])][['Month','Phone']]

,Month,Phone
437,January,"=""085174064002"""
483,January,"=""087809969138"""
500,January,"=""081287876336"""
545,January,"=""085217701160"""
583,January,"=""08119781899"""
...,...,...
1199781,August,"=""089603101288"""
1199794,August,"=""08121220706"""
1199810,August,"=""088220283866"""
1199879,August,"=""089603101288"""


In [ ]:
#CUSTOMER LAZ

In [57]:
df = data_all[data_all['Store'].isin(['Lazada']) & data_all['Bundle Name'].isnull() & (data_all['Month']).isin(['August', 'June', 'July'])].groupby(['Month','Phone']).agg({'Total Net Before PPN' : 'sum'}).reset_index()
df

,Month,Phone,Total Net Before PPN
0,August,"=""02292547038""",114000.0
1,August,"=""0266211066""",106000.0
2,August,"=""0275322798""",152000.0
3,August,"=""0318941957""",76000.0
4,August,"=""062 85848469352""",46000.0
...,...,...,...
39932,June,"=""08999936656""",378600.0
39933,June,"=""08999951788""",400800.0
39934,June,"=""08999958787""",34000.0
39935,June,"=""08999996239""",3877200.0


In [71]:
df1 = df.groupby(['Phone']).agg({'Total Net Before PPN' : 'mean'}).reset_index()
df11 = df1[df1['Phone'].isin(cust)]

In [64]:
cust = df1[(df1['Total Net Before PPN'] >= 500000)]['Phone']
cust

0                   =""
63       ="08111001700"
74       ="08111082494"
94        ="0811118020"
115      ="08111393992"
              ...      
32239    ="08998389654"
32266    ="08999061396"
32282    ="08999351635"
32309    ="08999996239"
32311            ="nan"
Name: Phone, Length: 1330, dtype: object

In [67]:
df2 = data_all[data_all['Phone'].isin(cust) & data_all['Store'].isin(['Lazada']) & data_all['Bundle Name'].isnull() & (data_all['Month']).isin(['August', 'June', 'July'])]
df2.groupby(['Phone']).agg({'True datetime' : 'max',
                            'Order #' : 'nunique'})

,True datetime,Order #
Phone,,
"=""""",2023-06-27 23:57:00,65
"=""08111001700""",2023-07-07 18:05:00,5
"=""08111082494""",2023-08-27 21:20:00,101
"=""0811118020""",2023-08-29 12:06:00,25
"=""08111393992""",2023-06-26 23:24:00,4
...,...,...
"=""08998389654""",2023-08-26 14:02:00,10
"=""08999061396""",2023-08-05 11:05:00,3
"=""08999351635""",2023-07-11 06:43:00,3


In [70]:
df3 = df2.groupby(['Phone']).agg({'True datetime' : 'max',
                                  'Order #' : 'nunique'}).reset_index()
df3['Last Order (Days)'] = (pd.to_datetime("now")-df3['True datetime']).dt.days
df3

C:\Users\steven.nathanael\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


,Phone,True datetime,Order #,Last Order (Days)
0,"=""""",2023-06-27 23:57:00,65,64
1,"=""08111001700""",2023-07-07 18:05:00,5,54
2,"=""08111082494""",2023-08-27 21:20:00,101,3
3,"=""0811118020""",2023-08-29 12:06:00,25,1
4,"=""08111393992""",2023-06-26 23:24:00,4,65
...,...,...,...,...
1325,"=""08998389654""",2023-08-26 14:02:00,10,4
1326,"=""08999061396""",2023-08-05 11:05:00,3,25
1327,"=""08999351635""",2023-07-11 06:43:00,3,51
1328,"=""08999996239""",2023-08-10 07:48:00,38,21


In [72]:
df4 = pd.merge(df11,
               df3,
               on = 'Phone',
               how = 'inner')
df4

,Phone,Total Net Before PPN,True datetime,Order #,Last Order (Days)
0,"=""""",1.764750e+07,2023-06-27 23:57:00,65,64
1,"=""08111001700""",7.770000e+05,2023-07-07 18:05:00,5,54
2,"=""08111082494""",1.246667e+07,2023-08-27 21:20:00,101,3
3,"=""0811118020""",2.446333e+06,2023-08-29 12:06:00,25,1
4,"=""08111393992""",1.680000e+06,2023-06-26 23:24:00,4,65
...,...,...,...,...,...
1325,"=""08998389654""",1.570000e+06,2023-08-26 14:02:00,10,4
1326,"=""08999061396""",5.614498e+05,2023-08-05 11:05:00,3,25
1327,"=""08999351635""",5.310000e+05,2023-07-11 06:43:00,3,51
1328,"=""08999996239""",2.655067e+06,2023-08-10 07:48:00,38,21


In [73]:
df4.to_excel("Customer Lazada.xlsx", index = False)

In [ ]:
## Data Kak Hen

In [15]:
df = data_all[(data_all['True datetime'] > '2023-01-01') & (data_all['True datetime'] < '2023-09-01') &
              data_all['Order Status'].isin(osf['order filter'].unique()) &
              data_all['Brand'].isin(['TS'])]
df

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Shipped Date,Note,Ready to Ship Date,Completed Date,Cancelled Reason,Item Note,Shipping Fee (Cashless),Picklist ID,Package ID,Shipment ID
176,1000109597,NaN,NTRM1000109597,NaN,delivered,2023-02-01 00:47:00,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,1000109597,NaN,NTRM1000109597,NaN,delivered,2023-02-01 00:47:00,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,1000109598,NaN,0125432300000234,NaN,completed_with_awb,2023-02-01 00:51:00,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,1000109599,NaN,0125432300000242,NaN,completed_with_awb,2023-02-01 03:58:00,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,1000109600,NaN,0125432300000259,NaN,completed_with_awb,2023-02-01 08:01:00,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573989,Nubi -ECOM RETAIL - EMOS,Nubi -ECOM RETAIL - EMOS,,1.0,Delivered,2023-08-01 00:00:00,31.0,1.0,August,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573990,Nubi -ECOM RETAIL - EMOS,Nubi -ECOM RETAIL - EMOS,,3.0,Delivered,2023-08-03 00:00:00,31.0,3.0,August,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573991,Nubi -ECOM RETAIL - EMOS,Nubi -ECOM RETAIL - EMOS,,1.0,Delivered,2023-08-01 00:00:00,31.0,1.0,August,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573992,Nubi -ECOM RETAIL - EMOS,Nubi -ECOM RETAIL - EMOS,,3.0,Delivered,2023-08-03 00:00:00,31.0,3.0,August,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
prod = ['Tropicana Slim Collagen Drink Strawberry 200 gram','Tropicana Slim 7 Fruits Fiber Daily 12 sachet',
        'Tropicana Slim RTD Almond Drink Chocolicious 190 ml','Tropicana Slim Oat Drink 190 ml (RTD)', 'Tropicana Slim RTD Oat Drink Melon 190 ml',
        'Tropicana Slim Milk Low Fat Vanilla 500gr','Tropicana Slim Low Fat Milk Korean Strawberry 500g','Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram']

df1 = df[df['Real Nama Produk'].isin(prod) & df['Store'].isin(['Tokopedia','TikTok','Shopee','Lazada','Nutrimart','Blibli','Bukalapak'])]
df1['Unique Cust'] = df1['Customer Name'] + '_' + df1['Phone']

C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_16816\3392195210.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Unique Cust'] = df1['Customer Name'] + '_' + df1['Phone']


In [110]:
for i in df1.columns :
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4

In [17]:
kahen = df1[['Order #','Store','Date','Month','Year','Real SKU','Real Nama Produk','Unique Cust', 'Customer Name','Phone',
             'Qty. Invoiced', 'Total Net Before PPN', 'Real Region', 'City','Address']]

In [125]:
kahen[kahen['Store'].isin(['Shopee'])]

,Order #,Store,Date,Month,Year,Real SKU,Real Nama Produk,Unique Cust,Customer Name,Phone,Qty. Invoiced,Total Net Before PPN,Real Region,City,Address
438,2301028FG081CD,Shopee,1.0,January,2023.0,2105084180,Tropicana Slim Milk Low Fat Vanilla 500gr,"C******a_=""88""",C******a,"=""88""",1.0,70000.0,Jawa Timur,Kota Surabaya,"Jalan Nginden VI D no. 01 (Kos putri), KOTA SU..."
478,2301028F745M1X,Shopee,1.0,January,2023.0,2105028180,Tropicana Slim Low Fat Milk Korean Strawberry ...,"L******i_=""84""",L******i,"=""84""",1.0,72000.0,Jabodetabek,Kota Jakarta Barat,"Jalan Tosiga III Dalam No.63, RT.1/RW.4, Kebon..."
504,2301028EK70V9T,Shopee,1.0,January,2023.0,2105084180,Tropicana Slim Milk Low Fat Vanilla 500gr,"S******a_=""58""",S******a,"=""58""",1.0,70000.0,Kalimantan,Kota Pontianak,"Jl. Imam bonjol no 2/134, toko bangunan PD Mit..."
505,2301028EK70V9T,Shopee,1.0,January,2023.0,2105055180,Tropicana Slim Susu Low Fat Macchiato Coffee 5...,"S******a_=""58""",S******a,"=""58""",1.0,70000.0,Kalimantan,Kota Pontianak,"Jl. Imam bonjol no 2/134, toko bangunan PD Mit..."
605,2301028CPAKT99,Shopee,1.0,January,2023.0,2105028180,Tropicana Slim Low Fat Milk Korean Strawberry ...,"G**n_=""09""",G**n,"=""09""",1.0,72000.0,Jabodetabek,Kota Jakarta Timur,Jl Delima IV No 5C RT 05 RW 08 Curug Pondok Ke...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535921,2309233RWRXU5Y,Shopee,23.0,September,2023.0,2T00805043,Tropicana Slim Collagen Drink Strawberry 200 gram,"L******P_=""******71""",L******P,"=""******71""",1.0,165000.0,Jawa Tengah,Kabupaten Sleman,"Nglengkong Sukoharjo, RT.1/RW.24, Suko Harjo, ..."
1535931,2309233RVC4GQX,Shopee,23.0,September,2023.0,2105028180,Tropicana Slim Low Fat Milk Korean Strawberry ...,"S******K_=""******56""",S******K,"=""******56""",1.0,76000.0,Jawa Tengah,Kabupaten Semarang,"Dsn. Kedesen Rt. 01/Rw.06, Ds. Kradenan, Kec. ..."
1536068,2309233P5J2BAK,Shopee,23.0,September,2023.0,2T00805043,Tropicana Slim Collagen Drink Strawberry 200 gram,"N**i_=""******64""",N**i,"=""******64""",1.0,165000.0,Jabodetabek,Kota Tangerang Selatan,"Pondok Maharta V, Blok a 14 No.18, RT.16/RW.10..."
1536271,2309233FMVDESV,Shopee,23.0,September,2023.0,2105084180,Tropicana Slim Milk Low Fat Vanilla 500gr,"S****h_=""******20""",S****h,"=""******20""",2.0,148000.0,Jawa Barat,Kota Cimahi,"Jalan Kebon Kopi no.17/186, RT 06 RW 09, Kelur..."


In [18]:
kahen.to_excel('Kahen.xlsx', index = False)

In [99]:
data_all['Channel'].unique()

array(['L-Men Store Blibli', 'Nutrimart', 'Tokopedia', 'Shopee', 'FBL',
       'Blibli', 'TikTok', 'Lazada', 'JD Indonesia', 'Bukalapak',
       'Aladin Mall', 'Order Online Surabaya', 'Order Online Sumsel',
       'Order Online Jateng', 'Order Online Bali', 'Order Online Medan',
       'Order Online Lampung', 'Order Online Pekanbaru',
       'Order Online Jabar', 'Order Online Makassar',
       'Order Online Samarinda', 'Order Online Banjarmasin',
       'Orami Shopping', 'GENERAL', 'GARUDA TIMUR PACIFIC, PT',
       'SHOPEE INTERNATIONAL INDONESIA, PT', 'TANI SUPPLY INDONESIA, PT',
       'WICAKSANA OVERSEAS INDONESIA, PT', 'RITEL BERSAMA NASIONAL, PT',
       'WARUNG PINTAR DISTRIBUSI, PT',
       'ENSEVAL PUTERA MEGATRADING TBK, PT', 'TEMPO, PT',
       'KREASI TANI LAKSMI, PT', 'BINTANG ASET INDONESIA, PT',
       'CHILIBELI BAGUS INDONESIA, PT', 'NATA GERAI ERABARU, PT',
       'KONEKSI NIAGA SOLUSINDO, PT', 'PASKOMNAS NIAGA UTAMA, PT',
       'RITEL BERSAMA NASIONAL', 'ENSEVAL P

In [100]:
df = data_all[data_all['Store Type'].isin(['Marketplace']) & (~data_all['Store'].isin(['Orami Shopping'])) & (~data_all['Channel'].isin(['L-Men Store Blibli'])) & data_all['Bundle Name'].isnull() & (data_all['Year']).isin([2023])]

In [104]:
df1 = df.groupby(['Month','Store']).agg({'Order #' : 'nunique'}).reset_index()

In [105]:
df1.to_excel('forstok.xlsx', index = False)

In [ ]:
#BLIBLI

In [42]:
df = data_all[data_all['Store'].isin(['Blibli'])]

In [50]:
df1 = df[df['Channel'].isin(['Blibli'])]

In [55]:
df2 = df1.groupby(['Phone','Month','Order #']).agg({'Total Net Before PPN' : 'sum'}).reset_index()
df2 =df2[~df2['Phone'].isin(['=""', '="nan"'])]

In [69]:
df3 = df2.groupby(['Month','Phone']).agg({'Total Net Before PPN' : 'mean'}).reset_index()
df3

df3['Month_N'] = df3['Month'].apply(lambda x : 9 if x == "September" else
                                            (10 if x == "October" else
                                            (11 if x == "November" else 
                                            (12 if x == "December" else
                                            (1 if x == "January" else
                                            (2 if x == "February" else
                                            (3 if x == "March" else
                                            (4 if x == "April" else
                                            (5 if x == "May" else
                                            (6 if x == "June" else
                                            (7 if x == "July" else 8)))))))))))
df3

,Month,Phone,Total Net Before PPN,Month_N
0,April,"=""0213160455""",292000.0,4
1,April,"=""08111020124""",74000.0,4
2,April,"=""08111029855""",595200.0,4
3,April,"=""08111032966""",127000.0,4
4,April,"=""08111082494""",2420000.0,4
...,...,...,...,...
13296,September,"=""08999204542""",7500.0,9
13297,September,"=""08999304012""",292000.0,9
13298,September,"=""08999760200""",234000.0,9
13299,September,"=""08999911192""",25000.0,9


In [70]:
df4 = df[~df['Channel'].isin(['Blibli'])]
df5 = df4.groupby(['Month','Order #']).agg({'Total Net Before PPN' : 'sum'}).reset_index()
df5['Month_N'] = df5['Month'].apply(lambda x : 9 if x == "September" else
                                            (10 if x == "October" else
                                            (11 if x == "November" else 
                                            (12 if x == "December" else
                                            (1 if x == "January" else
                                            (2 if x == "February" else
                                            (3 if x == "March" else
                                            (4 if x == "April" else
                                            (5 if x == "May" else
                                            (6 if x == "June" else
                                            (7 if x == "July" else 8)))))))))))
df5

,Month,Order #,Total Net Before PPN,Month_N
0,April,12127573992,580000.0,4
1,April,12127574775,290000.0,4
2,April,12127581930,945600.0,4
3,April,12127584455,192000.0,4
4,April,12127593315,2292000.0,4
...,...,...,...,...
1941,September,12139349149,300000.0,9
1942,September,12139352252,528000.0,9
1943,September,12139361182,792000.0,9
1944,September,12139365049,300000.0,9


In [71]:
with pd.ExcelWriter("Persebaran Data Blibli.xlsx") as writer:
    df3.to_excel(writer, sheet_name="NM",index = False)
    df5.to_excel(writer,sheet_name="LMS",index = False)

In [ ]:
#TRISL UNBUNDLING J

In [42]:
data_all[data_all['Real SKU'].str.contains('J', na= False, case = False)][['Real SKU','Real Nama Produk']].drop_duplicates()

,Real SKU,Real Nama Produk
1409,(J)71110130,Parsel Tropicana Slim Chill Out Kit - 2022
24914,(J)71110131,Paket NutriSari 35 Rasa
204896,(J)71110133,Paket Valentine NutriSari Squeezed Orange - Fe...
406104,(J)71110138,Box Packing NutriMart
409676,(J)71110134,Paket Sobat Melek Lokalate (Segitiga)
425955,(J)71110146,Hampers Exclusive Ramadan - Tropicana Slim Pac...
781642,(J)71110147,SPECIAL 6.6 - Brand Box Nutrimart
917902,(J)71110143,Green Packing - L-Men Hi Protein 2 Go Chocolat...
921750,(J)71110141,Green Packing - Tropicana Slim Oat Drink 190 m...
922185,(J)71110144,Green Packing - HiLo School Chocolate Ready To...


In [ ]:
#Analisis Lazada

In [81]:
df = data_all[data_all['Store'].isin(['Lazada']) & data_all['Order Status'].isin(osf['order filter'].unique())]

In [82]:
laz = df.groupby(['Order #','Year','Month','Date','Store Type','Store','Channel','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Customer Name','Customer Email','Phone','Payment Channel','City','Real Region'], dropna = False).agg({'Qty. Invoiced':'sum','Total Net Before PPN':'sum','Total Net' : 'sum','Total' : 'sum'}).reset_index()

In [86]:
#DATA TAHUN 2022
import numpy as np
import pandas as pd
import xlwt
import datetime as dt
pd.options.mode.chained_assignment = None
import datetime
from datetime import date

#Read CSV
df22 = pd.read_csv(r"D:\Masterdata\Clean Data\data_all_20 February With Order Online.csv", sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})

df22['True datetime']=pd.to_datetime(df22['True datetime'])
df22['Real SKU']=df22['Real SKU'].astype(str)
df22['Phone']=df22['Phone'].str.extract('(\d+)')
df22 = df22[df22['Year'].isin([2022])]
df22 = df22[['Order #','Sales Order ID','Order Status','Order date','True datetime','Date','Month','Year','Store Type','Store','Customer Name','Phone','Brand','Sub Brand','Real SKU','Real Nama Produk','Parent Item','Bundle Name','Qty. Invoiced','Total Net Before PPN','Total']]

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Masterdata\\Clean Data\\data_all_20 February With Order Online.csv'

In [22]:
for i in data_all.columns :
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4

In [33]:
data_all[data_all['Store'].isin(['Lazada'])]['Region Group'].unique()

array(['Jawa Timur', 'Jawa Barat', 'Jabodetabek', 'Di Yogyakarta',
       'Kalimantan Tengah', 'Jawa Tengah', 'Kalimantan Timur',
       'Sumatera Utara', 'Riau', 'Sulawesi Tengah', 'Sumatera Selatan',
       'Jambi', 'Lampung', 'Bali', 'Nusa Tenggara Barat',
       'Sumatera Barat', 'Kalimantan Barat', 'Maluku Utara',
       'Sulawesi Selatan', 'Kepulauan Riau', 'Kepulauan Bangka Belitung',
       'Papua', 'Kalimantan Selatan', 'Aceh', 'Maluku',
       'Kalimantan Utara', 'Sulawesi Tenggara', 'Bengkulu',
       'Sulawesi Barat', 'Papua Barat', 'Nusa Tenggara Timur',
       'Sulawesi Utara', 'Gorontalo'], dtype=object)

In [5]:
len(ph2)

8

In [8]:
ph = data_all[data_all['Region Group'].str.contains('Sulawesi Selatan', case = False, na = False) &
         data_all['Store'].isin(['Lazada'])&
         (data_all['True datetime'] > '2023-01-01')&
         (data_all['True datetime'] < '2023-12-01')&     
         data_all['Brand'].isin(['HiLo','TS','L-Men','WDANK'])]['Phone'].unique()

In [11]:
active = data_all[data_all['Phone'].isin(ph) &
                 data_all['Store'].isin(['Lazada'])&
                 data_all['Year'].isin([2023])].groupby('Phone').agg({'Order #' : 'nunique'}).reset_index()
ph2 = active[active['Order #'] > 3][['Phone']].drop_duplicates()    

In [69]:
diamond = data_all[data_all['Phone'].isin(ph2) &
                 data_all['Store'].isin(['Lazada'])&
                 data_all['Year'].isin([2023])].groupby('Phone').agg({'Total' : 'sum'}).reset_index()
ph3 = diamond[(diamond['Total'] > 2000000)]['Phone'].unique()  

In [65]:
phs = data_all[data_all['Region Group'].str.contains('Sulawesi Selatan|South Sulawesi', case = False, na = False) &
         data_all['Store'].isin(['Lazada']) &
         (data_all['True datetime'] > '2023-01-01')&
         data_all['Brand'].isin(['HiLo','TS','L-Men','WDANK'])]['Phone'].unique()
len(phs)

113

In [72]:
phone = np.concatenate((phs, ph3))
len(phone)

416

In [79]:
df = data_all[data_all['Phone'].isin(phone)].groupby(['Phone','Region Group']).agg({'True datetime' : 'max',
                                                              'Order #' : 'nunique',
                                                              'Total' : 'sum'}).reset_index()
df

,Phone,Region Group,True datetime,Order #,Total
0,08113084107,Jawa Timur,2023-11-12 07:00:00,9,2382100.0
1,08113253000,Jawa Timur,2023-11-27 08:09:00,102,42136540.0
2,0811410783,Sulawesi Selatan,2023-05-06 17:53:00,1,313300.0
3,081216247983,Jawa Timur,2023-09-27 08:57:00,8,2737200.0
4,08121628388,Jawa Timur,2023-11-15 22:41:00,93,96068600.0
...,...,...,...,...,...
418,08981574760,Sulawesi Selatan,2023-03-10 14:42:00,1,73700.0
419,08982928314,Jawa Timur,2023-10-10 00:05:00,7,2921200.0
420,08986354541,Jawa Timur,2023-09-25 04:26:00,8,3006700.0
421,08990140094,Jawa Timur,2023-08-26 20:04:00,26,11422000.0


In [12]:
with pd.ExcelWriter("HP Laz Makasar.xlsx") as writer:
    ph2.to_excel(writer, sheet_name="NM",index = False)

In [54]:
data_all[data_all['Store'].isin(['Nutrimart']) & (data_all['True datetime'] >= "2023-11-28")
         & data_all['Brand'].isin(['NS','TS','HiLo','L-Men','WDANK'])]

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Note,Ready to Ship Date,Completed Date,Cancelled Reason,Item Note,Shipping Fee (Cashless),Picklist ID,Package ID,Shipment ID,Bundle SKU
1861976,1000118049,NaN,,NaN,ready_to_ship,2023-11-28 08:42:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861978,1000118050,NaN,,NaN,pending_payment,2023-11-28 09:56:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861980,1000118051,NaN,,NaN,processing,2023-11-28 09:58:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861982,1000118052,NaN,,NaN,processing,2023-11-28 10:05:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861984,1000118052,NaN,,NaN,processing,2023-11-28 10:05:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861986,1000118052,NaN,,NaN,processing,2023-11-28 10:05:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861988,1000118052,NaN,,NaN,processing,2023-11-28 10:05:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861990,1000118052,NaN,,NaN,processing,2023-11-28 10:05:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861992,1000118052,NaN,,NaN,processing,2023-11-28 10:05:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861995,1000118052,NaN,,NaN,processing,2023-11-28 10:05:00,48.0,28.0,November,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


array(['02/01/2023 09:21', '02/01/2023 08:19', '01/01/2023 23:38', ...,
       Timestamp('2023-11-09 00:00:00'), Timestamp('2023-11-16 00:00:00'),
       Timestamp('2023-12-01 00:00:00')], dtype=object)

In [74]:
#Mangoove
df = data_all[data_all['Store'].isin(['Lazada','Shopee','TikTok','Tokopedia','Nutrimart','Blibli','Bukalapak']) &
              data_all['Order Status'].isin(osf['order filter'].unique()) &
              data_all['Brand'].isin(['NS','TS','WDANK','HiLo','Lokalate'])]
df['datetime'] = df['True datetime'].astype(str)
df = df[df['datetime'].str.contains('2023-11-11|2023-12-12|2023-10-10', case = False, na = False)]


In [75]:
df1 = df.groupby(['Month','Store','Order #']).agg({'Total' : 'sum'}).reset_index()
df1

,Month,Store,Order #,Total
0,December,Blibli,12144450802,47000.0
1,December,Blibli,12144455080,560785.0
2,December,Blibli,12144456902,642600.0
3,December,Blibli,12144457003,213000.0
4,December,Blibli,12144457500,142002.0
...,...,...,...,...
32996,October,Tokopedia,INV/20231010/MPL/3503928802,82100.0
32997,October,Tokopedia,INV/20231010/MPL/3503929861,69300.0
32998,October,Tokopedia,INV/20231010/MPL/3503929913,23300.0
32999,October,Tokopedia,INV/20231010/MPL/3503934947,317400.0


In [76]:
df2 = df1[df1['Total'] >= 100000].groupby(['Store']).agg({'Order #' : 'nunique', 'Total' : 'sum'}).reset_index()#.style.format({"Total": "{:.2f}"})
df2['Total'] = df2['Total'].astype(int)
df2.style.format({"Order #" : "{:,d}","Total" : "{:,d}"})

,Store,Order #,Total
0,Blibli,259,"72,449,110"
1,Bukalapak,5,"740,700"
2,Lazada,"7,581","2,140,297,320"
3,Nutrimart,13,"23,272,394"
4,Shopee,"5,845","1,141,436,884"
5,TikTok,92,"14,639,355"
6,Tokopedia,"2,132","413,116,246"


In [77]:
df2['Order #'].sum()

15927

In [78]:
df2['Total'].sum()

3805952009

In [57]:
df['datetime'].unique()

array(['2023-10-10 00:00:00', '2023-10-10 23:59:00',
       '2023-10-10 23:58:00', ..., '2023-12-12 03:13:00',
       '2023-12-12 01:46:00', '2023-12-12 01:44:00'], dtype=object)

In [ ]:
#Ci Evelyn

In [9]:
osf=pd.read_excel(r"data_supp\order filter.xlsx")
df = data_all[data_all['Store'].isin(['Lazada','Shopee']) &
              data_all['Order Status'].isin(osf['order filter'].unique()) &
              data_all['Brand'].isin(['NS','TS','WDANK','HiLo','L-Men']) &
              (data_all['True datetime'] >= '2023-06-01')]
orderNS = data_all[data_all['Brand'].isin(['NS'])]['Order #'].unique()
df = df[~df['Order #'].isin(orderNS)]

In [10]:
##SHOPEE
dfs = df[df['Store'].isin(['Shopee'])]
orders = dfs['Order #'].nunique()
totals = dfs['Total'].sum()
aovs = totals/orders
aovs

orders = dfs.groupby('Order #').agg({'Total' : 'sum'}).reset_index()
orders = orders[orders['Total'] > aovs]['Order #'].unique()

In [11]:
##Lazada
dfl = df[df['Store'].isin(['Lazada'])]
orderl = dfl['Order #'].nunique()
totall = dfl['Total'].sum()
aovl = totall/orderl
aovl

orderl = dfl.groupby('Order #').agg({'Total' : 'sum'}).reset_index()
orderl = orderl[orderl['Total'] > aovl]['Order #'].unique()

In [6]:
order = np.concatenate((orders, orderl))
order

array(['2306017BBHYMHW', '2306017BDWADY9', '2306017BF2FM0K', ...,
       '1340582734667883', '1340599144156819', '1340600909359390'],
      dtype=object)

In [12]:
eve = df[df['Order #'].isin(order)][['Month','Store','Brand','Real Nama Produk','Qty. Invoiced']]
with pd.ExcelWriter("Ci Eve.xlsx") as writer:
    eve.to_excel(writer, index = False)

In [13]:
print(aovs)
print(aovl)

131888.56502567287
137631.3005585275


In [9]:
hut = pd.read_excel(r"C:\Users\steven.nathanael\OneDrive - PT Nutrifood Indonesia\hut.xlsx")
hut

,SKU
0,2HH1409249
1,2HG0103050
2,2HC0113021
3,2HC0112021
4,2101609180
5,2LH1204172
6,2LC0903181
7,2304515112
8,2304008180
9,2LM0801029


In [10]:
df1 = data_all[data_all['Real SKU'].isin(hut['SKU'])]
df1

,Order #,Sales Order ID,AWB,Paid Date,Order Status,Order date,Week,Date,Month,Quarter,...,Note,Ready to Ship Date,Completed Date,Cancelled Reason,Item Note,Shipping Fee (Cashless),Picklist ID,Package ID,Shipment ID,Bundle SKU
27,12121427924,12170372672,BLIRA0000049663675,NaN,Delivered,02/01/2023 11:49,1.0,2.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,12121553450,12170552392,nan,NaN,Order Batal,03/01/2023 23:00,1.0,3.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,12121660419,12170707342,BLI1411228674113,NaN,Delivered,05/01/2023 12:53,1.0,5.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,12121969124,12171149264,BLI2209226677605,NaN,Delivered,09/01/2023 23:29,2.0,9.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,12122067474,12171289228,BLI2209227928380,NaN,Sudah Settlement,11/01/2023 10:24,2.0,11.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425036,Nubi -ECOM RETAIL - TOKO NOW,Nubi -ECOM RETAIL - TOKO NOW,,13.0,Delivered,2024-01-13 00:00:00,2.0,13.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2425037,Nubi -ECOM RETAIL - TOKO NOW,Nubi -ECOM RETAIL - TOKO NOW,,13.0,Delivered,2024-01-13 00:00:00,2.0,13.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2425043,Nubi -ECOM RETAIL - TOKO NOW,Nubi -ECOM RETAIL - TOKO NOW,,13.0,Delivered,2024-01-13 00:00:00,2.0,13.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2425050,Nubi -ECOM RETAIL - TOKO NOW,Nubi -ECOM RETAIL - TOKO NOW,,13.0,Delivered,2024-01-13 00:00:00,2.0,13.0,January,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
df1[(df1['True datetime'] >= '2023-11-01')&
         (df1['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Hour','Store Type','Store','Channel','Warehouse Name','Real Region','City','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Exported Parent Item'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index().to_excel(f'D:\Masterdata\Masterdata Lite\sales hut.xlsx',index=False)
# df1.groupby(['Year','Month','Date','Store Type','Store','Channel','Order Status'])[['Order #']].nunique().reset_index().to_excel(f'Cek order gerak {date.today()}.xlsx',index=False)